In [ ]:
""

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!fusermount -u drive
!google-drive-ocamlfuse drive

In [ ]:
!pip install --upgrade tensorflow-gpu==2.9.1

In [ ]:
# from unet_model_with_functions_of_blocks import build_unet
from __future__ import division
from keras.models import Model

import tensorflow as tf

from tensorflow.keras.layers import Input, concatenate, Conv2D, MaxPooling2D, UpSampling2D, Reshape, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, ReduceLROnPlateau, CSVLogger, EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import *
from tensorflow.keras.layers import *
import numpy as np
from tensorflow.keras import callbacks
import pickle
import os
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from glob import glob

import os
import random
import numpy as np

from tqdm import tqdm

from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
import cv2 as cv
import h5py
import pandas as pd

from PIL import Image
from PIL import ImageDraw


from csv import DictReader
from pathlib import Path


In [ ]:
!python --version

In [ ]:
import tensorflow
tensorflow.__version__

In [ ]:
import keras
keras.__version__

In [ ]:
!nvidia-smi

In [ ]:
os.environ['PYTHONHASHSEED'] = str(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
!zip Standard.zip '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model'

In [ ]:
!apt-get install rar

In [ ]:
!rar a '/content/drive/MyDrive/Test DDSM 185 Images/StandardModel.rar' '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model'

In [ ]:
!rar a '/content/drive/MyDrive/Test DDSM 185 Images/without2skipconnection.rar' '/content/drive/MyDrive/Test DDSM 185 Images/without 2 skip connection'

### Load data

In [ ]:
!unrar x "/content/drive/MyDrive/Mammo dataset/New Dataset/New_Training_20k_bkg.rar"
!unrar x "/content/drive/MyDrive/Mammo dataset/New Dataset/New_Validation_3k_bkg.rar"

In [ ]:
batch_size = 32
learning_rate = 1e-4
epochs = 100
height = 256
width= 256

In [ ]:
drive_path = '/content/drive/MyDrive'
dataset_path = '/content'

files_dir = os.path.join(drive_path, 'Colab Notebooks', 'files_15_5_23', 'training_dataset_20_bkg')
model_file = os.path.join(files_dir,'unet_effb0-new-data-20_bkg-15_5_2023.h5')
log_file = os.path.join(files_dir,'log_new-data-bkg_15-5_v1.csv')

In [ ]:
def create_dir(path):
  if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
create_dir(files_dir)

In [ ]:
def load_data(path):
  train_x = sorted(glob(os.path.join(path, 'New_Training_20k_bkg', 'Patch', '*')))
  train_y = sorted(glob(os.path.join(path, 'New_Training_20k_bkg', 'Mask', '*')))

  valid_x = sorted(glob(os.path.join(path, 'New_Validation_3k_bkg', 'Patch', '*')))
  valid_y = sorted(glob(os.path.join(path, 'New_Validation_3k_bkg', 'Mask', '*')))

  return (train_x, train_y), (valid_x, valid_y)

In [ ]:
def read_image(path):
  path = path.decode()
  x = cv.imread(path).astype(np.float32)
  x = x/255.0
  return x

In [ ]:
def read_mask(path):
  path = path.decode()
  x = cv.imread(path, 0).astype(np.bool_)
  # x = x/255.0
  x = np.expand_dims(x, axis = -1)
  return x

In [ ]:
def tf_parse(x,y):
  def _parse(x,y):
    x = read_image(x)
    y = read_mask(y)
    return x, y

  x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.bool])
  x.set_shape([height, width, 3])
  y.set_shape([height, width, 1])

  return x, y

In [ ]:
def tf_dataset(x, y, batch = 8):
  dataset = tf.data.Dataset.from_tensor_slices((x,y))
  dataset = dataset.map(tf_parse, num_parallel_calls= tf.data.AUTOTUNE)
  dataset = dataset.batch(batch)
  dataset = dataset.prefetch(tf.data.AUTOTUNE)
  return dataset

In [ ]:
(train_x, train_y), (valid_x, valid_y) = load_data(dataset_path)
print(f'TRAIN:{len(train_x)}')
print(f'VALID:{len(valid_x)}')

In [ ]:
train_dataset = tf_dataset(train_x, train_y, batch = batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch = batch_size)

In [ ]:
n = 1
for x, y in valid_dataset:
  if n in [1,5, 7, 8, 9]:
  # print(x.shape, y.shape)
  # print(y[0].dtype)
    plt.subplot(1,2,1)
    plt.imshow(x[0], cmap='gray')
    plt.subplot(1,2,2)
    plt.imshow(np.squeeze(y[0]), cmap='gray')
    plt.show()
  n = n + 1

### Model and Training

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Concatenate, MaxPooling2D, BatchNormalization, Activation, DepthwiseConv2D, SeparableConv2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def decoder_block(inputs, skip, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip])
    x = conv_block(x, num_filters)

    return x

def efficientnetb0_unet(input_shape):
    """ Input """
    inputs = Input(input_shape)

    """ Pre-trained Encoder """
    encoder = EfficientNetB0(include_top=False, weights="imagenet", input_tensor=inputs)

    s1 = encoder.get_layer("input_1").output                      ## 256
    s2 = encoder.get_layer("block2a_expand_activation").output    ## 128
    s3 = encoder.get_layer("block3a_expand_activation").output    ## 64
    s4 = encoder.get_layer("block4a_expand_activation").output    ## 32

    """ Bottleneck """
    b1 = encoder.get_layer("block6a_expand_activation").output    ## 16

    """ Decoder """
    d1 = decoder_block(b1, s4, 512)                               ## 32
    d2 = decoder_block(d1, s3, 256)                               ## 64
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)                                ## 128

    """ Output """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="EfficientNetB0_UNET")
    return model

In [ ]:
input_shape = (height, width, 3)
model = efficientnetb0_unet(input_shape)
model.summary()

In [ ]:
def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)

def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def iou_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(tf.cast(y_true, tf.float32), y_pred) + 0.5 * dice_loss(tf.cast(y_true, tf.float32), y_pred)

In [ ]:
def segment_sensitivity(y,y_pred):
  bitwise_and_roi_patch = np.logical_and(np.squeeze(y), np.squeeze(y_pred))
  segment_sensitivity = round(np.sum(bitwise_and_roi_patch) / np.sum(np.squeeze(y)), 2)
  return segment_sensitivity

In [ ]:
opt = Adam(learning_rate)

model.compile(loss = 'binary_crossentropy', optimizer=opt, metrics = ['accuracy'])

In [ ]:
callbacks = [
    ModelCheckpoint(filepath = model_file, verbose = 1, save_best_only= True),
    ReduceLROnPlateau(factor=0.1, patience=5, verbose=1),
    CSVLogger(log_file),
    EarlyStopping(monitor = 'val_loss', patience = 20,verbose=1)
]

In [ ]:
learning_rate = 1.0000e-10

In [ ]:
import numpy as np
from numpy.testing import assert_allclose
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.callbacks import ModelCheckpoint

# load the model
new_model = load_model(model_file
                       )

# fit the model
new_model.fit(
    train_dataset,
    validation_data= valid_dataset,
    epochs= epochs,
    callbacks= callbacks
)

In [ ]:
from keras.losses import binary_crossentropy
from tensorflow.keras.metrics import Precision, Recall, MeanIoU

model.fit(
    train_dataset,
    validation_data= valid_dataset,
    epochs= epochs,
    callbacks= callbacks
)

#### plot loss and accuracy diagram

In [ ]:
import pandas as pd

data = pd.read_csv(log_file)
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.plot(data["epoch"],data["accuracy"])
plt.plot(data["epoch"],data["val_accuracy"])
plt.legend(['train_acc', 'val_acc'], loc='lower right')
plt.show()

In [ ]:
plt.xlabel("epoch")
plt.ylabel("loss")
plt.plot(data["epoch"],data["loss"])
plt.plot(data["epoch"],data["val_loss"])
plt.legend(['train_loss', 'val_loss'], loc='upper right')
plt.show()

In [ ]:
plt.xlabel("epoch")
plt.ylabel("loss")
plt.plot(data["epoch"][7:],data["loss"][7:])
plt.plot(data["epoch"][7:],data["val_loss"][7:])
plt.legend(['train_loss', 'val_loss'], loc='upper right')
plt.show()

In [ ]:
files_dir_2nd = os.path.join(drive_path, 'Colab Notebooks', 'files_15_5_23', 'training_dataset_20_bkg_unet_without_skip')
model_file_2nd = os.path.join(files_dir_2nd,'unet_effb0-new-data-20_bkg-without-skip.h5')
log_file_2nd = os.path.join(files_dir_2nd,'log_new-data-bkg_15-5-without-skip_v2.csv')

In [ ]:
files_dir = os.path.join(drive_path, 'Colab Notebooks', 'files_15_5_23', 'training_dataset_20_bkg')
model_file = os.path.join(files_dir,'unet_effb0-new-data-20_bkg-15_5_2023.h5')
log_file = os.path.join(files_dir,'log_new-data-bkg_15-5.csv')

In [ ]:
import pandas as pd

data = pd.read_csv(log_file)
data_2nd = pd.read_csv(log_file_2nd)
plt.xlabel("epoch")
plt.ylabel("loss")
plt.plot(data["epoch"][10:],data["loss"][10:], color = 'blue')
plt.plot(data["epoch"][10:],data["val_loss"][10:], color = 'green')

plt.plot(data_2nd["epoch"][10:],data_2nd["loss"][10:], color = 'red')
plt.plot(data_2nd["epoch"][10:],data_2nd["val_loss"][10:], color = 'orange')
plt.legend(['train_loss_1', 'val_loss_1', 'train_loss_2', 'val_loss_2'], loc='upper right')
plt.show()

In [ ]:
data = pd.read_csv(log_file)
data_2nd = pd.read_csv(log_file_2nd)
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.plot(data["epoch"][10:],data["accuracy"][10:], color = 'blue')
plt.plot(data["epoch"][10:],data["val_accuracy"][10:], color = 'green')

plt.plot(data_2nd["epoch"][10:],data_2nd["accuracy"][10:], color = 'red')
plt.plot(data_2nd["epoch"][10:],data_2nd["val_accuracy"][10:], color = 'orange')
plt.legend(['train_acc_1', 'val_acc_1', 'train_acc_2', 'val_acc_2'], loc='lower right')
plt.show()

In [ ]:
data = pd.read_csv(log_file)
plt.xlabel("epoch")
# plt.ylabel("accuracy")
plt.plot(data["epoch"][10:],data["accuracy"][10:])
plt.plot(data["epoch"][10:],data["loss"][10:])
plt.legend(['train_acc', 'train_loss'], loc='lower right')
plt.show()

In [ ]:
def cal_overlap(x,y,y_pred):
  bitwise_and_roi_patch = np.logical_and(np.squeeze(y), np.squeeze(y_pred))
  overlap = round(np.sum(bitwise_and_roi_patch) / np.sum(np.squeeze(y)), 2)
  return overlap

In [ ]:
def show_image_roi_predict1_img(index, x, y, predict):
    overlap = cal_overlap(x[index], y[index], predict)
    print (overlap)

    img = x[index]

    plt.subplot(131)
    plt.imshow(img)
    plt.title('Image' + str(index))
    plt.axis('off')

    roi = np.squeeze(y[index])
    plt.subplot(132)
    plt.axis('off')
    plt.imshow(roi, 'gray')
    plt.title('ROI ' + str(index))

    roi_pred = np.squeeze(predict)
    plt.subplot(133)
    plt.axis('off')
    plt.imshow(roi_pred)
    plt.title('ROI pred ' + str(index))

    plt.show()


#### predict

In [ ]:
preds_test = model.predict(valid_dataset, verbose=1)
test_set_y_pred = (preds_test > 0.5).astype(np.float32)

In [ ]:
i = 0
for x, y in valid_dataset:
    show_image_roi_predict1_img(5, x, y, test_set_y_pred[(i)*batch_size + 5])
    i = i + 1

### Multi Tumors Testing (CBIS-DDSM)

In [ ]:
!unrar x "/content/drive/MyDrive/Mammo dataset/New Dataset/CBIS Test set_preprocessing.rar"

In [ ]:
!unrar x "/content/drive/MyDrive/Mammo dataset/New Dataset/CBIS Test set - CLAHE.rar"


In [ ]:
!pip install patchify

In [ ]:
drive_path = '/content/drive/MyDrive'
dataset_path = '/content'

files_dir = os.path.join(drive_path, 'Colab Notebooks', 'files_15_5_23', 'training_dataset_20_bkg')
model_file = os.path.join(files_dir,'unet_effb0-new-data-20_bkg-15_5_2023.h5')

# test_x = sorted(glob(os.path.join(dataset_path, 'CBIS Test set_preprocessing', 'Image', '*')))
# test_y = sorted(glob(os.path.join(dataset_path, 'CBIS Test set_preprocessing', 'Mask', '*')))

test_x_clahe = sorted(glob(os.path.join(dataset_path, 'CBIS Test set - CLAHE', 'Image', '*')))
test_y_clahe = sorted(glob(os.path.join(dataset_path, 'CBIS Test set - CLAHE', 'Mask', '*')))

In [ ]:
import pandas as pd
df = pd.DataFrame({'img': test_x_clahe, 'roi': test_y_clahe})
df.to_csv ('/content/drive/MyDrive/Mammo dataset/New Dataset/cbis_test_clahe_multi_rois.csv', index = False, header=True)

In [ ]:
def show_y_predict(randomlist, y, y_pred):
  fg, ax = plt.subplots(2, 5, figsize=(20, 8))
  for i, index in enumerate(randomlist):
    ax[0, i].imshow(np.squeeze(y[index]), 'gray')
    ax[0, i].set_xlabel('y ' + str(index))
    # print(y[index].shape)

  for i, index in enumerate(randomlist):
    ax[1, i].imshow(np.squeeze(y_pred[index]), 'gray')
    ax[1, i].set_xlabel('y_pred ' + str(index))

def prediction(model, large_image, patch_size, step):
  patches = patchify(large_image, (patch_size, patch_size), step=step)  #Step=256 for 256 patches means no overlap

  print(patches.shape)

  redicted_patches = []
  patches_input = []
  for i in range(patches.shape[0]):
    for j in range(patches.shape[1]):
        # print(i,j)
        single_patch = patches[i,j,:,:]
        single_patch_input=np.expand_dims(single_patch, -1)
        # single_patch = np.array(single_patch)
        # print(single_patch.shape)
        #Predict and threshold for values above 0.5 probability

        single_patch_input = np.zeros((patch_size,patch_size,3), dtype="float32")
        single_patch_input[:,:,0] = single_patch
        single_patch_input[:,:,1] = single_patch
        single_patch_input[:,:,2] = single_patch

        single_patch_input = cv.resize(single_patch_input, (256,256), cv.INTER_AREA)
        single_patch_input = np.array(single_patch_input)
        patches_input.append(single_patch_input)

  patches_input = np.array(patches_input)

  patches_prediction = (model.predict(patches_input) >= 0.5).astype(np.uint8)

  # randomlist = random.sample(range(0, len(patches_prediction)),5)
  # show_y_predict(randomlist, patches_input, patches_prediction)
  # plt.show()

  patches_prediction_resize = []
  for i in patches_prediction:
    i = cv.resize(i, (patch_size,patch_size), cv.INTER_AREA)
    patches_prediction_resize.append(i)

  predicted_patches_reshaped = np.reshape(patches_prediction_resize, (patches.shape[0], patches.shape[1], patch_size,patch_size) )
  print(predicted_patches_reshaped.shape)

  reconstructed_image = unpatchify(predicted_patches_reshaped, large_image.shape)
  plt.imshow(np.squeeze(reconstructed_image), cmap='gray')
  return reconstructed_image

def filter_FP(y_pred, thresh):
  y_pred_filter = y_pred.copy()
  kernel = np.ones((3,3),np.uint8)
  y_pred_filter = cv.erode(y_pred_filter, kernel)

  contours, _ = cv.findContours(y_pred_filter.copy(), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
  n = 0
  for ct in contours:
      contour_area = cv.contourArea(ct)
      if (contour_area>= thresh):
          n += 1
      else:
        y_pred_filter = cv.fillPoly(y_pred_filter, pts=[ct], color=(0, 0))
  y_pred_filter =  cv.dilate(y_pred_filter, kernel)
  return y_pred_filter, n


def draw_contour_pred_and_truth(large_image, pred, roi):
  img_copy = large_image.copy()

  contours_pred, _ = cv.findContours(pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  contours_roi, _ = cv.findContours(roi.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  cv.drawContours(img_copy, contours_roi , -1, (255,0,0), 3)
  cv.drawContours(img_copy, contours_pred, -1, (0,255,0), 3)

  return img_copy

def check_Left_Orient(img, roi):
    left_nonzero = cv.countNonZero(img[:, 0:int(img.shape[1] / 2)])
    right_nonzero = cv.countNonZero(img[:, int(img.shape[1] / 2):])

    if (left_nonzero <= right_nonzero):
            img = cv.flip(img, 1)
            roi = cv.flip(roi, 1)

    return img, roi

def cal_overlap_per_large_img(roi, overlap_img):
  bitwise_and_roi_pred = cv.bitwise_and(overlap_img, roi)
  area_truth = np.count_nonzero(bitwise_and_roi_pred)
  overlap = round(area_truth / np.count_nonzero(roi), 4)*100
  return bitwise_and_roi_pred, area_truth, overlap


In [ ]:
def cal_overlap_per_large_img(roi, overlap_img):
  bitwise_and_roi_pred = cv.bitwise_and(overlap_img, roi)
  area_truth = np.count_nonzero(bitwise_and_roi_pred)
  overlap = round(area_truth / np.count_nonzero(roi), 4)*100
  return bitwise_and_roi_pred, area_truth, overlap

In [ ]:
new_model_file = tf.keras.models.load_model(model_file)

#### not clahe, patch 512, step 128

In [ ]:
csv_path = '/content/drive/MyDrive/Mammo dataset/New Dataset/cbis_test_multi_rois.csv'
pred_path = '/content/drive/MyDrive/Test CBIS Predict/original_predict'
overlap_path = '/content/drive/MyDrive/Test CBIS Predict/overlap'
draw_contour_path = '/content/drive/MyDrive/Test CBIS Predict/draw contour'

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path

overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        # path = os.path.dirname(img_file)
        name_file_saver = os.path.basename(img_file)

        large_image = cv.imread(img_file, 0).astype(np.float32)
        large_image = large_image/255.0

        roi = cv.imread(roi_file, 0).astype(np.uint8)

        # bit_depth = img.depth
        # print(large_image.depth, roi.depth)

        img = np.array(large_image)
        h,w = img.shape
        patch_size = 1024
        step = 128

        row = int(( h - patch_size)/step)
        col = int(( w - patch_size)/step)
        new_img_h = row * step + patch_size
        new_img_w = col * step + patch_size

        img = img[0:new_img_h, 0: new_img_w]
        roi = roi[0:new_img_h, 0: new_img_w]

        # plt.imshow(img)
        # plt.show()

        # plt.imshow(roi)
        # plt.show()

        segmented_image = prediction(new_model_file, img, patch_size, step)

        pred_img = np.zeros(roi.shape, dtype="uint8")

        h, w = segmented_image.shape
        print(segmented_image.dtype)

        plt.imshow(segmented_image)
        plt.show()

        pred_img[0:h, 0:w] = segmented_image


        y_pred_filter, n = filter_FP(pred_img, 2000 )

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_CLOSE, kernel)
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)

        num_FP = num_FP + n
        num_P_list.append(n)
        print("number FP  = ", n )

        pred_file_path = os.path.join(pred_path,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path, y_pred_filter, cmap='gray')

        overlap_img, area_truth, overlap = cal_overlap_per_large_img(roi, y_pred_filter)
        print(overlap)
        area_truth_list.append(area_truth)
        overlap_list.append(overlap)
        overlap_img = 1*overlap_img
        file_overlap_img_path = os.path.join(overlap_path,
                                 ''.join([name_file_saver]))
        plt.imsave(file_overlap_img_path, overlap_img, cmap='gray')

        img_draw = cv.imread(img_file)
        img_draw_contour = draw_contour_pred_and_truth(img_draw,y_pred_filter,roi)

        plt.imshow(img_draw_contour)
        plt.title(name_file_saver + "- overlap = "+ str(overlap) + ", Roi = " + str(n))
        plt.show()
        img_draw_contour_path = os.path.join(draw_contour_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_draw_contour_path, img_draw_contour, cmap='gray')




#### not clahe, patch 1024, step 128

In [ ]:
csv_path = '/content/drive/MyDrive/Mammo dataset/New Dataset/cbis_test_multi_rois.csv'
pred_path = '/content/drive/MyDrive/Test CBIS Predict/original_predict'
overlap_path = '/content/drive/MyDrive/Test CBIS Predict/overlap'
draw_contour_path = '/content/drive/MyDrive/Test CBIS Predict/draw contour'

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path

overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        # path = os.path.dirname(img_file)
        name_file_saver = os.path.basename(img_file)

        large_image = cv.imread(img_file, 0).astype(np.float32)
        large_image = large_image/255.0

        roi = cv.imread(roi_file, 0).astype(np.uint8)

        # bit_depth = img.depth
        # print(large_image.depth, roi.depth)

        img = np.array(large_image)
        h,w = img.shape
        patch_size = 1024
        step = 128

        row = int(( h - patch_size)/step)
        col = int(( w - patch_size)/step)
        new_img_h = row * step + patch_size
        new_img_w = col * step + patch_size

        img = img[0:new_img_h, 0: new_img_w]
        roi = roi[0:new_img_h, 0: new_img_w]

        # plt.imshow(img)
        # plt.show()

        # plt.imshow(roi)
        # plt.show()

        segmented_image = prediction(new_model_file, img, patch_size, step)

        pred_img = np.zeros(roi.shape, dtype="uint8")

        h, w = segmented_image.shape
        print(segmented_image.dtype)

        plt.imshow(segmented_image)
        plt.show()

        pred_img[0:h, 0:w] = segmented_image


        y_pred_filter, n = filter_FP(pred_img, 2000 )

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_CLOSE, kernel)
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)

        num_FP = num_FP + n
        num_P_list.append(n)
        print("number FP  = ", n )

        pred_file_path = os.path.join(pred_path,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path, y_pred_filter, cmap='gray')

        overlap_img, area_truth, overlap = cal_overlap_per_large_img(roi, y_pred_filter)
        print(overlap)
        area_truth_list.append(area_truth)
        overlap_list.append(overlap)
        overlap_img = 1*overlap_img
        file_overlap_img_path = os.path.join(overlap_path,
                                 ''.join([name_file_saver]))
        plt.imsave(file_overlap_img_path, overlap_img, cmap='gray')

        img_draw = cv.imread(img_file)
        img_draw_contour = draw_contour_pred_and_truth(img_draw,y_pred_filter,roi)

        plt.imshow(img_draw_contour)
        plt.title(name_file_saver + "- overlap = "+ str(overlap) + ", Roi = " + str(n))
        plt.show()
        img_draw_contour_path = os.path.join(draw_contour_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_draw_contour_path, img_draw_contour, cmap='gray')




#### not clahe, patch 384, step 64

In [ ]:
csv_path = '/content/drive/MyDrive/Mammo dataset/New Dataset/cbis_test_multi_rois.csv'
pred_path = '/content/drive/MyDrive/Test CBIS Predict/original_predict'
overlap_path = '/content/drive/MyDrive/Test CBIS Predict/overlap'
draw_contour_path = '/content/drive/MyDrive/Test CBIS Predict/draw contour'

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path

overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        # path = os.path.dirname(img_file)
        name_file_saver = os.path.basename(img_file)

        large_image = cv.imread(img_file, 0).astype(np.float32)
        large_image = large_image/255.0

        roi = cv.imread(roi_file, 0).astype(np.uint8)

        # bit_depth = img.depth
        # print(large_image.depth, roi.depth)

        img = np.array(large_image)
        h,w = img.shape
        patch_size = 384
        step = 64

        row = int(( h - patch_size)/step)
        col = int(( w - patch_size)/step)
        new_img_h = row * step + patch_size
        new_img_w = col * step + patch_size

        img = img[0:new_img_h, 0: new_img_w]
        roi = roi[0:new_img_h, 0: new_img_w]

        # plt.imshow(img)
        # plt.show()

        # plt.imshow(roi)
        # plt.show()

        segmented_image = prediction(new_model_file, img, patch_size, step)

        pred_img = np.zeros(roi.shape, dtype="uint8")

        h, w = segmented_image.shape
        print(segmented_image.dtype)

        plt.imshow(segmented_image)
        plt.show()

        pred_img[0:h, 0:w] = segmented_image


        y_pred_filter, n = filter_FP(pred_img, 2000 )

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_CLOSE, kernel)
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)

        num_FP = num_FP + n
        num_P_list.append(n)
        print("number FP  = ", n )

        pred_file_path = os.path.join(pred_path,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path, y_pred_filter, cmap='gray')

        overlap_img, area_truth, overlap = cal_overlap_per_large_img(roi, y_pred_filter)
        print(overlap)
        area_truth_list.append(area_truth)
        overlap_list.append(overlap)
        overlap_img = 1*overlap_img
        file_overlap_img_path = os.path.join(overlap_path,
                                 ''.join([name_file_saver]))
        plt.imsave(file_overlap_img_path, overlap_img, cmap='gray')

        img_draw = cv.imread(img_file)
        img_draw_contour = draw_contour_pred_and_truth(img_draw,y_pred_filter,roi)

        plt.imshow(img_draw_contour)
        plt.title(name_file_saver + "- overlap = "+ str(overlap) + ", Roi = " + str(n))
        plt.show()
        img_draw_contour_path = os.path.join(draw_contour_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_draw_contour_path, img_draw_contour, cmap='gray')




### Multi Tumors CBIS, clahe, patch 1024, step 128

In [ ]:
csv_path = '/content/drive/MyDrive/Mammo dataset/New Dataset/cbis_test_clahe_multi_rois.csv'
pred_path = '/content/drive/MyDrive/Test CBIS CLAHE Predict - 1024 step 128/original_predict'
overlap_path = '/content/drive/MyDrive/Test CBIS CLAHE Predict - 1024 step 128/overlap'
draw_contour_path = '/content/drive/MyDrive/Test CBIS CLAHE Predict - 1024 step 128/draw contour'

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path

overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        # path = os.path.dirname(img_file)
        name_file_saver = os.path.basename(img_file)

        large_image = cv.imread(img_file, 0).astype(np.float32)
        large_image = large_image/255.0

        roi = cv.imread(roi_file, 0).astype(np.uint8)

        # bit_depth = img.depth
        # print(large_image.depth, roi.depth)

        img = np.array(large_image)
        h,w = img.shape
        patch_size = 1024
        step = 128

        row = int(( h - patch_size)/step)
        col = int(( w - patch_size)/step)
        new_img_h = row * step + patch_size
        new_img_w = col * step + patch_size

        img = img[0:new_img_h, 0: new_img_w]
        roi = roi[0:new_img_h, 0: new_img_w]

        # plt.imshow(img)
        # plt.show()

        # plt.imshow(roi)
        # plt.show()

        segmented_image = prediction(new_model_file, img, patch_size, step)

        pred_img = np.zeros(roi.shape, dtype="uint8")

        h, w = segmented_image.shape
        print(segmented_image.dtype)

        plt.imshow(segmented_image)
        plt.show()

        pred_img[0:h, 0:w] = segmented_image


        y_pred_filter, n = filter_FP(pred_img, 2000 )

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_CLOSE, kernel)
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)

        num_FP = num_FP + n
        num_P_list.append(n)
        print("number FP  = ", n )

        pred_file_path = os.path.join(pred_path,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path, y_pred_filter, cmap='gray')

        overlap_img, area_truth, overlap = cal_overlap_per_large_img(roi, y_pred_filter)
        print(overlap)
        area_truth_list.append(area_truth)
        overlap_list.append(overlap)
        overlap_img = 1*overlap_img
        file_overlap_img_path = os.path.join(overlap_path,
                                 ''.join([name_file_saver]))
        plt.imsave(file_overlap_img_path, overlap_img, cmap='gray')

        img_draw = cv.imread(img_file)
        img_draw_contour = draw_contour_pred_and_truth(img_draw,y_pred_filter,roi)

        plt.imshow(img_draw_contour)
        plt.title(name_file_saver + "- overlap = "+ str(overlap) + ", Roi = " + str(n))
        plt.show()
        img_draw_contour_path = os.path.join(draw_contour_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_draw_contour_path, img_draw_contour, cmap='gray')

#### clahe patch 512, step 64

In [ ]:
csv_path = '/content/drive/MyDrive/Mammo dataset/New Dataset/cbis_test_clahe_multi_rois.csv'
pred_path = '/content/drive/MyDrive/Test CBIS CLAHE Predict - 1024 step 128/original_predict'
overlap_path = '/content/drive/MyDrive/Test CBIS CLAHE Predict - 1024 step 128/overlap'
draw_contour_path = '/content/drive/MyDrive/Test CBIS CLAHE Predict - 1024 step 128/draw contour'

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path

overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        # path = os.path.dirname(img_file)
        name_file_saver = os.path.basename(img_file)

        large_image = cv.imread(img_file, 0).astype(np.float32)
        large_image = large_image/255.0

        roi = cv.imread(roi_file, 0).astype(np.uint8)

        # bit_depth = img.depth
        # print(large_image.depth, roi.depth)

        img = np.array(large_image)
        h,w = img.shape
        patch_size = 1024
        step = 128

        row = int(( h - patch_size)/step)
        col = int(( w - patch_size)/step)
        new_img_h = row * step + patch_size
        new_img_w = col * step + patch_size

        img = img[0:new_img_h, 0: new_img_w]
        roi = roi[0:new_img_h, 0: new_img_w]

        # plt.imshow(img)
        # plt.show()

        # plt.imshow(roi)
        # plt.show()

        segmented_image = prediction(new_model_file, img, patch_size, step)

        pred_img = np.zeros(roi.shape, dtype="uint8")

        h, w = segmented_image.shape
        print(segmented_image.dtype)

        plt.imshow(segmented_image)
        plt.show()

        pred_img[0:h, 0:w] = segmented_image


        y_pred_filter, n = filter_FP(pred_img, 2000 )

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_CLOSE, kernel)
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)

        num_FP = num_FP + n
        num_P_list.append(n)
        print("number FP  = ", n )

        pred_file_path = os.path.join(pred_path,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path, y_pred_filter, cmap='gray')

        overlap_img, area_truth, overlap = cal_overlap_per_large_img(roi, y_pred_filter)
        print(overlap)
        area_truth_list.append(area_truth)
        overlap_list.append(overlap)
        overlap_img = 1*overlap_img
        file_overlap_img_path = os.path.join(overlap_path,
                                 ''.join([name_file_saver]))
        plt.imsave(file_overlap_img_path, overlap_img, cmap='gray')

        img_draw = cv.imread(img_file)
        img_draw_contour = draw_contour_pred_and_truth(img_draw,y_pred_filter,roi)

        plt.imshow(img_draw_contour)
        plt.title(name_file_saver + "- overlap = "+ str(overlap) + ", Roi = " + str(n))
        plt.show()
        img_draw_contour_path = os.path.join(draw_contour_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_draw_contour_path, img_draw_contour, cmap='gray')

#### clahe, patch 512, step 128

In [ ]:
csv_path = '/content/drive/MyDrive/Mammo dataset/New Dataset/cbis_test_clahe_multi_rois.csv'
pred_path = '/content/drive/MyDrive/Test CBIS CLAHE Predict - 512 step 128/original_predict'
overlap_path = '/content/drive/MyDrive/Test CBIS CLAHE Predict - 512 step 128/overlap'
draw_contour_path = '/content/drive/MyDrive/Test CBIS CLAHE Predict - 512 step 128/draw contour'

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path

overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        # path = os.path.dirname(img_file)
        name_file_saver = os.path.basename(img_file)

        large_image = cv.imread(img_file, 0).astype(np.float32)
        large_image = large_image/255.0

        roi = cv.imread(roi_file, 0).astype(np.uint8)

        # bit_depth = img.depth
        # print(large_image.depth, roi.depth)

        img = np.array(large_image)
        h,w = img.shape
        patch_size = 512
        step = 128

        row = int(( h - patch_size)/step)
        col = int(( w - patch_size)/step)
        new_img_h = row * step + patch_size
        new_img_w = col * step + patch_size

        img = img[0:new_img_h, 0: new_img_w]
        roi = roi[0:new_img_h, 0: new_img_w]

        # plt.imshow(img)
        # plt.show()

        # plt.imshow(roi)
        # plt.show()

        segmented_image = prediction(new_model_file, img, patch_size, step)

        pred_img = np.zeros(roi.shape, dtype="uint8")

        h, w = segmented_image.shape
        print(segmented_image.dtype)

        plt.imshow(segmented_image)
        plt.show()

        pred_img[0:h, 0:w] = segmented_image


        y_pred_filter, n = filter_FP(pred_img, 2000 )

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_CLOSE, kernel)
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)

        num_FP = num_FP + n
        num_P_list.append(n)
        print("number FP  = ", n )

        pred_file_path = os.path.join(pred_path,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path, y_pred_filter, cmap='gray')

        overlap_img, area_truth, overlap = cal_overlap_per_large_img(roi, y_pred_filter)
        print(overlap)
        area_truth_list.append(area_truth)
        overlap_list.append(overlap)
        overlap_img = 1*overlap_img
        file_overlap_img_path = os.path.join(overlap_path,
                                 ''.join([name_file_saver]))
        plt.imsave(file_overlap_img_path, overlap_img, cmap='gray')

        img_draw = cv.imread(img_file)
        img_draw_contour = draw_contour_pred_and_truth(img_draw,y_pred_filter,roi)

        plt.imshow(img_draw_contour)
        plt.title(name_file_saver + "- overlap = "+ str(overlap) + ", Roi = " + str(n))
        plt.show()
        img_draw_contour_path = os.path.join(draw_contour_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_draw_contour_path, img_draw_contour, cmap='gray')

### Load model and predict INBREAST


#### INBreast patch 1024, step 128

In [ ]:
def cal_overlap(x,y,y_pred):
  bitwise_and_roi_patch = np.logical_and(np.squeeze(y), np.squeeze(y_pred))
  overlap = round(np.sum(bitwise_and_roi_patch) / np.sum(np.squeeze(y)), 2)
  return overlap

In [ ]:
def show_image_roi_predict1_img(index, x, y, predict):
    overlap = cal_overlap(x[index], y[index], predict)
    print (overlap)

    img = x[index]

    plt.subplot(131)
    plt.imshow(img)
    plt.title('Image' + str(index))
    plt.axis('off')

    roi = np.squeeze(y[index])
    plt.subplot(132)
    plt.axis('off')
    plt.imshow(roi, 'gray')
    plt.title('ROI ' + str(index))

    roi_pred = np.squeeze(predict)
    plt.subplot(133)
    plt.axis('off')
    plt.imshow(roi_pred)
    plt.title('ROI pred ' + str(index))

    plt.show()


In [ ]:
!pip install patchify

In [ ]:
!unrar x "/content/drive/MyDrive/Mammo dataset/Test Dataset INbreast/INBreast one roi 512.rar"

In [ ]:
!unrar x "/content/drive/MyDrive/Mammo dataset/Test Dataset INbreast/INBreast one roi 512.rar"

drive_path = '/content/drive/MyDrive'
dataset_path = '/content'

files_dir = os.path.join(drive_path, 'Colab Notebooks', 'files_15_5_23', 'training_dataset_20_bkg')
model_file = os.path.join(files_dir,'unet_effb0-new-data-20_bkg-15_5_2023.h5')

test_x= sorted(glob(os.path.join(dataset_path, 'INBreast one roi 512', 'image_thresh_2800_one_roi_512', '*')))
test_y= sorted(glob(os.path.join(dataset_path, 'INBreast one roi 512', 'roi_thresh_2800_one_roi_512', '*')))

In [ ]:
print(len(test_x), len(test_y))

In [ ]:
import pandas as pd
df = pd.DataFrame({'img': test_x, 'roi': test_y})
df.to_csv ('/content/drive/MyDrive/Mammo dataset/Test Dataset INbreast/img_mask_path_one_roi.csv', index = False, header=True)

In [ ]:
csv_path = '/content/drive/MyDrive/Mammo dataset/Test Dataset INbreast/img_mask_path_one_roi.csv'

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path

overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        # path = os.path.dirname(img_file)
        name_file = os.path.basename(img_file)
        print(name_file)

        large_image = cv.imread(img_file, 0).astype(np.float32)
        large_image = large_image/255.0

        roi = cv.imread(roi_file, 0).astype(np.uint8)

        large_image, roi= check_Left_Orient (large_image, roi)

        # bit_depth = img.depth
        # print(large_image.depth, roi.depth)

        img = np.array(large_image)
        h,w = img.shape
        patch_size = 1024
        step = 128
        print(img.shape, roi.shape)


        row = int(( h - patch_size)/step)
        col = int(( w - patch_size)/step)
        new_img_h = row * step + patch_size
        new_img_w = col * step + patch_size

        img = img[0:new_img_h, 0: new_img_w]
        roi = roi[0:new_img_h, 0: new_img_w]

        print(img.shape, roi.shape)

        plt.imsave(img_file, img, cmap='gray')
        plt.imsave(roi_file, roi, cmap='gray')


        # plt.imshow(img, 'gray')
        # plt.show()

        # plt.imshow(roi, 'gray')
        # plt.show()


In [ ]:
def show_y_predict(randomlist, y, y_pred):
  fg, ax = plt.subplots(2, 5, figsize=(20, 8))
  for i, index in enumerate(randomlist):
    ax[0, i].imshow(np.squeeze(y[index]), 'gray')
    ax[0, i].set_xlabel('y ' + str(index))
    # print(y[index].shape)

  for i, index in enumerate(randomlist):
    ax[1, i].imshow(np.squeeze(y_pred[index]), 'gray')
    ax[1, i].set_xlabel('y_pred ' + str(index))

In [ ]:
def prediction(model, large_image, patch_size, step):
  patches = patchify(large_image, (patch_size, patch_size), step=step)  #Step=256 for 256 patches means no overlap

  print(patches.shape)

  redicted_patches = []
  patches_input = []
  for i in range(patches.shape[0]):
    for j in range(patches.shape[1]):
        # print(i,j)
        single_patch = patches[i,j,:,:]
        single_patch_input=np.expand_dims(single_patch, -1)
        # single_patch = np.array(single_patch)
        # print(single_patch.shape)
        #Predict and threshold for values above 0.5 probability

        single_patch_input = np.zeros((patch_size,patch_size,3), dtype="float32")
        single_patch_input[:,:,0] = single_patch
        single_patch_input[:,:,1] = single_patch
        single_patch_input[:,:,2] = single_patch

        single_patch_input = cv.resize(single_patch_input, (256,256), cv.INTER_AREA)
        single_patch_input = np.array(single_patch_input)
        patches_input.append(single_patch_input)

  patches_input = np.array(patches_input)

  patches_prediction = (model.predict(patches_input) >= 0.5).astype(np.uint8)

  # randomlist = random.sample(range(0, len(patches_prediction)),5)
  # show_y_predict(randomlist, patches_input, patches_prediction)
  # plt.show()

  patches_prediction_resize = []
  for i in patches_prediction:
    i = cv.resize(i, (patch_size,patch_size), cv.INTER_AREA)
    patches_prediction_resize.append(i)

  predicted_patches_reshaped = np.reshape(patches_prediction_resize, (patches.shape[0], patches.shape[1], patch_size,patch_size) )
  print(predicted_patches_reshaped.shape)

  reconstructed_image = unpatchify(predicted_patches_reshaped, large_image.shape)
  plt.imshow(np.squeeze(reconstructed_image), cmap='gray')
  return reconstructed_image

In [ ]:
def filter_FP(y_pred, thresh):
  y_pred_filter = y_pred.copy()
  kernel = np.ones((3,3),np.uint8)
  y_pred_filter = cv.erode(y_pred_filter, kernel)

  contours, _ = cv.findContours(y_pred_filter.copy(), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
  n = 0
  for ct in contours:
      contour_area = cv.contourArea(ct)
      if (contour_area>= thresh):
          n += 1
      else:
        y_pred_filter = cv.fillPoly(y_pred_filter, pts=[ct], color=(0, 0))
  y_pred_filter =  cv.dilate(y_pred_filter, kernel)
  return y_pred_filter, n

In [ ]:
def cal_overlap_per_large_img(roi, overlap_img):
  bitwise_and_roi_pred = np.logical_and(overlap_img, roi)
  area_truth = np.count_nonzero(bitwise_and_roi_pred)
  overlap = round(area_truth / np.count_nonzero(roi), 4)*100
  return bitwise_and_roi_pred, area_truth, overlap

In [ ]:
def draw_contour_pred_and_truth(large_image, pred, roi):
  img_copy = large_image.copy()

  contours_pred, _ = cv.findContours(pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  contours_roi, _ = cv.findContours(roi.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  cv.drawContours(img_copy, contours_roi , -1, (255,0,0), 3)
  cv.drawContours(img_copy, contours_pred, -1, (0,255,0), 3)

  return img_copy

In [ ]:
def check_Left_Orient(img, roi):
    left_nonzero = cv.countNonZero(img[:, 0:int(img.shape[1] / 2)])
    right_nonzero = cv.countNonZero(img[:, int(img.shape[1] / 2):])

    if (left_nonzero <= right_nonzero):
            img = cv.flip(img, 1)
            roi = cv.flip(roi, 1)

    return img, roi

In [ ]:
new_model_file = tf.keras.models.load_model(model_file)

In [ ]:
csv_path = '/content/drive/MyDrive/Mammo dataset/Test Dataset INbreast/img_mask_path_one_roi.csv'
pred_path = '/content/drive/MyDrive/Test_INBreast_Predict - 1024 step 128/original_predict'
overlap_path = '/content/drive/MyDrive/Test_INBreast_Predict - 1024 step 128/overlap'
draw_contour_path = '/content/drive/MyDrive/Test_INBreast_Predict - 1024 step 128/draw contour'

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path

overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        # path = os.path.dirname(img_file)
        name_file_saver = os.path.basename(img_file)

        large_image = cv.imread(img_file, 0).astype(np.float32)
        large_image = large_image/255.0

        roi = cv.imread(roi_file, 0).astype(np.uint8)

        large_image, roi= check_Left_Orient (large_image, roi)

        # bit_depth = img.depth
        # print(large_image.depth, roi.depth)

        img = np.array(large_image)
        h,w = img.shape
        patch_size = 1024
        step = 128

        row = int(( h - patch_size)/step)
        col = int(( w - patch_size)/step)
        new_img_h = row * step + patch_size
        new_img_w = col * step + patch_size

        img = img[0:new_img_h, 0: new_img_w]
        roi = roi[0:new_img_h, 0: new_img_w]

        plt.imsave(img_file, img, cmap='gray')
        plt.imsave(roi_file, roi, cmap='gray')

        # print (img.dtype, roi.dtype)

        # plt.imshow(img, 'gray')
        # plt.show()

        # plt.imshow(roi, 'gray')
        # plt.show()

        segmented_image = prediction(new_model_file, img, patch_size, step)

        pred_img = np.zeros(roi.shape, dtype="uint8")

        h, w = segmented_image.shape
        print(segmented_image.dtype)

        plt.imshow(segmented_image)
        plt.show()

        pred_img[0:h, 0:w] = segmented_image


        y_pred_filter, n = filter_FP(pred_img, 2000 )

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_CLOSE, kernel)
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)

        num_FP = num_FP + n
        num_P_list.append(n)
        print("number FP  = ", n )

        pred_file_path = os.path.join(pred_path,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path, y_pred_filter, cmap='gray')

        overlap_img, area_truth, overlap = cal_overlap_per_large_img(roi, y_pred_filter)
        print(overlap)
        area_truth_list.append(area_truth)
        overlap_list.append(overlap)
        overlap_img = 1*overlap_img
        file_overlap_img_path = os.path.join(overlap_path,
                                 ''.join([name_file_saver]))
        plt.imsave(file_overlap_img_path, overlap_img, cmap='gray')

        img_draw = cv.imread(img_file)
        img_draw_contour = draw_contour_pred_and_truth(img_draw,y_pred_filter,roi)

        plt.imshow(img_draw_contour)
        plt.title(name_file_saver + "- overlap = "+ str(overlap) + ", Roi = " + str(n))
        plt.show()
        img_draw_contour_path = os.path.join(draw_contour_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_draw_contour_path, img_draw_contour, cmap='gray')


In [ ]:
data = pd.read_csv(csv_path)
img = data["img"]
roi = data["roi"]
import pandas as pd
save_csv_path= '/content/drive/MyDrive/Test_INBreast_Predict - 1024 step 128/overlap_patch_1024_step_128.csv'
df = pd.DataFrame({'img': img, 'roi': roi, 'overlap': overlap_list, 'area_truth_list': area_truth_list, 'num_FP': num_P_list})
df.to_csv (save_csv_path, index = False, header=True)

#### INbreast patch 512, step 64

In [ ]:
csv_path = '/content/drive/MyDrive/Mammo dataset/Test Dataset INbreast/img_mask_path_one_roi.csv'
pred_path = '/content/drive/MyDrive/Test_INBreast_Predict - 512 step 64/original_predict'
overlap_path = '/content/drive/MyDrive/Test_INBreast_Predict - 512 step 64/overlap'
draw_contour_path = '/content/drive/MyDrive/Test_INBreast_Predict - 512 step 64/draw contour'

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path

overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        # path = os.path.dirname(img_file)
        name_file_saver = os.path.basename(img_file)

        large_image = cv.imread(img_file, 0).astype(np.float32)
        large_image = large_image/255.0

        roi = cv.imread(roi_file, 0).astype(np.uint8)

        large_image, roi= check_Left_Orient (large_image, roi)

        # bit_depth = img.depth
        # print(large_image.depth, roi.depth)

        img = np.array(large_image)
        h,w = img.shape
        patch_size = 512
        step = 64

        row = int(( h - patch_size)/step)
        col = int(( w - patch_size)/step)
        new_img_h = row * step + patch_size
        new_img_w = col * step + patch_size

        img = img[0:new_img_h, 0: new_img_w]
        roi = roi[0:new_img_h, 0: new_img_w]

        plt.imsave(img_file, img, cmap='gray')
        plt.imsave(roi_file, roi, cmap='gray')

        # print (img.dtype, roi.dtype)

        # plt.imshow(img, 'gray')
        # plt.show()

        # plt.imshow(roi, 'gray')
        # plt.show()

        segmented_image = prediction(new_model_file, img, patch_size, step)

        pred_img = np.zeros(roi.shape, dtype="uint8")

        h, w = segmented_image.shape
        print(segmented_image.dtype)

        plt.imshow(segmented_image)
        plt.show()

        pred_img[0:h, 0:w] = segmented_image


        y_pred_filter, n = filter_FP(pred_img, 2000 )

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_CLOSE, kernel)
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)

        num_FP = num_FP + n
        num_P_list.append(n)
        print("number FP  = ", n )

        pred_file_path = os.path.join(pred_path,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path, y_pred_filter, cmap='gray')

        overlap_img, area_truth, overlap = cal_overlap_per_large_img(roi, y_pred_filter)
        print(overlap)
        area_truth_list.append(area_truth)
        overlap_list.append(overlap)
        overlap_img = 1*overlap_img
        file_overlap_img_path = os.path.join(overlap_path,
                                 ''.join([name_file_saver]))
        plt.imsave(file_overlap_img_path, overlap_img, cmap='gray')

        img_draw = cv.imread(img_file)
        img_draw_contour = draw_contour_pred_and_truth(img_draw,y_pred_filter,roi)

        plt.imshow(img_draw_contour)
        plt.title(name_file_saver + "- overlap = "+ str(overlap) + ", Roi = " + str(n))
        plt.show()
        img_draw_contour_path = os.path.join(draw_contour_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_draw_contour_path, img_draw_contour, cmap='gray')


In [ ]:
data = pd.read_csv(csv_path)
img = data["img"]
roi = data["roi"]
import pandas as pd
save_csv_path= '/content/drive/MyDrive/Test_INBreast_Predict - 512 step 64/overlap_patch_512_step_64.csv'
df = pd.DataFrame({'img': img, 'roi': roi, 'overlap': overlap_list, 'area_truth_list': area_truth_list, 'num_FP': num_P_list})
df.to_csv (save_csv_path, index = False, header=True)



```
# This is formatted as code
```

#### INBreast patch 384, step 64

In [ ]:
!pip install patchify

In [ ]:
!unrar x "/content/drive/MyDrive/Mammo dataset/Test Dataset INbreast/INBreast one roi 512.rar"

In [ ]:
drive_path = '/content/drive/MyDrive'
dataset_path = '/content'

files_dir = os.path.join(drive_path, 'Colab Notebooks', 'files_15_5_23', 'training_dataset_20_bkg')
model_file = os.path.join(files_dir,'unet_effb0-new-data-20_bkg-15_5_2023.h5')

test_x= sorted(glob(os.path.join(dataset_path, 'INBreast one roi 512', 'image_thresh_2800_one_roi_512', '*')))
test_y= sorted(glob(os.path.join(dataset_path, 'INBreast one roi 512', 'roi_thresh_2800_one_roi_512', '*')))

In [ ]:
def show_y_predict(randomlist, y, y_pred):
  fg, ax = plt.subplots(2, 5, figsize=(20, 8))
  for i, index in enumerate(randomlist):
    ax[0, i].imshow(np.squeeze(y[index]), 'gray')
    ax[0, i].set_xlabel('y ' + str(index))
    # print(y[index].shape)

  for i, index in enumerate(randomlist):
    ax[1, i].imshow(np.squeeze(y_pred[index]), 'gray')
    ax[1, i].set_xlabel('y_pred ' + str(index))

def prediction(model, large_image, patch_size, step):
  patches = patchify(large_image, (patch_size, patch_size), step=step)  #Step=256 for 256 patches means no overlap

  print(patches.shape)

  redicted_patches = []
  patches_input = []
  for i in range(patches.shape[0]):
    for j in range(patches.shape[1]):
        # print(i,j)
        single_patch = patches[i,j,:,:]
        single_patch_input=np.expand_dims(single_patch, -1)
        # single_patch = np.array(single_patch)
        # print(single_patch.shape)
        #Predict and threshold for values above 0.5 probability

        single_patch_input = np.zeros((patch_size,patch_size,3), dtype="float32")
        single_patch_input[:,:,0] = single_patch
        single_patch_input[:,:,1] = single_patch
        single_patch_input[:,:,2] = single_patch

        single_patch_input = cv.resize(single_patch_input, (256,256), cv.INTER_AREA)
        single_patch_input = np.array(single_patch_input)
        patches_input.append(single_patch_input)

  patches_input = np.array(patches_input)

  patches_prediction = (model.predict(patches_input) >= 0.5).astype(np.uint8)

  # randomlist = random.sample(range(0, len(patches_prediction)),5)
  # show_y_predict(randomlist, patches_input, patches_prediction)
  # plt.show()

  patches_prediction_resize = []
  for i in patches_prediction:
    i = cv.resize(i, (patch_size,patch_size), cv.INTER_AREA)
    patches_prediction_resize.append(i)

  predicted_patches_reshaped = np.reshape(patches_prediction_resize, (patches.shape[0], patches.shape[1], patch_size,patch_size) )
  print(predicted_patches_reshaped.shape)

  reconstructed_image = unpatchify(predicted_patches_reshaped, large_image.shape)
  plt.imshow(np.squeeze(reconstructed_image), cmap='gray')
  return reconstructed_image

def filter_FP(y_pred, thresh):
  y_pred_filter = y_pred.copy()
  kernel = np.ones((3,3),np.uint8)
  y_pred_filter = cv.erode(y_pred_filter, kernel)

  contours, _ = cv.findContours(y_pred_filter.copy(), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
  n = 0
  for ct in contours:
      contour_area = cv.contourArea(ct)
      if (contour_area>= thresh):
          n += 1
      else:
        y_pred_filter = cv.fillPoly(y_pred_filter, pts=[ct], color=(0, 0))
  y_pred_filter =  cv.dilate(y_pred_filter, kernel)
  return y_pred_filter, n

def cal_overlap_per_large_img(roi, overlap_img):
  bitwise_and_roi_pred = np.logical_and(overlap_img, roi)
  area_truth = np.count_nonzero(bitwise_and_roi_pred)
  overlap = round(area_truth / np.count_nonzero(roi), 4)*100
  return bitwise_and_roi_pred, area_truth, overlap

def draw_contour_pred_and_truth(large_image, pred, roi):
  img_copy = large_image.copy()

  contours_pred, _ = cv.findContours(pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  contours_roi, _ = cv.findContours(roi.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  cv.drawContours(img_copy, contours_roi , -1, (255,0,0), 3)
  cv.drawContours(img_copy, contours_pred, -1, (0,255,0), 3)

  return img_copy

def check_Left_Orient(img, roi):
    left_nonzero = cv.countNonZero(img[:, 0:int(img.shape[1] / 2)])
    right_nonzero = cv.countNonZero(img[:, int(img.shape[1] / 2):])

    if (left_nonzero <= right_nonzero):
            img = cv.flip(img, 1)
            roi = cv.flip(roi, 1)

    return img, roi

def cal_overlap_per_large_img(roi, overlap_img):
  bitwise_and_roi_pred = cv.bitwise_and(overlap_img, roi)
  area_truth = np.count_nonzero(bitwise_and_roi_pred)
  overlap = round(area_truth / np.count_nonzero(roi), 4)*100
  return bitwise_and_roi_pred, area_truth, overlap

In [ ]:
new_model_file = tf.keras.models.load_model(model_file)

In [ ]:
csv_path = '/content/drive/MyDrive/Mammo dataset/Test Dataset INbreast/img_mask_path_one_roi.csv'
pred_path = '/content/drive/MyDrive/Test_INBreast_Predict - 384 step 64/original_predict'
overlap_path = '/content/drive/MyDrive/Test_INBreast_Predict - 384 step 64/overlap'
draw_contour_path = '/content/drive/MyDrive/Test_INBreast_Predict - 384 step 64/draw contour'

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path

overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        # path = os.path.dirname(img_file)
        name_file_saver = os.path.basename(img_file)

        large_image = cv.imread(img_file, 0).astype(np.float32)
        large_image = large_image/255.0

        roi = cv.imread(roi_file, 0).astype(np.uint8)

        large_image, roi= check_Left_Orient (large_image, roi)

        # bit_depth = img.depth
        # print(large_image.depth, roi.depth)

        img = np.array(large_image)
        h,w = img.shape
        patch_size = 384
        step = 64

        row = int(( h - patch_size)/step)
        col = int(( w - patch_size)/step)
        new_img_h = row * step + patch_size
        new_img_w = col * step + patch_size

        img = img[0:new_img_h, 0: new_img_w]
        roi = roi[0:new_img_h, 0: new_img_w]

        plt.imsave(img_file, img, cmap='gray')
        plt.imsave(roi_file, roi, cmap='gray')

        # print (img.dtype, roi.dtype)

        # plt.imshow(img, 'gray')
        # plt.show()

        # plt.imshow(roi, 'gray')
        # plt.show()

        segmented_image = prediction(new_model_file, img, patch_size, step)

        pred_img = np.zeros(roi.shape, dtype="uint8")

        h, w = segmented_image.shape
        print(segmented_image.dtype)

        plt.imshow(segmented_image)
        plt.show()

        pred_img[0:h, 0:w] = segmented_image


        y_pred_filter, n = filter_FP(pred_img, 2000)

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_CLOSE, kernel)
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)

        num_FP = num_FP + n
        num_P_list.append(n)
        print("number FP  = ", n )

        pred_file_path = os.path.join(pred_path,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path, y_pred_filter, cmap='gray')

        overlap_img, area_truth, overlap = cal_overlap_per_large_img(roi, y_pred_filter)
        print(overlap)
        area_truth_list.append(area_truth)
        overlap_list.append(overlap)
        overlap_img = 1*overlap_img
        file_overlap_img_path = os.path.join(overlap_path,
                                 ''.join([name_file_saver]))
        plt.imsave(file_overlap_img_path, overlap_img, cmap='gray')

        img_draw = cv.imread(img_file)
        img_draw_contour = draw_contour_pred_and_truth(img_draw,y_pred_filter,roi)

        plt.imshow(img_draw_contour)
        plt.title(name_file_saver + "- overlap = "+ str(overlap) + ", Roi = " + str(n))
        plt.show()
        img_draw_contour_path = os.path.join(draw_contour_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_draw_contour_path, img_draw_contour, cmap='gray')


In [ ]:
data = pd.read_csv(csv_path)
img = data["img"]
roi = data["roi"]
import pandas as pd
save_csv_path= '/content/drive/MyDrive/Test_INBreast_Predict - 384 step 64/overlap.csv'
df = pd.DataFrame({'img': img, 'roi': roi, 'overlap': overlap_list, 'area_truth_list': area_truth_list, 'num_FP': num_P_list})
df.to_csv (save_csv_path, index = False, header=True)

### combine patch 512 and 1024 - Test INBreast Predict

In [ ]:
!unrar x "/content/drive/MyDrive/Mammo dataset/Test Dataset INbreast/INBreast one roi 512.rar"

In [ ]:
drive_path = '/content/drive/MyDrive'
dataset_path = '/content'

predict_patch_1024= sorted(glob(os.path.join(drive_path, 'Test_INBreast_Predict - 1024 step 128', 'original_predict', '*')))
predict_patch_512= sorted(glob(os.path.join(drive_path, 'Test_INBreast_Predict - 512 step 64', 'original_predict', '*')))
mask = sorted(glob(os.path.join(dataset_path, 'INBreast one roi 512', 'roi_thresh_2800_one_roi_512', '*')))
img = sorted(glob(os.path.join(dataset_path, 'INBreast one roi 512', 'image_thresh_2800_one_roi_512', '*')))


In [ ]:
print(len(predict_patch_512), len(predict_patch_1024), len(mask), len(img))

In [ ]:
import pandas as pd
df = pd.DataFrame({'img': img, 'mask' : mask  ,'predict_patch_512': predict_patch_512, 'predict_patch_1024': predict_patch_1024})
df.to_csv ('/content/drive/MyDrive/Test_INBreast_combine_1024_512/combine_512_1024_predict.csv', index = False, header=True)
print('Done')

In [ ]:
csv_path = '/content/drive/MyDrive/Test_INBreast_combine_1024_512/combine_512_1024_predict.csv'
pred_path = '/content/drive/MyDrive/Test_INBreast_combine_1024_512/original_predict'
overlap_path = '/content/drive/MyDrive/Test_INBreast_combine_1024_512/overlap'
draw_contour_path = '/content/drive/MyDrive/Test_INBreast_combine_1024_512/draw contour'

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path

overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['mask']

        mask_patch_512_file = row['predict_patch_512']
        mask_patch_1024_file = row['predict_patch_1024']

        name_file_saver = os.path.basename(img_file)

        large_image = cv.imread(img_file, 0)

        roi = cv.imread(roi_file, 0)
        mask_512 = cv.imread(mask_patch_512_file, 0)
        mask_1024 = cv.imread(mask_patch_1024_file, 0)

        combine = np.zeros(mask_1024.shape, dtype=np.uint8)

        combine = np.logical_or(mask_512,mask_1024).astype(np.uint8)
        # combine = np.array(combine)
        plt.imshow(combine)
        plt.title(name_file_saver + " - combine predict ")
        plt.show()

        y_pred_filter, n = filter_FP(combine, 2000 )

        num_P_list.append(n)

        pred_file_path = os.path.join(pred_path,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path, combine, cmap='gray')

        overlap_img, area_truth, overlap = cal_overlap_per_large_img(roi, combine)
        print(overlap)
        area_truth_list.append(overlap)

        overlap_img = 1*overlap_img
        file_overlap_img_path = os.path.join(overlap_path,
                                 ''.join([name_file_saver]))
        plt.imsave(file_overlap_img_path, overlap_img, cmap='gray')

        img_draw = cv.imread(img_file)
        combine_predict = cv.imread(pred_file_path, 0)
        img_draw_contour = draw_contour_pred_and_truth(img_draw, combine, roi)

        plt.imshow(img_draw_contour)
        plt.title(name_file_saver + "- overlap = "+ str(overlap))
        plt.show()
        img_draw_contour_path = os.path.join(draw_contour_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_draw_contour_path, img_draw_contour, cmap='gray')



In [ ]:
data = pd.read_csv(csv_path)
img = data["img"]
roi = data["mask"]
import pandas as pd
save_csv_path= '/content/drive/MyDrive/Test_INBreast_combine_1024_512/overlap_combine_patch_512_1024.csv'
df = pd.DataFrame({'img': img, 'mask': roi, 'overlap': area_truth_list, 'num_FP': num_P_list})
df.to_csv (save_csv_path, index = False, header=True)

###combine patch 384, 512 and 1024 - Test INBreast Predict

In [ ]:
drive_path = '/content/drive/MyDrive/Test_INBreast_Predict - 384 step 64'
draw_contour= sorted(glob(os.path.join(dataset_path, 'draw contour', '*')))
original_predict= sorted(glob(os.path.join(dataset_path,'original_predict', '*')))
overlap= sorted(glob(os.path.join(dataset_path,'overlap', '*')))

drive_path = '/content/drive/MyDrive'
dataset_path = '/content'

draw_contour= sorted(glob(os.path.join(drive_path, 'Test_INBreast_Predict - 384 step 64', 'draw contour', '*')))
original_predict= sorted(glob(os.path.join(drive_path, 'Test_INBreast_Predict - 384 step 64', 'original_predict', '*')))
overlap= sorted(glob(os.path.join(drive_path, 'Test_INBreast_Predict - 384 step 64', 'overlap', '*')))


print(draw_contour)

for i, n in enumerate(draw_contour):
        draw_contour_path = draw_contour[i]
        original_predict_path = original_predict[i]
        overlap_path = overlap[i]

        draw_contour_img = cv.imread(draw_contour_path, 0).astype(np.uint8)
        original_predict_img = cv.imread(original_predict_path, 0).astype(np.uint8)
        overlap_img = cv.imread(overlap_path, 0).astype(np.uint8)

        img = np.array(draw_contour_img)
        h,w = img.shape
        patch_size = 1024
        step = 128

        row = int(( h - patch_size)/step)
        col = int(( w - patch_size)/step)
        new_img_h = row * step + patch_size
        new_img_w = col * step + patch_size

        draw_contour_img = draw_contour_img[0:new_img_h, 0: new_img_w]
        original_predict_img = original_predict_img[0:new_img_h, 0: new_img_w]
        overlap_img = overlap_img[0:new_img_h, 0: new_img_w]

        print(draw_contour_img.shape,original_predict_img.shape,overlap_img.shape)
        plt.imshow(img_draw_contour)
        plt.show()

        # plt.imsave(img_file, img, cmap='gray')
        # plt.imsave(roi_file, roi, cmap='gray')
        # plt.imsave(roi_file, roi, cmap='gray')


In [ ]:
drive_path = '/content/drive/MyDrive'
dataset_path = '/content'

predict_patch_1024= sorted(glob(os.path.join(drive_path, 'Test_INBreast_Predict - 1024 step 128', 'original_predict', '*')))
predict_patch_512= sorted(glob(os.path.join(drive_path, 'Test_INBreast_Predict - 512 step 64', 'original_predict', '*')))
predict_patch_384= sorted(glob(os.path.join(drive_path, 'Test_INBreast_Predict - 384 step 64', 'original_predict', '*')))

mask = sorted(glob(os.path.join(dataset_path, 'INBreast one roi 512', 'roi_thresh_2800_one_roi_512', '*')))
img = sorted(glob(os.path.join(dataset_path, 'INBreast one roi 512', 'image_thresh_2800_one_roi_512', '*')))

In [ ]:
print(len(predict_patch_384),len(predict_patch_512), len(predict_patch_1024), len(mask), len(img))

In [ ]:
import pandas as pd
df = pd.DataFrame({'img': img, 'mask' : mask  ,'predict_patch_384': predict_patch_384 ,'predict_patch_512': predict_patch_512, 'predict_patch_1024': predict_patch_1024})
df.to_csv ('/content/drive/MyDrive/Test_INBreast_combine_1024_512_384/combine_384_512_1024_predict.csv', index = False, header=True)
print('Done')

In [ ]:
csv_path = '/content/drive/MyDrive/Test_INBreast_combine_1024_512_384/combine_384_512_1024_predict.csv'
pred_path = '/content/drive/MyDrive/Test_INBreast_combine_1024_512_384/original_predict'
overlap_path = '/content/drive/MyDrive/Test_INBreast_combine_1024_512_384/overlap'
draw_contour_path = '/content/drive/MyDrive/Test_INBreast_combine_1024_512_384/draw contour'

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path

overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['mask']

        mask_patch_384_file = row['predict_patch_384']
        mask_patch_512_file = row['predict_patch_512']
        mask_patch_1024_file = row['predict_patch_1024']

        name_file_saver = os.path.basename(img_file)

        large_image = cv.imread(img_file, 0)

        roi = cv.imread(roi_file, 0)
        mask_512 = cv.imread(mask_patch_512_file, 0)
        mask_1024 = cv.imread(mask_patch_1024_file, 0)

        mask_384 = cv.imread(mask_patch_384_file, 0)
        mask_384 = np.array(mask_384)
        h,w = mask_512.shape

        mask_384 = mask_384[0:h, 0: w]

        combine = np.zeros(mask_1024.shape, dtype=np.uint8)

        combine_1 = np.logical_or(mask_384, mask_512).astype(np.uint8)
        combine = np.logical_or(combine_1, mask_1024).astype(np.uint8)
        # combine = np.array(combine)
        plt.imshow(combine)
        plt.title(name_file_saver + " - combine predict ")
        plt.show()

        y_pred_filter, n = filter_FP(combine, 2000 )

        num_P_list.append(n)

        pred_file_path = os.path.join(pred_path,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path, combine, cmap='gray')

        overlap_img, area_truth, overlap = cal_overlap_per_large_img(roi, combine)
        print(overlap)
        area_truth_list.append(overlap)

        overlap_img = 1*overlap_img
        file_overlap_img_path = os.path.join(overlap_path,
                                 ''.join([name_file_saver]))
        plt.imsave(file_overlap_img_path, overlap_img, cmap='gray')

        img_draw = cv.imread(img_file)
        combine_predict = cv.imread(pred_file_path, 0)
        img_draw_contour = draw_contour_pred_and_truth(img_draw, combine, roi)

        plt.imshow(img_draw_contour)
        plt.title(name_file_saver + "- overlap = "+ str(overlap))
        plt.show()
        img_draw_contour_path = os.path.join(draw_contour_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_draw_contour_path, img_draw_contour, cmap='gray')


In [ ]:
data = pd.read_csv(csv_path)
img = data["img"]
roi = data["mask"]
import pandas as pd
save_csv_path= '/content/drive/MyDrive/Test_INBreast_combine_1024_512_384/overlap_combine_384_512_1024.csv'
df = pd.DataFrame({'img': img, 'mask': roi, 'overlap': area_truth_list, 'num_FP': num_P_list})
df.to_csv (save_csv_path, index = False, header=True)


### Final Solution for INBreast

##### BB < 850

In [ ]:
!pip install patchify

In [ ]:
!unrar x "/content/drive/MyDrive/Mammo dataset/Test Dataset INbreast/INBreast roi 850 remove bkg.rar"

In [ ]:
drive_path = '/content/drive/MyDrive'
dataset_path = '/content'

files_dir = os.path.join(drive_path, 'Colab Notebooks', 'files_15_5_23', 'training_dataset_20_bkg')
model_file = os.path.join(files_dir,'unet_effb0-new-data-20_bkg-15_5_2023.h5')

test_x= sorted(glob(os.path.join(dataset_path, 'INBreast roi 850 remove bkg', 'image_thresh_2800_roi_850', '*')))
test_y= sorted(glob(os.path.join(dataset_path, 'INBreast roi 850 remove bkg', 'roi_thresh_2800_roi_850', '*')))

In [ ]:
import pandas as pd
df = pd.DataFrame({'img': test_x, 'roi': test_y})
df.to_csv ('/content/drive/MyDrive/Mammo dataset/Test Dataset INbreast/img_mask_path_roi_850_remove_bkg.csv', index = False, header=True)

In [ ]:
load_model_file = tf.keras.models.load_model(model_file)

In [ ]:
def show_y_predict(randomlist, y, y_pred):
  fg, ax = plt.subplots(2, 5, figsize=(20, 8))
  for i, index in enumerate(randomlist):
    ax[0, i].imshow(np.squeeze(y[index]), 'gray')
    ax[0, i].set_xlabel('y ' + str(index))
    # print(y[index].shape)

  for i, index in enumerate(randomlist):
    ax[1, i].imshow(np.squeeze(y_pred[index]), 'gray')
    ax[1, i].set_xlabel('y_pred ' + str(index))

def prediction(model, large_image, patch_size, step):
  patches = patchify(large_image, (patch_size, patch_size), step=step)  #Step=256 for 256 patches means no overlap

  print(patches.shape)

  redicted_patches = []
  patches_input = []
  for i in range(patches.shape[0]):
    for j in range(patches.shape[1]):
        # print(i,j)
        single_patch = patches[i,j,:,:]
        single_patch_input=np.expand_dims(single_patch, -1)
        # single_patch = np.array(single_patch)
        # print(single_patch.shape)
        #Predict and threshold for values above 0.5 probability

        single_patch_input = np.zeros((patch_size,patch_size,3), dtype="float32")
        single_patch_input[:,:,0] = single_patch
        single_patch_input[:,:,1] = single_patch
        single_patch_input[:,:,2] = single_patch

        single_patch_input = cv.resize(single_patch_input, (256,256), cv.INTER_AREA)
        single_patch_input = np.array(single_patch_input)
        patches_input.append(single_patch_input)

  patches_input = np.array(patches_input)

  patches_prediction = (model.predict(patches_input) >= 0.5).astype(np.uint8)

  # randomlist = random.sample(range(0, len(patches_prediction)),5)
  # show_y_predict(randomlist, patches_input, patches_prediction)
  # plt.show()

  patches_prediction_resize = []
  for i in patches_prediction:
    i = cv.resize(i, (patch_size,patch_size), cv.INTER_AREA)
    patches_prediction_resize.append(i)

  predicted_patches_reshaped = np.reshape(patches_prediction_resize, (patches.shape[0], patches.shape[1], patch_size,patch_size) )
  print(predicted_patches_reshaped.shape)

  reconstructed_image = unpatchify(predicted_patches_reshaped, large_image.shape)
  plt.imshow(np.squeeze(reconstructed_image), cmap='gray')
  return reconstructed_image

def filter_FP(y_pred, thresh):
  y_pred_filter = y_pred.copy()
  kernel = np.ones((3,3),np.uint8)
  y_pred_filter = cv.erode(y_pred_filter, kernel)

  contours, _ = cv.findContours(y_pred_filter.copy(), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
  n = 0
  for ct in contours:
      contour_area = cv.contourArea(ct)
      if (contour_area>= thresh):
          n += 1
      else:
        y_pred_filter = cv.fillPoly(y_pred_filter, pts=[ct], color=(0, 0))
  y_pred_filter =  cv.dilate(y_pred_filter, kernel)
  return y_pred_filter, n


def draw_contour_pred_and_truth(large_image, pred, roi):
  img_copy = large_image.copy()

  contours_pred, _ = cv.findContours(pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  contours_roi, _ = cv.findContours(roi.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  cv.drawContours(img_copy, contours_roi , -1, (255,0,0), 3)
  cv.drawContours(img_copy, contours_pred, -1, (0,255,0), 3)

  return img_copy

def check_Left_Orient(img, roi):
    left_nonzero = cv.countNonZero(img[:, 0:int(img.shape[1] / 2)])
    right_nonzero = cv.countNonZero(img[:, int(img.shape[1] / 2):])

    if (left_nonzero <= right_nonzero):
            img = cv.flip(img, 1)
            roi = cv.flip(roi, 1)

    return img, roi

def cal_overlap_per_large_img(roi, overlap_img):
  bitwise_and_roi_pred = cv.bitwise_and(overlap_img, roi)
  area_truth = np.count_nonzero(bitwise_and_roi_pred)
  overlap = round(area_truth / np.count_nonzero(roi), 4)*100
  return bitwise_and_roi_pred, area_truth, overlap



In [ ]:
def cal_overlaps_per_large_img(roi, pred, large_image):

  contours_pred, _ = cv.findContours(pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  contours_roi, _ = cv.findContours(roi.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

  img = large_image.copy()

  overlap_list = []

  roi_num_cnt = len(contours_roi)

  if  roi_num_cnt == 1:
       bitwise_and_roi_pred = cv.bitwise_and(pred, roi)
       area_truth = np.count_nonzero(bitwise_and_roi_pred)
       overlap = round(area_truth/np.count_nonzero(roi) * 100,2)
       overlap_list.append(overlap)
       print("overlap", overlap)
       cv.drawContours(img, contours_roi , -1, (255,0,0), 3)
       cv.drawContours(img, contours_pred, -1, (0,255,0), 3)

       if overlap!=0:

           contours_overlaps, _ = cv.findContours(bitwise_and_roi_pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

           c = max(contours_overlaps, key = cv.contourArea)

           x, y, w, h = cv.boundingRect(c)

           font = cv.FONT_HERSHEY_SIMPLEX
           org = (x-10, y-10)
           fontScale = 3
           color = (0, 0, 255)
           thickness = 2
           text = str(overlap)
           img = cv.putText(img, text, org, font, fontScale, color, thickness, cv.LINE_AA)

  else:
       bitwise_and_roi_pred = cv.bitwise_and(pred, roi)
       for ct in contours_roi:
            each_roi= np.zeros(roi.shape, dtype="uint8")
            each_roi = cv.fillPoly(each_roi, pts=[ct], color=(255, 255, 255))
            # plt.imshow(each_roi, 'gray')
            # plt.show()
            cv.drawContours(img, ct , -1, (255,0,0), 3)

            for ct_pred in contours_pred:
                  each_pred = np.zeros(pred.shape, dtype="uint8")
                  each_pred = cv.fillPoly(each_pred, pts=[ct_pred], color=(255, 255, 255))

                  # plt.imshow(each_pred, 'gray')
                  # plt.show()

                  bitwise_and_roi_pred_1 = cv.bitwise_and(each_pred, each_roi)

                  area_truth = np.count_nonzero(bitwise_and_roi_pred_1)
                  overlap = round(area_truth/np.count_nonzero(each_roi) *100,3)
                  cv.drawContours(img, ct_pred, -1, (0,255,0), 3)

                  # cv.drawContours(img, ct_pred , -1, (255,0,0), 3)

                  if overlap>0:
                       overlap_list.append(overlap)
                       print("overlap", overlap)

                       x, y, w, h = cv.boundingRect(ct_pred)

                      #  overlap_img_anotation = cv.fillPoly(overlap_img_anotation, pts=[ct_pred], color=(0, 0))

                      #  overlap_img_anotation = ImageDraw.Draw(overlap_img_anotation)
                      #  overlap_img_anotation.text((x, y), overlap, fill=(0 , 0, 255))
                       font = cv.FONT_HERSHEY_SIMPLEX
                       org = (x-10, y-20)
                       fontScale = 3
                       color = (0, 0, 255)
                       thickness = 2
                       text = str(overlap)
                       img = cv.putText(img, text, org, font, fontScale, color, thickness, cv.LINE_AA)
  # cv.imshow(overlap_img_anotation, 0)
  plt.imshow(bitwise_and_roi_pred, 'gray')
  plt.show()
  # plt.imshow(img)
  # plt.show()
  return bitwise_and_roi_pred, overlap_list, img

In [ ]:
predict_path ="/content/drive/MyDrive/Test_INBreast_Predict/Standard Model/original_predict/20586908.png"
roi_path = "/content/INBreast roi 850 remove bkg/roi_thresh_2800_roi_850/20586908.png"
image_path = "/content/INBreast roi 850 remove bkg/image_thresh_2800_roi_850/20586908.png"

predict = cv.imread(predict_path, 0).astype(np.uint8)
predict = np.array(predict)

image = cv.imread(image_path)


roi = cv.imread(roi_path, 0).astype(np.uint8)
roi = np.array(roi)

h_r, w_r = roi.shape



h_i, w_i = predict.shape

if (h_i>=h_r) or (w_i>=w_r):
      predict = predict[0:h_r, 0:w_r]
else:
      roi = roi[0:h_i, 0:w_i]

bitwise_and_roi_pred, overlap_list, img_anotation = cal_overlaps_per_large_img(roi, predict, image)



In [ ]:
def detect_roi_by_patch_size(img, load_model_file, patch_size, step):

        h,w = img.shape

        roi_detect= np.zeros(img.shape, dtype="uint8")

        row = int(( h - patch_size)/step)
        col = int(( w - patch_size)/step)
        new_img_h = row * step + patch_size
        new_img_w = col * step + patch_size

        img_by_patch_size = img[0:new_img_h, 0: new_img_w]

        segmented_image_by_patch_size = prediction(load_model_file, img_by_patch_size, patch_size , step)
        h_1, w_1 = segmented_image_by_patch_size.shape
        roi_detect[0:h_1, 0:w_1] = segmented_image_by_patch_size


        return roi_detect

In [ ]:
csv_path = '/content/drive/MyDrive/Mammo dataset/Test Dataset INbreast/img_mask_path_roi_850_remove_bkg.csv'
pred_path = '/content/drive/MyDrive/Test_INBreast_Predict/Standard Model/original_predict'
overlap_path = '/content/drive/MyDrive/Test_INBreast_Predict/Standard Model/overlap'
draw_contour_path = '/content/drive/MyDrive/Test_INBreast_Predict/Standard Model/draw contour'
overlap_anotation_path = '/content/drive/MyDrive/Test_INBreast_Predict/Standard Model/overlap anotation'

import time

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path


overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    start_time = time.time()

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        # path = os.path.dirname(img_file)
        name_file_saver = os.path.basename(img_file)

        large_image_1 = cv.imread(img_file, 0)
        image_anotation = cv.imread(img_file)
        large_image= np.float32(large_image_1)
        large_image = large_image/255.0

        roi = cv.imread(roi_file, 0).astype(np.uint8)

        # roi = cv.imread(roi_file, 0)
        # roi= np.uint8(roi)
        # print(roi.shape)
        h_r, w_r = roi.shape

        large_image, roi= check_Left_Orient (large_image, roi)

        img = np.array(large_image)

        h_i, w_i = img.shape

        if (h_i>=h_r) or (w_i>=w_r):
          img = img[0:h_r, 0:w_r]
          image_anotation = image_anotation[0:h_r, 0:w_r]
        else:
          roi = roi[0:h_i, 0:w_i]

        # print(img.shape)

        roi_predict_384 = detect_roi_by_patch_size(img, load_model_file, 348, 64)
        print(roi_predict_384.shape)
        roi_predict_512 = detect_roi_by_patch_size(img, load_model_file, 512, 64)

        if w_i < 1024:
          roi_predict_1024 = np.zeros(img.shape, dtype="uint8")
        else:
          roi_predict_1024 = detect_roi_by_patch_size(img, load_model_file, 1024, 128)

        print(roi_predict_384.shape, roi_predict_512.shape, roi_predict_1024.shape)


        combine = np.zeros(roi.shape, dtype=np.uint8)

        combine_1 = np.logical_or(roi_predict_384, roi_predict_512).astype(np.uint8)
        combine = np.logical_or(combine_1, roi_predict_1024).astype(np.uint8)


        y_pred_filter, n = filter_FP(combine, 2000)

        num_P_list.append(n)

        pred_file_path = os.path.join(pred_path,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path, y_pred_filter, cmap='gray')


        overlap_img, overlap_list, img_anotation_1 = cal_overlaps_per_large_img(roi, y_pred_filter, image_anotation)

        # overlap_img_anotation = 1*overlap_img_anotation
        img_overlap_anotation_path = os.path.join(overlap_anotation_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_overlap_anotation_path, img_anotation_1, cmap='gray')
        plt.title(name_file_saver)
        plt.imshow(img_anotation_1)
        plt.show()
        # print(overlap)
        area_truth_list.append(overlap_list)

        # overlap_img = 1*overlap_img
        file_overlap_img_path = os.path.join(overlap_path,
                                 ''.join([name_file_saver]))
        plt.imsave(file_overlap_img_path, overlap_img, cmap='gray')

        img_draw = cv.imread(img_file)
        # combine_predict = cv.imread(pred_file_path, 0)
        img_draw_contour = draw_contour_pred_and_truth(img_draw, y_pred_filter, roi)

        # plt.imshow(img_draw_contour)

        # plt.title(name_file_saver + "- overlap = "+ str(overlap))
        # plt.show()
        img_draw_contour_path = os.path.join(draw_contour_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_draw_contour_path, img_draw_contour, cmap='gray')

print("Time taken: %.2fs" % (time.time() - start_time))


In [ ]:
data = pd.read_csv(csv_path)
img = data["img"]
roi = data["roi"]
import pandas as pd
save_csv_path= '/content/drive/MyDrive/Test_INBreast_Predict/Standard Model/overlap_combine_98_images.csv'
df = pd.DataFrame({'img': img, 'roi': roi, 'overlap': area_truth_list, 'num_FP': num_P_list})
df.to_csv (save_csv_path, index = False, header=True)

In [ ]:
from csv import DictReader
csv_path = '/content/drive/MyDrive/Mammo dataset/Test Dataset INbreast/img_mask_path_roi_850_remove_bkg.csv'
predict_path = '/content/drive/MyDrive/Test_INBreast_Predict/Standard Model/original_predict/'

from csv import DictReader
from pathlib import Path

iou_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)
    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        name_file_saver = os.path.basename(img_file)

        pred_path= os.path.join(predict_path,''.join([name_file_saver]))

        roi = cv.imread(roi_file, 0)
        roi, n_1 = filter_FP(roi, 300)


        pred = cv.imread(pred_path, 0)
        pred, n_1 = filter_FP(pred, 300)

        iou = cal_iou(roi, pred)

        print(iou)
        iou_list.append(iou)


In [ ]:
from csv import DictReader
csv_path = '/content/drive/MyDrive/Mammo dataset/Test Dataset INbreast/img_mask_path_roi_850_remove_bkg.csv'
predict_path = '/content/drive/MyDrive/Test_INBreast_Predict/Model without 2nd skip connection/original_predict/'

from csv import DictReader
from pathlib import Path

iou_list_2nd = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)
    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        name_file_saver = os.path.basename(img_file)

        pred_path= os.path.join(predict_path,''.join([name_file_saver]))

        roi = cv.imread(roi_file, 0)
        roi, n_1 = filter_FP(roi, 300)


        pred = cv.imread(pred_path, 0)
        pred, n_1 = filter_FP(pred, 300)

        iou = cal_iou(roi, pred)

        print(iou)
        iou_list_2nd.append(iou)


In [ ]:
from csv import DictReader
csv_path = '/content/drive/MyDrive/Mammo dataset/Test Dataset INbreast/img_mask_path_roi_850_remove_bkg.csv'
predict_path = '/content/drive/MyDrive/Test_INBreast_Predict/Standard Model/original_predict/'

from csv import DictReader
from pathlib import Path

dice_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)
    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        name_file_saver = os.path.basename(img_file)

        pred_path= os.path.join(predict_path,''.join([name_file_saver]))

        roi = cv.imread(roi_file, 0)
        roi, n_1 = filter_FP(roi, 300)


        pred = cv.imread(pred_path, 0)
        pred, n_1 = filter_FP(pred, 300)

        dice = cal_dice(roi, pred)

        print(dice)
        dice_list.append(dice)


In [ ]:
from csv import DictReader
csv_path = '/content/drive/MyDrive/Mammo dataset/Test Dataset INbreast/img_mask_path_roi_850_remove_bkg.csv'
predict_path = '/content/drive/MyDrive/Test_INBreast_Predict/Model without 2nd skip connection/original_predict/'

from csv import DictReader
from pathlib import Path

dice_list_2nd = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)
    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        name_file_saver = os.path.basename(img_file)

        pred_path= os.path.join(predict_path,''.join([name_file_saver]))

        roi = cv.imread(roi_file, 0)
        roi, n_1 = filter_FP(roi, 300)


        pred = cv.imread(pred_path, 0)
        pred, n_1 = filter_FP(pred, 300)

        dice = cal_dice(roi, pred)

        print(dice)
        dice_list_2nd.append(dice)


In [ ]:
import pandas as pd
df = pd.DataFrame({'standard_dice': dice_list, '2nd_dice': dice_list_2nd, 'standard_iou': iou_list,'2nd_iou': iou_list_2nd })
df.to_csv ('/content/drive/MyDrive/Mammo dataset/dice_iou_INBREAST.csv', index = False, header=True)

#### vẽ trên cả 2 mô hình

In [ ]:
image_path = "/content/INBreast roi 850 remove bkg/image_thresh_2800_roi_850/50995762.png"
roi_path = "/content/INBreast roi 850 remove bkg/roi_thresh_2800_roi_850/50995762.png"
predict_path = "/content/drive/MyDrive/Test_INBreast_Predict/Standard Model/original_predict/50995762.png"
predict_2nd_path ="/content/drive/MyDrive/Test_INBreast_Predict/Model without 2nd skip connection/original_predict/50995762.png"

save_path = "/content/drive/MyDrive/Test DDSM 185 Images"
name_file_saver = os.path.basename(image_path)
predict = cv.imread(predict_path, 0).astype(np.uint8)
predict = np.array(predict)

predict_2nd = cv.imread(predict_2nd_path, 0).astype(np.uint8)
predict_2nd = np.array(predict_2nd)

image = cv.imread(image_path)

roi = cv.imread(roi_path, 0).astype(np.uint8)
roi = np.array(roi)

large_image_roi_predict_pred_2nd_overlap(image, roi, predict, predict_2nd)

In [ ]:
image_path = "/content/INBreast roi 850 remove bkg/image_thresh_2800_roi_850/24055445.png"
roi_path = "/content/INBreast roi 850 remove bkg/roi_thresh_2800_roi_850/24055445.png"
predict_path = "/content/drive/MyDrive/Test_INBreast_Predict/Standard Model/original_predict/24055445.png"
predict_2nd_path ="/content/drive/MyDrive/Test_INBreast_Predict/Model without 2nd skip connection/original_predict/24055445.png"

save_path = "/content/drive/MyDrive/Test DDSM 185 Images"
name_file_saver = os.path.basename(image_path)
predict = cv.imread(predict_path, 0).astype(np.uint8)
predict = np.array(predict)

predict_2nd = cv.imread(predict_2nd_path, 0).astype(np.uint8)
predict_2nd = np.array(predict_2nd)

image = cv.imread(image_path)

roi = cv.imread(roi_path, 0).astype(np.uint8)
roi = np.array(roi)

large_image_roi_predict_pred_2nd_overlap(image, roi, predict, predict_2nd)

In [ ]:
image_path = "/content/INBreast roi 850 remove bkg/image_thresh_2800_roi_850/50996406.png"
roi_path = "/content/INBreast roi 850 remove bkg/roi_thresh_2800_roi_850/50996406.png"
predict_path = "/content/drive/MyDrive/Test_INBreast_Predict/Standard Model/original_predict/50996406.png"
predict_2nd_path ="/content/drive/MyDrive/Test_INBreast_Predict/Model without 2nd skip connection/original_predict/50996406.png"

save_path = "/content/drive/MyDrive/Test DDSM 185 Images"
name_file_saver = os.path.basename(image_path)
predict = cv.imread(predict_path, 0).astype(np.uint8)
predict = np.array(predict)

predict_2nd = cv.imread(predict_2nd_path, 0).astype(np.uint8)
predict_2nd = np.array(predict_2nd)

image = cv.imread(image_path)

roi = cv.imread(roi_path, 0).astype(np.uint8)
roi = np.array(roi)

large_image_roi_predict_pred_2nd_overlap(image, roi, predict, predict_2nd)

In [ ]:
image_path = "/content/INBreast roi 850 remove bkg/image_thresh_2800_roi_850/50996352.png"
roi_path = "/content/INBreast roi 850 remove bkg/roi_thresh_2800_roi_850/50996352.png"
predict_path = "/content/drive/MyDrive/Test_INBreast_Predict/Standard Model/original_predict/50996352.png"
predict_2nd_path ="/content/drive/MyDrive/Test_INBreast_Predict/Model without 2nd skip connection/original_predict/50996352.png"

save_path = "/content/drive/MyDrive/Test DDSM 185 Images"
name_file_saver = os.path.basename(image_path)
predict = cv.imread(predict_path, 0).astype(np.uint8)
predict = np.array(predict)

predict_2nd = cv.imread(predict_2nd_path, 0).astype(np.uint8)
predict_2nd = np.array(predict_2nd)

image = cv.imread(image_path)

roi = cv.imread(roi_path, 0).astype(np.uint8)
roi = np.array(roi)

large_image_roi_predict_pred_2nd_overlap(image, roi, predict, predict_2nd)

In [ ]:
image_path = "/content/INBreast roi 850 remove bkg/image_thresh_2800_roi_850/20586986.png"
roi_path = "/content/INBreast roi 850 remove bkg/roi_thresh_2800_roi_850/20586986.png"
predict_path = "/content/drive/MyDrive/Test_INBreast_Predict/Standard Model/original_predict/20586986.png"
predict_2nd_path ="/content/drive/MyDrive/Test_INBreast_Predict/Model without 2nd skip connection/original_predict/20586986.png"

save_path = "/content/drive/MyDrive/Test DDSM 185 Images"
name_file_saver = os.path.basename(image_path)
predict = cv.imread(predict_path, 0).astype(np.uint8)
predict = np.array(predict)

predict_2nd = cv.imread(predict_2nd_path, 0).astype(np.uint8)
predict_2nd = np.array(predict_2nd)

image = cv.imread(image_path)

roi = cv.imread(roi_path, 0).astype(np.uint8)
roi = np.array(roi)

large_image_roi_predict_pred_2nd_overlap(image, roi, predict, predict_2nd)

###Final Solution for DDSM (NORMAL FROM DDSM, MULTITUMOR FROM TEST SET CBIS

####Test Patch (133 mass images)

In [ ]:
!pip install patchify

In [ ]:
!unrar x "//content/drive/MyDrive/Mammo dataset/Test DDSM/Test_bkg.rar"

In [ ]:
drive_path = '/content/drive/MyDrive'
dataset_path = '/content'

files_dir = os.path.join(drive_path, 'Colab Notebooks', 'files_15_5_23', 'training_dataset_20_bkg')
model_file = os.path.join(files_dir,'unet_effb0-new-data-20_bkg-15_5_2023.h5')

test_x= sorted(glob(os.path.join(dataset_path, 'Test_bkg', 'Patch', '*')))
test_y= sorted(glob(os.path.join(dataset_path, 'Test_bkg', 'Mask', '*')))

test_y_pred= sorted(glob(os.path.join(drive_path, 'Mammo dataset', 'predict test 133 images patch', '*')))

In [ ]:
print(len(test_y_pred))

In [ ]:
test_set_y_pred = np.ndarray(shape = (len(test_y_pred), 256, 256, 1), dtype= np.uint8)
for n, file_path in enumerate(test_y_pred):
        roi = cv.imread(file_path, 0)
        roi = np.array(roi)
        # roi = roi / 255.
        # print(file_path)
        roi = np.expand_dims(roi, -1)
        test_set_y_pred[n] = roi

In [ ]:
test_y_pred_2nd= sorted(glob(os.path.join(drive_path, 'Mammo dataset', 'predict test 133 images patch without 2 nd', '*')))
test_set_y_pred_2nd = np.ndarray(shape = (len(test_y_pred_2nd), 256, 256, 1), dtype= np.uint8)
for n, file_path in enumerate(test_y_pred_2nd):
        roi = cv.imread(file_path, 0)
        roi = np.array(roi)
        # roi = roi / 255.
        # print(file_path)
        roi = np.expand_dims(roi, -1)
        test_set_y_pred_2nd[n] = roi

In [ ]:
import pandas as pd
df = pd.DataFrame({'img': test_x, 'roi': test_y})
df.to_csv ('/content/drive/MyDrive/Mammo dataset/Test DDSM/patch_mask_Test_set_bkg.csv', index = False, header=True)

In [ ]:
batch_size = 32
test_dataset = tf_dataset(test_x, test_y, batch = batch_size)

In [ ]:
n = 1
for x, y in test_dataset:
  if n in [1,5, 7, 8, 9]:
  # print(x.shape, y.shape)
  # print(y[0].dtype)
    plt.subplot(1,2,1)
    plt.imshow(x[0], cmap='gray')
    plt.subplot(1,2,2)
    plt.imshow(np.squeeze(y[0]), cmap='gray')
    plt.show()
  n = n + 1

In [ ]:
def create_h5_file1(csv_file, h5_path, filename_x, filename_y):
    col_list = [filename_x, filename_y]
    df = pd.read_csv(csv_file, usecols=col_list)

    x_train = df[filename_x]
    y_train = df[filename_y]

    # x_train.sort()
    # y_train.sort()

    IMG_HEIGHT=256
    IMG_WIDTH=256
    IMG_CHANNELS=3


    # X_train = np.zeros((len(x_train), IMG_HEIGHT, IMG_WIDTH, 3), dtype= np.uint8)
    X_train = np.ndarray(shape = (len(x_train), IMG_HEIGHT, IMG_WIDTH, 3), dtype=np.float32)
    Y_train = np.ndarray(shape = (len(y_train), IMG_HEIGHT, IMG_WIDTH, 1), dtype= np.bool_)

    x_train.sort_values()
    y_train.sort_values()

    for n, file_path in enumerate(x_train):
        img = cv.imread(file_path)
        img = np.array(img)
        img = img / 255.
        # print(file_path)
        X_train[n] = img
    #
    for n, file_path in enumerate(y_train):
        roi = cv.imread(file_path, 0)
        roi = np.array(roi)
        # roi = roi / 255.
        # print(file_path)
        roi = np.expand_dims(roi, -1)
        Y_train[n] = roi

    print("number of X examples = " + str(X_train.shape[0]))
    print("X shape: " + str(X_train.shape))
    print("Y shape: " + str(Y_train.shape))
    # print('pixel values in the mask are: ', np.unique(Y_train))

    # return X_train, Y_train

    with h5py.File(h5_path, 'w') as hf:
        dset_x_train = hf.create_dataset('x', data=X_train, shape=(X_train.shape), compression='gzip',
                                         chunks=True)
        dset_y_train = hf.create_dataset('y', data=Y_train, shape=(Y_train.shape), compression='gzip',
                                         chunks=True)
    hf.close()
    print('Done')

In [ ]:
create_h5_file1('/content/drive/MyDrive/Mammo dataset/Test DDSM/patch_mask_Test_set_bkg.csv', '/content/drive/MyDrive/Mammo dataset/Test DDSM/patch_mask_Test_set.h5','img', 'roi')


In [ ]:
def read_h5_file(h5_path):
    test_dataset = h5py.File(h5_path, "r")
    test_set_x_orig = np.array(test_dataset["x"][:])  # your train set features
    test_set_y_orig = np.array(test_dataset["y"][:])

    print("number of training examples = " + str(test_set_x_orig.shape[0]))
    print("X_train shape: " + str(test_set_x_orig.shape))
    print("Y_train shape: " + str(test_set_y_orig.shape))

    return test_set_x_orig, test_set_y_orig

In [ ]:
test_set_x, test_set_y = read_h5_file('/content/drive/MyDrive/Mammo dataset/Test DDSM/patch_mask_Test_set.h5')

In [ ]:
def show_roi_predict_dice(roi, predict, name):
    dice = dice_coef(roi, predict )

    plt.subplot(121)
    plt.imshow(roi,'gray')
    plt.title('roi' + str(dice))
    plt.axis('off')

    plt.subplot(122)
    plt.axis('off')
    plt.imshow(predict, 'gray')
    plt.title('pred ' )


In [ ]:
def show_image_roi(randomlist, x, y, name):
  fg, ax = plt.subplots(2, 5, figsize=(20, 8))
  fg.suptitle(name)

  for i, index in enumerate(randomlist):
    ax[0, i].imshow(x[index], 'gray')
    ax[0, i].set_xlabel('Image ' + str(i+1))

  for i, index in enumerate(randomlist):
    ax[1, i].imshow(np.squeeze(y[index]), 'gray')
    ax[1, i].set_xlabel('ROI ' + str(i+1))

In [ ]:
randomlist = random.sample(range(0, len(test_set_x)),5)
print(randomlist)
show_image_roi(randomlist, test_set_x, test_set_y,'Image and ROi of Train Set')

In [ ]:
load_model_file = tf.keras.models.load_model(model_file)

In [ ]:
loss, acc= load_model_file.evaluate(test_dataset, verbose=2)


In [ ]:
preds_test = load_model_file.predict(test_set_x, verbose=1)
test_set_y_pred = (preds_test > 0.5).astype(np.float32)

In [ ]:
print(len(test_set_y_pred))

In [ ]:
print(test_set_y_pred.shape)

In [ ]:
def dice_coef(groundtruth_mask, pred_mask):
    intersect = np.sum(pred_mask*groundtruth_mask)
    total_sum = np.sum(pred_mask) + np.sum(groundtruth_mask)
    dice = np.mean(100*2*intersect/total_sum)
    return round(dice, 3)

In [ ]:
def compute_dice(label_img, pred_img, p_threshold=0.5):
    p = pred_img.astype(np.float)
    l = label_img.astype(np.float)
    if p.max() > 127:
        p /= 255.
    if l.max() > 127:
        l /= 255.

    p = np.clip(p, 0, 1.0)
    l = np.clip(l, 0, 1.0)
    p[p > 0.5] = 1.0
    p[p < 0.5] = 0.0
    l[l > 0.5] = 1.0
    l[l < 0.5] = 0.0
    product = np.dot(l.flatten(), p.flatten())
    dice_num = 2 * product + 1
    pred_sum = p.sum()
    label_sum = l.sum()
    dice_den = pred_sum + label_sum + 1
    dice_val = 100*dice_num / dice_den
    return round(dice_val,2)

In [ ]:
def compute_dice_1(label_img, pred_img, p_threshold=0.5):
    p = pred_img.astype(np.float)
    l = label_img.astype(np.float)
    if p.max() > 127:
        p /= 255.
    if l.max() > 127:
        l /= 255.

    p = np.clip(p, 0, 1.0)
    l = np.clip(l, 0, 1.0)
    p[p > 0.5] = 1.0
    p[p < 0.5] = 0.0
    l[l > 0.5] = 1.0
    l[l < 0.5] = 0.0
    product = ((l.flatten())*(p.flatten())).sum()
    dice_num = 2 * product + 1
    pred_sum = p.sum()
    label_sum = l.sum()
    dice_den = pred_sum + label_sum + 1
    dice_val = 100*dice_num / dice_den
    return round(dice_val,2)


In [ ]:
def compute_iou(label_img, pred_img, p_threshold=0.5):
    p = pred_img.astype(np.float)
    l = label_img.astype(np.float)
    if p.max() > 127:
        p /= 255.
    if l.max() > 127:
        l /= 255.

    p = np.clip(p, 0, 1.0)
    l = np.clip(l, 0, 1.0)
    p[p > 0.5] = 1.0
    p[p < 0.5] = 0.0
    l[l > 0.5] = 1.0
    l[l < 0.5] = 0.0

    product = ((l.flatten())* (p.flatten())).sum()
    iou_num = product + 1
    pred_sum = p.sum()
    label_sum = l.sum()
    iou_den = pred_sum + label_sum + 1 - product
    iou = 100*iou_num / iou_den

    return round(iou,2)


In [ ]:
from csv import DictReader
csv_path = '/content/drive/MyDrive/Mammo dataset/Test DDSM/patch_mask_Test_set_bkg.csv'
predict_path = "/content/drive/MyDrive/Mammo dataset/predict test 133 images patch/"

from csv import DictReader
from pathlib import Path


dice_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)
    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        name_file_saver = os.path.basename(img_file)

        pred_path= os.path.join(predict_path,''.join([name_file_saver]))



        roi = cv.imread(roi_file)
        # roi_bw = cv.threshold(roi, 1 , 255, cv.THRESH_BINARY)
        # roi = np.array(roi_bw)


        pred = cv.imread(pred_path)
        # kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
        # pred = cv.morphologyEx(pred, cv.MORPH_CLOSE, kernel)
        # pred = cv.morphologyEx(pred, cv.MORPH_OPEN, kernel)
        # pred_bw = cv.threshold(pred, 1 , 255, cv.THRESH_BINARY)
        # pred1 = np.array(pred_bw)

        dice = compute_dice_1(roi, pred)

        # pred_bw = cv.threshold(pred, 1 , 255, cv.THRESH_BINARY)
        # show_roi_predict_dice(roi, pred, name_file_saver)


        print(dice)
        dice_list.append(dice)



In [ ]:
from csv import DictReader
csv_path = '/content/drive/MyDrive/Mammo dataset/Test DDSM/patch_mask_Test_set_bkg.csv'
predict_path = "/content/drive/MyDrive/Mammo dataset/predict test 133 images patch without 2 nd/"

from csv import DictReader
from pathlib import Path


dice_list_2nd = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)
    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        name_file_saver = os.path.basename(img_file)

        pred_path= os.path.join(predict_path,''.join([name_file_saver]))

        roi = cv.imread(roi_file)

        pred = cv.imread(pred_path)

        dice = compute_dice_1(roi, pred)

        print(dice)
        dice_list_2nd.append(dice)


In [ ]:
import pandas as pd
df = pd.DataFrame({'standard': dice_list, '2nd': dice_list_2nd })
df.to_csv ('/content/drive/MyDrive/Mammo dataset/dice_cbis_test_set_bkg_5670_1.csv', index = False, header=True)


In [ ]:
def show_image_roi_predict(randomlist, x, y, predict, name):
  fg, ax = plt.subplots(3, 5, figsize=(20, 8))
  fg.suptitle(name)

  for i, index in enumerate(randomlist):
    ax[0, i].imshow(x[index], 'gray')
    ax[0, i].set_xlabel('Image ' + str(index))

  for i, index in enumerate(randomlist):
    ax[1, i].imshow(np.squeeze(y[index]), 'gray')
    ax[1, i].set_xlabel('ROI ' + str(index))

  for i, index in enumerate(randomlist):
    ax[2, i].imshow(np.squeeze(predict[index]), 'gray')
    ax[2, i].set_xlabel('Predict ' + str(index))

In [ ]:
randomlist = random.sample(range(0, 100),5)
print(randomlist)
show_image_roi_predict(randomlist, test_set_x, test_set_y, test_set_y_pred, 'Train set')

In [ ]:
randomlist = random.sample(range(0, 100),5)
print(randomlist)
show_image_roi_predict(randomlist, test_set_x, test_set_y, test_set_y_pred, 'Train set')

In [ ]:
randomlist = random.sample(range(0, 100),5)
print(randomlist)
show_image_roi_predict(randomlist, test_set_x, test_set_y, test_set_y_pred, 'Train set')

In [ ]:
randomlist = random.sample(range(0, 100),5)
print(randomlist)
show_image_roi_predict(randomlist, test_set_x, test_set_y, test_set_y_pred, 'Train set')

In [ ]:
import pandas as pd
df = pd.DataFrame({'overlap': overlap_list, 'iou': iou_list, 'dice': dice_list })
df.to_csv ('/content/drive/MyDrive/test INBreast patch/overlap_iou_dice_inbreast_bb_850.csv', index = False, header=True)

In [ ]:
def cal_overlap(x,y,y_pred):
  bitwise_and_roi_patch = np.logical_and(np.squeeze(y), np.squeeze(y_pred))
  overlap = round(np.sum(bitwise_and_roi_patch) / np.sum(np.squeeze(y)), 2)
  return overlap

In [ ]:
show_image_roi_predict1_img(42, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(42, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(6, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(6, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(99, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(99, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(8, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(8, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(13, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(30, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(30, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(7, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(9, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(29, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(153, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(248, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(55, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(75, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(149, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(26, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(133, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(149, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(168, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(30, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(1798, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(1065, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(5118, test_set_x, test_set_y, test_set_y_pred)


In [ ]:
show_image_roi_predict1_img(989, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(1208, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
show_image_roi_predict1_img(1065, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
randomlist = random.sample(range(0, 5470),5)

for i, index in enumerate(randomlist):
    show_image_roi_predict1_img(index, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
randomlist = random.sample(range(0, 5470),5)

for i, index in enumerate(randomlist):
    show_image_roi_predict1_img(index, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
randomlist = random.sample(range(0, 5470),5)

for i, index in enumerate(randomlist):
    show_image_roi_predict1_img(index, test_set_x, test_set_y, test_set_y_pred)

In [ ]:
randomlist = random.sample(range(0, 5470),5)

for i, index in enumerate(randomlist):
    show_image_roi_predict1_img(index, test_set_x, test_set_y, test_set_y_pred)

####Calculate overlap test set pacth

In [ ]:
def image_roi_predict_overlap(index, x, y, predict):
    overlap = cal_overlap(x[index], y[index], predict[index])
    print (overlap)

    roi = np.squeeze(y[index])
    plt.subplot(131)
    plt.axis('off')
    plt.imshow(roi, 'gray')
    plt.title('ROI ' + str(index))
    image_8bit = np.uint8(roi * 255)
    _, binarized = cv.threshold(image_8bit, 0, 255, cv.THRESH_BINARY)
    contours_roi, hierarchy = cv.findContours(binarized, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)


    roi_pred = np.squeeze(predict[index])
    plt.subplot(132)
    plt.axis('off')
    plt.imshow(roi_pred)
    plt.title('ROI pred ' + str(index))
    image_8bit = np.uint8(roi_pred * 255)
    _, binarized = cv.threshold(image_8bit, 0, 255, cv.THRESH_BINARY)
    contours_pred, hierarchy = cv.findContours(binarized,cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    img = x[index]
    img_copy = img.copy()
    cv.drawContours(img_copy, contours_roi,-1,(255,0,0),2)
    cv.drawContours(img_copy, contours_pred,-1, (0,255,0), 2)
    plt.subplot(133)
    plt.imshow(img_copy)
    plt.title('Image' + str(index))
    plt.axis('off')
    plt.show()


In [ ]:
def cal_overlap_label(x,y,y_pred):
  overlap = 0
  label_y = 0
  label_y_pred = 0
  num_y = np.sum(np.squeeze(y))


  kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
  y_pred_filter = cv.morphologyEx(np.squeeze(y_pred), cv.MORPH_CLOSE, kernel)
  y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)

  num_y_pred = np.sum(y_pred_filter)

  bitwise_and_roi_patch = np.logical_and(np.squeeze(y), np.squeeze(y_pred_filter))


  if (num_y == 0) & (num_y_pred == 0):
    label_y = 0
    label_y_pred = 0
    overlap = 1.0
  elif (num_y == 0) & (num_y_pred != 0):
    label_y = 0
    label_y_pred = 1
    overlap = 0
  elif (num_y != 0) & (num_y_pred == 0):
    label_y = 1
    label_y_pred = 0
    overlap = 0
  else:
    overlap = round(np.sum(bitwise_and_roi_patch) / np.sum(np.squeeze(y)), 2)
    if overlap>= 0.5:
          label_y = 1
          label_y_pred = 1
    else:
          label_y = 1
          label_y_pred = 0

  return overlap, label_y, label_y_pred

In [ ]:
overlaps = []
labels_y = []
labels_y_pred = []
for i in range(len(test_set_y)):
  x = test_set_x[i]
  y = test_set_y[i]
  y_pred = test_set_y_pred[i]
  overlap, label_y, label_y_pred = cal_overlap_label(x,y,y_pred)
  overlaps.append(overlap)
  labels_y.append(label_y)
  labels_y_pred.append(label_y_pred)

In [ ]:
import pandas as pd
df = pd.DataFrame({'overlap': overlaps, 'label_y': labels_y, 'label_y_pred': labels_y_pred })
df.to_csv ('/content/drive/MyDrive/Mammo dataset/Test DDSM/overlap_Test_set_bkg_label_1.csv', index = False, header=True)

In [ ]:
def create_label(data):
  y_label = []
  for i in range(len(data)):
    patch_pixel_non_zero = np.sum(np.squeeze(data[i]))
    if (patch_pixel_non_zero == 0):
      y_label.append(0)
    else:
      y_label.append(1)
  print(len(y_label))
  return y_label

In [ ]:
test_set_y_label = create_label(test_set_y)
test_set_y_pred_label = create_label(test_set_y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
import seaborn as sn
cm = confusion_matrix(test_set_y_label, test_set_y_pred_label) # rows = truth, cols = prediction
df_cm = pd.DataFrame(cm, index = (0, 1), columns = (0, 1))
plt.figure(figsize = (10,7))
plt.title('confusion matrix - Test data')
sn.set(font_scale=1.4)
sn.heatmap(df_cm, annot=True, fmt='g')

In [ ]:
total1=sum(sum(cm))
#####from confusion matrix calculate accuracy
accuracy1=(cm[0,0]+cm[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cm[0,0]/(cm[0,0]+cm[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cm[1,1]/(cm[1,0]+cm[1,1])
print('Specificity : ', specificity1)

In [ ]:
def cal_overlap_label(x,y,y_pred):
  overlap = 0
  label_y = 0
  label_y_pred = 0
  num_y = np.sum(np.squeeze(y))


  kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
  y_pred_filter = cv.morphologyEx(np.squeeze(y_pred), cv.MORPH_CLOSE, kernel)
  y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)

  num_y_pred = np.sum(y_pred_filter)

  bitwise_and_roi_patch = np.logical_and(np.squeeze(y), np.squeeze(y_pred_filter))

  if (num_y == 0) & (num_y_pred == 0):
    label_y = 0
    label_y_pred = 0
    overlap = 1.0
  elif (num_y == 0) & (num_y_pred != 0):
    label_y = 0
    label_y_pred = 1
    overlap = 0
  elif (num_y != 0) & (num_y_pred == 0):
    label_y = 1
    label_y_pred = 0
    overlap = 0
  else:
    overlap = round(np.sum(bitwise_and_roi_patch) / np.sum(np.squeeze(y)), 2)
    if overlap>= 0.5:
          label_y = 1
          label_y_pred = 1
    else:
          label_y = 1
          label_y_pred = 0

  return overlap, label_y, label_y_pred

In [ ]:
def create_label_map(y, y_pred):
  y_label = []
  y_pred_label = []

  for i in range(len(y)):
    patch_pixel_non_zero_y = np.sum(np.squeeze(y[i]))
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
    y_pred_filter = cv.morphologyEx(np.squeeze(y_pred[i]), cv.MORPH_CLOSE, kernel)
    y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)

    patch_pixel_non_zero_y_pred = np.sum(y_pred_filter)

    if (patch_pixel_non_zero_y == 0)& (patch_pixel_non_zero_y_pred == 0):
      y_label.append(0)
      y_pred_label.append(0)
    elif (patch_pixel_non_zero_y == 0)& (patch_pixel_non_zero_y_pred != 0):
      y_label.append(0)
      y_pred_label.append(1)

    elif (patch_pixel_non_zero_y != 0)& (patch_pixel_non_zero_y_pred == 0):
      y_label.append(1)
      y_pred_label.append(0)

    else:
      bitwise_and_roi_patch = np.logical_and(np.squeeze(y), np.squeeze(y_pred))
      overlap = round(np.sum(bitwise_and_roi_patch) / np.sum(np.squeeze(y)), 2)

      if overlap >= 0.5:
         y_label.append(1)
         y_pred_label.append(1)
      else:
         y_label.append(1)
         y_pred_label.append(0)

  return y_label, y_pred_label

In [ ]:
y_label, y_pred_label = create_label_map(test_set_y,test_set_y_pred )

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
import seaborn as sn
cm = confusion_matrix(y_label, y_pred_label) # rows = truth, cols = prediction
df_cm = pd.DataFrame(cm, index = (0, 1), columns = (0, 1))
plt.figure(figsize = (10,7))
plt.title('confusion matrix - Test data')
sn.set(font_scale=1.4)
sn.heatmap(df_cm, annot=True, fmt='g')

In [ ]:
total1=sum(sum(cm))
#####from confusion matrix calculate accuracy
accuracy1=(cm[0,0]+cm[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cm[0,0]/(cm[0,0]+cm[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cm[1,1]/(cm[1,0]+cm[1,1])
print('Specificity : ', specificity1)

In [ ]:
from sklearn.metrics import confusion_matrix

# y_pred_2 = test_set_y_pred.ravel()
# test_set_y_2 = test_set_y.ravel()
cm=confusion_matrix(labels_y, labels_y_pred)
print(cm)

In [ ]:
import seaborn as sn

df_cm = pd.DataFrame(cm, index = (0, 1), columns = (0, 1))
plt.figure(figsize = (10,7))
plt.title('confusion matrix - Test data')
sn.set(font_scale=1.4)
sn.heatmap(df_cm, annot=True, fmt='g')

In [ ]:
total1=sum(sum(cm))
#####from confusion matrix calculate accuracy
accuracy1=(cm[0,0]+cm[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cm[0,0]/(cm[0,0]+cm[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cm[1,1]/(cm[1,0]+cm[1,1])
print('Specificity : ', specificity1)

In [ ]:
def image_roi_predict_pred_2nd_overlap(index, x, y, predict, pred_2nd):
    # overlap = cal_overlap(x[index], y[index], predict[index])
    # print (overlap)

    roi = np.squeeze(y[index])
    # print(roi.dtype)
    image_8bit = np.uint8(roi*255)
    _, binarized = cv.threshold(image_8bit, 0, 255, cv.THRESH_BINARY)
    contours_roi, hierarchy = cv.findContours(binarized, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
    y_pred_filter = cv.morphologyEx(np.squeeze(predict[index]), cv.MORPH_CLOSE, kernel)
    y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)
    # roi_pred = np.squeeze(predict[index])
    # print(roi_pred.dtype)
    # image_8bit = np.uint8(roi_pred*255)
    _, binarized = cv.threshold(y_pred_filter, 0, 255, cv.THRESH_BINARY)
    contours_pred, hierarchy = cv.findContours(binarized,cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    y_pred_filter_2nd = cv.morphologyEx(np.squeeze(pred_2nd[index]), cv.MORPH_CLOSE, kernel)
    y_pred_filter_2nd = cv.morphologyEx(y_pred_filter_2nd, cv.MORPH_OPEN, kernel)
    # roi_pred_2nd = np.squeeze(pred_2nd[index])
    # print(roi_pred_2nd.dtype)
    # image_8bit_2nd = np.uint8(roi_pred_2nd*255)
    _, binarized_2nd = cv.threshold(y_pred_filter_2nd, 0, 255, cv.THRESH_BINARY)
    contours_pred_2nd, hierarchy = cv.findContours(binarized_2nd,cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    img = x[index]
    img_copy = img.copy()
    cv.drawContours(img_copy, contours_roi,-1,(255,0,0),2)
    cv.drawContours(img_copy, contours_pred,-1, (0,255,0), 2)
    cv.drawContours(img_copy, contours_pred_2nd,-1, (0,0,255), 2)
    plt.imshow(img_copy)
    plt.axis('off')
    plt.title('Image' + str(index))
    plt.show()

In [ ]:
image_roi_predict_pred_2nd_overlap(6, test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)


In [ ]:
image_roi_predict_pred_2nd_overlap(38, test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)


In [ ]:
image_roi_predict_pred_2nd_overlap(69, test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)


In [ ]:
image_roi_predict_pred_2nd_overlap(112, test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)


In [ ]:
image_roi_predict_pred_2nd_overlap(158, test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)


In [ ]:
image_roi_predict_pred_2nd_overlap(286, test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)


In [ ]:
image_roi_predict_pred_2nd_overlap(284, test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)


In [ ]:
image_roi_predict_pred_2nd_overlap(359, test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)


In [ ]:
image_roi_predict_pred_2nd_overlap(365, test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)


In [ ]:
image_roi_predict_pred_2nd_overlap(426, test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)


In [ ]:
image_roi_predict_pred_2nd_overlap(481, test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)


In [ ]:
image_roi_predict_pred_2nd_overlap(495, test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)


In [ ]:
image_roi_predict_pred_2nd_overlap(522, test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)


In [ ]:
image_roi_predict_pred_2nd_overlap(594, test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)


In [ ]:
image_roi_predict_pred_2nd_overlap(42, test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(1789, test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(1065, test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(989, test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(3513, test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(1310, test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(4027, test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)


In [ ]:
image_roi_predict_pred_2nd_overlap(4151, test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)


In [ ]:
image_roi_predict_pred_2nd_overlap(2085, test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(4275, test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(4242, test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(2034, test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(575, test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
randomlist = random.sample(range(0, 5470),5)

for i, index in enumerate(randomlist):
  image_roi_predict_pred_2nd_overlap(index,
  test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
randomlist = random.sample(range(0, 5470),5)

for i, index in enumerate(randomlist):
  image_roi_predict_pred_2nd_overlap(index,
  test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
randomlist = random.sample(range(0, 5470),5)

for i, index in enumerate(randomlist):
  image_roi_predict_pred_2nd_overlap(index,
  test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(149,
  test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(133,
  test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(26,
  test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(4986,
  test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(4027,
  test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(11,
  test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(51,
  test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(54,
  test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(70,
  test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(129,
  test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(152,
  test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(162,
  test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(275,
  test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(311,
  test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(762,
  test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(771,
  test_set_x, test_set_y, test_set_y_pred, test_set_y_pred_2nd)

#### plot ROC, AUC

In [ ]:
train_x = sorted(glob(os.path.join(dataset_path, 'New_Training_20k_bkg', 'Patch', '*')))
train_y = sorted(glob(os.path.join(dataset_path, 'New_Training_20k_bkg', 'Mask', '*')))

In [ ]:
import pandas as pd
df = pd.DataFrame({'img': train_x, 'roi': train_y})
df.to_csv ('/content/drive/MyDrive/Mammo dataset/patch_mask_Train_set.csv', index = False, header=True)

In [ ]:
train_set_x, train_set_y =  create_h5_file1('/content/drive/MyDrive/Mammo dataset/patch_mask_Train_set.csv','img', 'roi')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve

In [ ]:
y_preds = load_model_file.predict(test_set_x).ravel()

In [ ]:
test_set_y_1 = test_set_y.ravel()
# test_set_y_pred_1 = test_set_y_pred.ravel()

In [ ]:
fpr, tpr, thresholds = roc_curve(test_set_y_1, y_preds)
plt.figure(1)
plt.plot([0, 1], [0, 1], 'y--')
plt.plot(fpr, tpr, marker='.')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.show()

In [ ]:
test_set_y_pred_2nd_1 = test_set_y_pred_2nd.ravel()

In [ ]:
fpr_2nd, tpr_2nd, thresholds_2nd = roc_curve(test_set_y_1, test_set_y_pred_2nd_1)
plt.figure(1)
plt.plot([0, 1], [0, 1], 'y--')
plt.plot(fpr, tpr, marker='.')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.show()

In [ ]:
fpr, tpr, _ = metrics.roc_curve(np.squeeze(test_set_y),  np.squeeze(test_set_y_pred))

In [ ]:
log_regression = LogisticRegression()

#fit the model using the training data
log_regression.fit(train_set_x,train_set_y)

In [ ]:
y_pred_proba = log_regression.predict_proba(test_set_x)[::,1]
fpr, tpr, _ = metrics.roc_curve(test_set_y,  y_pred_proba)

#create ROC curve
plt.plot(fpr,tpr)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#define metrics
y_pred_proba = log_regression.predict_proba(test_set_x)[::,1]
fpr, tpr, _ = metrics.roc_curve(test_set_y,  y_pred_proba)
auc = metrics.roc_auc_score(test_set_y, y_pred_proba)

#create ROC curve
plt.plot(fpr,tpr,label="AUC="+str(auc))
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.show()

#### tính lại dice, iou, overlap, confusion matrix

In [ ]:
!unrar x "/content/drive/MyDrive/Mammo dataset/predict test 133 images patch.rar"

In [ ]:
!unrar x "/content/drive/MyDrive/Mammo dataset/predict test 133 images patch without 2 nd.rar"

In [ ]:
drive_path = '/content/drive/MyDrive'
dataset_path = '/content'

files_dir = os.path.join(drive_path, 'Colab Notebooks', 'files_15_5_23', 'training_dataset_20_bkg')
model_file = os.path.join(files_dir,'unet_effb0-new-data-20_bkg-15_5_2023.h5')

test_x= sorted(glob(os.path.join(dataset_path, 'Test_bkg', 'Patch', '*')))
test_y= sorted(glob(os.path.join(dataset_path, 'Test_bkg', 'Mask', '*')))

In [ ]:
test_y_pred= sorted(glob(os.path.join(dataset_path, 'predict test 133 images patch', '*')))
test_y_pred_2nd= sorted(glob(os.path.join(dataset_path, 'predict test 133 images patch without 2 nd', '*')))

In [ ]:
import pandas as pd
df = pd.DataFrame({'img': test_x, 'roi': test_y, 'y_pred': test_y_pred, 'y_pred_2nd': test_y_pred_2nd})
df.to_csv ('/content/drive/MyDrive/Mammo dataset/Test DDSM/patch_mask__pred_Test_set_bkg.csv', index = False, header=True)

In [ ]:
test_set_x, test_set_y = read_h5_file('/content/drive/MyDrive/Mammo dataset/Test DDSM/patch_mask_Test_set.h5')

In [ ]:
def compute_dice_1(label_img, pred_img, p_threshold=0.5):
    p = pred_img.astype(np.float)
    l = label_img.astype(np.float)
    if p.max() > 127:
        p /= 255.
    if l.max() > 127:
        l /= 255.

    p = np.clip(p, 0, 1.0)
    l = np.clip(l, 0, 1.0)
    p[p > 0.5] = 1.0
    p[p < 0.5] = 0.0
    l[l > 0.5] = 1.0
    l[l < 0.5] = 0.0
    product = ((l.flatten())*(p.flatten())).sum()
    dice_num = 2 * product + 1
    pred_sum = p.sum()
    label_sum = l.sum()
    dice_den = pred_sum + label_sum + 1
    dice_val = 100*dice_num / dice_den
    return round(dice_val,2)
def compute_iou(label_img, pred_img, p_threshold=0.5):
    p = pred_img.astype(np.float)
    l = label_img.astype(np.float)
    if p.max() >= 127:
        p /= 255.
    if l.max() >= 127:
        l /= 255.

    p = np.clip(p, 0, 1.0)
    l = np.clip(l, 0, 1.0)
    p[p >= 0.5] = 1.0
    p[p < 0.5] = 0.0
    l[l >= 0.5] = 1.0
    l[l < 0.5] = 0.0

    product = ((l.flatten())* (p.flatten())).sum()
    iou_num = product + 1
    pred_sum = p.sum()
    label_sum = l.sum()
    iou_den = pred_sum + label_sum + 1 - product
    iou = 100*iou_num / iou_den

    return round(iou,2)
def compute_overlap(label_img, pred_img):
    p = pred_img.astype(np.float)
    l = label_img.astype(np.float)
    if p.max() >= 127:
        p /= 255.
    if l.max() >= 127:
        l /= 255.

    p = np.clip(p, 0, 1.0)
    l = np.clip(l, 0, 1.0)
    p[p >= 0.5] = 1.0
    p[p < 0.5] = 0.0
    l[l >= 0.5] = 1.0
    l[l < 0.5] = 0.0

    # product = ((l.flatten())* (p.flatten())).sum()
    # label_sum = l.sum()
    # overlap = 100*product /label_sum
    bitwise_and_roi_patch = np.logical_and(l, p).sum()
    overlap = 100*(bitwise_and_roi_patch + 1)/(np.sum(l)+1)

    return round(overlap,2)

In [ ]:
def compute_iou(label_img, pred_img):
    product = ((label_img.flatten())* (pred_img.flatten())).sum()
    iou_num = product + 1
    pred_sum = pred_img.sum()
    label_sum = label_img.sum()
    iou_den = pred_sum + label_sum + 1 - product
    iou = 100*iou_num / iou_den

    return round(iou,2)
def compute_dice_1(label_img, pred_img):
    product = ((label_img.flatten())* (pred_img.flatten())).sum()
    dice_num = 2 * product + 1
    pred_sum = pred_img.sum()
    label_sum = label_img.sum()
    dice_den = pred_sum + label_sum + 1
    dice_val = 100*dice_num / dice_den
    return round(dice_val,2)

def compute_overlap(label_img, pred_img):
    bitwise_and_roi_patch = np.logical_and(label_img, pred_img)
    overlap = 100*np.sum(bitwise_and_roi_patch)/np.sum(label_img)
    return round(overlap,2)

In [ ]:
def cal_dice_iou_overlap(roi, pred):

  _, roi_bina = cv.threshold(roi, 1, 255, cv.THRESH_BINARY)
  _, pred_bina = cv.threshold(pred, 1, 255, cv.THRESH_BINARY)

  contours_pred, _ = cv.findContours(pred_bina, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  contours_roi, _ = cv.findContours(roi_bina, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

  # pred = cv.fillPoly(pred, pts= contours_pred, color=(255, 255))
  # contours_pred, _ = cv.findContours(pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

  dice_list = []
  iou_list =[]
  overlap_list = []
  roi_num_cnt = len(contours_roi)
  num_y_pred = len(contours_pred)
  label_y = 999
  label_y_pred = 999
  print("num ground truth", roi_num_cnt)
  print("num Predict:", num_y_pred)

  if (roi_num_cnt == 0) & (num_y_pred == 0):
     label_y = 0
     label_y_pred = 0
  elif (roi_num_cnt == 0) & (num_y_pred != 0):
    label_y = 0
    label_y_pred = 1
  elif (roi_num_cnt != 0) & (num_y_pred == 0):
    label_y = 1
    label_y_pred = 0
  else:
       for ct in contours_roi:
              each_roi= np.zeros(roi.shape, dtype="uint8")
              each_roi = cv.fillPoly(each_roi, pts=[ct], color=(255, 255))
              for ct_pred in contours_pred:
                  each_pred = np.zeros(pred.shape, dtype="uint8")
                  each_pred = cv.fillPoly(each_pred, pts=[ct_pred], color=(255, 255))

                  dice = compute_dice_1(each_roi, each_pred)
                  iou = compute_iou(each_roi, each_pred)
                  overlap = compute_overlap(each_roi, each_pred)
                  dice_list.append(dice)
                  iou_list.append(iou)
                  overlap_list.append(overlap)
                  if dice >= 50:
                    label_y = 1
                    label_y_pred = 1


  return dice_list, iou_list, overlap_list, label_y, label_y_pred, roi_num_cnt, num_y_pred

In [ ]:
from csv import DictReader
csv_path = '/content/drive/MyDrive/Mammo dataset/Test DDSM/patch_mask__pred_Test_set_bkg.csv'

predict_path = "/content/predict test 133 images patch"
predict_2nd_path = "/content/predict test 133 images patch without 2 nd"

from pathlib import Path


dice_list = []
iou_list = []
overlap_list = []
label_y_list = []
label_y_pred_list = []
num_roi_list = []
num_pred_list = []

dice_2nd_list = []
iou_2nd_list = []
overlap_2nd_list = []
label_y_2nd_list = []
label_y_pred_2nd_list = []
num_roi_2nd_list = []
num_pred_2nd_list = []

with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)
    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']
        # pred_file = row['y_pred']
        # pred_2nd_file = row['y_pred_2nd']

        name_file_saver = os.path.basename(img_file)

        pred_file= os.path.join(predict_path,''.join([name_file_saver]))
        pred_2nd_file= os.path.join(predict_2nd_path,''.join([name_file_saver]))

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))

        roi = cv.imread(roi_file, 0)
        roi = cv.morphologyEx(roi, cv.MORPH_CLOSE, kernel)
        roi = cv.morphologyEx(roi, cv.MORPH_OPEN, kernel)
        # roi, n_1 = filter_FP(roi, 300)


        y_pred = cv.imread(pred_file, 0)
        y_pred = cv.morphologyEx(y_pred, cv.MORPH_CLOSE, kernel)
        y_pred = cv.morphologyEx(y_pred, cv.MORPH_OPEN, kernel)
        # y_pred, n_2 = filter_FP(pred, 300)

        y_pred_2nd = cv.imread(pred_2nd_file, 0)
        y_pred_2nd = cv.morphologyEx(y_pred_2nd, cv.MORPH_CLOSE, kernel)
        y_pred_2nd = cv.morphologyEx(y_pred_2nd, cv.MORPH_OPEN, kernel)
        # y_pred_2nd, n_3 = filter_FP(pred, 300)

        dice, iou, overlap, label_y, label_y_pred, roi_num_cnt, num_y_pred = cal_dice_iou_overlap(roi, y_pred)
        dice_2nd, iou_2nd, overlap_2nd, label_y_2nd, label_y_pred_2nd, roi_num_cnt_2nd, num_y_pred_2nd = cal_dice_iou_overlap(roi, y_pred_2nd)

        print(dice, iou, overlap)

        dice_list.append(dice)
        iou_list.append(iou)
        overlap_list.append(overlap)
        label_y_list.append(label_y)
        label_y_pred_list.append(label_y_pred)
        num_roi_list.append(roi_num_cnt)
        num_pred_list.append(num_y_pred)

        print(dice_2nd, iou_2nd, overlap_2nd)
        dice_2nd_list.append(dice_2nd)
        iou_2nd_list.append(iou_2nd)
        overlap_2nd_list.append(overlap_2nd)
        label_y_2nd_list.append(label_y_2nd)
        label_y_pred_2nd_list.append(label_y_pred_2nd)
        num_roi_2nd_list.append(roi_num_cnt_2nd)
        num_pred_2nd_list.append(num_y_pred_2nd)


In [ ]:
data = pd.read_csv(csv_path)
test_x = data["img"]
test_y = data["roi"]

In [ ]:
import pandas as pd
df = pd.DataFrame({'x':test_x, 'y': test_y,'dice': dice_list, 'iou': iou_list, 'overlap': overlap_list, 'label_y':label_y_list, 'label_y_pred':label_y_pred_list, 'roi_num_cnt':num_roi_list, 'num_y_pred': num_pred_list,
                   'dice_2nd': dice_2nd_list, 'iou_2nd': iou_2nd_list, 'overlap_2nd': overlap_2nd_list, 'label_y_2nd':label_y_2nd_list, 'label_y_pred_2nd':label_y_pred_2nd_list, 'roi_num_cnt_2nd':num_roi_2nd_list, 'num_y_pred_2nd': num_pred_2nd_list,})
df.to_csv ('/content/drive/MyDrive/Mammo dataset/Test DDSM/dice_iou_overlap_Test_set_patch_8.csv', index = False, header=True)

#### cal for 2nd

In [ ]:
overlaps = []
labels_y = []
labels_y_pred_2nd = []
for i in range(len(test_set_y)):
  x = test_set_x[i]
  y = test_set_y[i]
  y_pred_2nd = test_set_y_pred_2nd[i]
  overlap, label_y, label_y_pred_2nd = cal_overlap_label(x,y,y_pred_2nd)
  overlaps.append(overlap)
  labels_y.append(label_y)
  labels_y_pred_2nd.append(label_y_pred_2nd)

In [ ]:
import pandas as pd
df = pd.DataFrame({'overlap': overlaps, 'label_y': labels_y, 'label_y_pred': labels_y_pred_2nd })
df.to_csv ('/content/drive/MyDrive/Mammo dataset/Test DDSM/overlap_Test_set_bkg_label_2nd_1.csv', index = False, header=True)

In [ ]:
import seaborn as sn
from sklearn.metrics import confusion_matrix

# y_pred_2 = test_set_y_pred.ravel()
# test_set_y_2 = test_set_y.ravel()
cm=confusion_matrix(labels_y, labels_y_pred_2nd)
print(cm)
df_cm = pd.DataFrame(cm, index = (0, 1), columns = (0, 1))
plt.figure(figsize = (10,7))
plt.title('confusion matrix - Test data')
sn.set(font_scale=1.4)
sn.heatmap(df_cm, annot=True, fmt='g')

In [ ]:
total1=sum(sum(cm))
#####from confusion matrix calculate accuracy
accuracy1=(cm[0,0]+cm[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cm[0,0]/(cm[0,0]+cm[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cm[1,1]/(cm[1,0]+cm[1,1])
print('Specificity : ', specificity1)

#### Test

In [ ]:
!pip install patchify

In [ ]:
!unrar x "//content/drive/MyDrive/Mammo dataset/Test DDSM/test_set_ddsm_remove_bkg.rar"

In [ ]:
drive_path = '/content/drive/MyDrive'
dataset_path = '/content'

files_dir = os.path.join(drive_path, 'Colab Notebooks', 'files_15_5_23', 'training_dataset_20_bkg')
model_file = os.path.join(files_dir,'unet_effb0-new-data-20_bkg-15_5_2023.h5')

test_x= sorted(glob(os.path.join(dataset_path, 'test_set_ddsm_remove_bkg', 'image', '*')))
test_y= sorted(glob(os.path.join(dataset_path, 'test_set_ddsm_remove_bkg', 'mask', '*')))

In [ ]:
print(len(test_x))

In [ ]:
import pandas as pd
df = pd.DataFrame({'img': test_x, 'roi': test_y})
df.to_csv ('/content/drive/MyDrive/Mammo dataset/Test DDSM/img_mask_test_set_remove_bkg.csv', index = False, header=True)

In [ ]:
load_model_file = tf.keras.models.load_model(model_file)

In [ ]:
def show_y_predict(randomlist, y, y_pred):
  fg, ax = plt.subplots(2, 5, figsize=(20, 8))
  for i, index in enumerate(randomlist):
    ax[0, i].imshow(np.squeeze(y[index]), 'gray')
    ax[0, i].set_xlabel('y ' + str(index))
    # print(y[index].shape)

  for i, index in enumerate(randomlist):
    ax[1, i].imshow(np.squeeze(y_pred[index]), 'gray')
    ax[1, i].set_xlabel('y_pred ' + str(index))

def prediction(model, large_image, patch_size, step):
  patches = patchify(large_image, (patch_size, patch_size), step=step)  #Step=256 for 256 patches means no overlap

  print(patches.shape)

  redicted_patches = []
  patches_input = []
  for i in range(patches.shape[0]):
    for j in range(patches.shape[1]):
        # print(i,j)
        single_patch = patches[i,j,:,:]
        single_patch_input=np.expand_dims(single_patch, -1)
        # single_patch = np.array(single_patch)
        # print(single_patch.shape)
        #Predict and threshold for values above 0.5 probability

        single_patch_input = np.zeros((patch_size,patch_size,3), dtype="float32")
        single_patch_input[:,:,0] = single_patch
        single_patch_input[:,:,1] = single_patch
        single_patch_input[:,:,2] = single_patch

        single_patch_input = cv.resize(single_patch_input, (256,256), cv.INTER_AREA)
        single_patch_input = np.array(single_patch_input)
        patches_input.append(single_patch_input)

  patches_input = np.array(patches_input)

  patches_prediction = (model.predict(patches_input, batch_size = 8) >= 0.5).astype(np.uint8)
  # randomlist = random.sample(range(0, len(patches_prediction)),5)
  # show_y_predict(randomlist, patches_input, patches_prediction)
  # plt.show()

  patches_prediction_resize = []
  for i in patches_prediction:
    i = cv.resize(i, (patch_size,patch_size), cv.INTER_AREA)
    patches_prediction_resize.append(i)

  predicted_patches_reshaped = np.reshape(patches_prediction_resize, (patches.shape[0], patches.shape[1], patch_size,patch_size) )
  print(predicted_patches_reshaped.shape)

  reconstructed_image = unpatchify(predicted_patches_reshaped, large_image.shape)
  plt.imshow(np.squeeze(reconstructed_image), cmap='gray')
  return reconstructed_image

def filter_FP(y_pred, thresh):
  y_pred_filter = y_pred.copy()
  kernel = np.ones((3,3),np.uint8)
  y_pred_filter = cv.erode(y_pred_filter, kernel)

  contours, _ = cv.findContours(y_pred_filter.copy(), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
  n = 0
  y_pred_filter_new = np.zeros(y_pred.shape, dtype="uint8")
  for ct in contours:
      contour_area = cv.contourArea(ct)
      if (contour_area>= thresh):
          n += 1
          y_pred_filter_new = cv.fillPoly(y_pred_filter_new, pts= [ct], color=(255, 255))
      # else:
      #   y_pred_filter = cv.fillPoly(y_pred_filter, pts=[ct], color=(0, 0))
  y_pred_filter_new =  cv.dilate(y_pred_filter_new, kernel)
  print("n = ", n)
  return y_pred_filter_new, n


def draw_contour_pred_and_truth(large_image, pred, roi):
  img_copy = large_image.copy()

  contours_pred, _ = cv.findContours(pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  contours_roi, _ = cv.findContours(roi.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  cv.drawContours(img_copy, contours_roi , -1, (255,0,0), 3)
  cv.drawContours(img_copy, contours_pred, -1, (0,255,0), 3)

  return img_copy

def check_Left_Orient(img, roi):
    left_nonzero = cv.countNonZero(img[:, 0:int(img.shape[1] / 2)])
    right_nonzero = cv.countNonZero(img[:, int(img.shape[1] / 2):])

    if (left_nonzero <= right_nonzero):
            img = cv.flip(img, 1)
            roi = cv.flip(roi, 1)

    return img, roi

def cal_overlap_per_large_img(roi, overlap_img):
  bitwise_and_roi_pred = cv.bitwise_and(overlap_img, roi)
  area_truth = np.count_nonzero(bitwise_and_roi_pred)
  overlap = round(area_truth / np.count_nonzero(roi), 4)*100
  return bitwise_and_roi_pred, area_truth, overlap



In [ ]:
def cal_overlaps_per_large_img(roi, pred, large_image):

  pred, n = filter_FP(pred, 2000)

  contours_pred, _ = cv.findContours(pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  contours_roi, _ = cv.findContours(roi.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

  img = large_image.copy()

  overlap_list = []

  roi_num_cnt = len(contours_roi)
  print("num Predict:", len(contours_pred))

  bitwise_and_roi_pred = cv.bitwise_and(pred, roi)

  if  roi_num_cnt == 1:
       area_truth = np.count_nonzero(bitwise_and_roi_pred)
       overlap = round(area_truth/np.count_nonzero(roi) * 100,3)
       overlap_list.append(overlap)
       print("overlap", overlap)
       cv.drawContours(img, contours_roi , -1, (255,0,0), 3)
       cv.drawContours(img, contours_pred, -1, (0,255,0), 3)

       if overlap!=0:

           contours_overlaps, _ = cv.findContours(bitwise_and_roi_pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

           c = max(contours_overlaps, key = cv.contourArea)

           x, y, w, h = cv.boundingRect(c)

           font = cv.FONT_HERSHEY_SIMPLEX
           org = (x-10, y-10)
           fontScale = 3
           color = (0, 0, 255)
           thickness = 2
           text = str(overlap)
           img = cv.putText(img, text, org, font, fontScale, color, thickness, cv.LINE_AA)

  elif roi_num_cnt == 0:
       cv.drawContours(img, contours_pred, -1, (0,255,0), 3)
  else:
       bitwise_and_roi_pred = cv.bitwise_and(pred, roi)
       area_truth = np.count_nonzero(bitwise_and_roi_pred)
       overlap = round(area_truth/np.count_nonzero(roi) * 100,3)
       if overlap == 0:
          cv.drawContours(img, contours_roi , -1, (255,0,0), 3)
          cv.drawContours(img, contours_pred, -1, (0,255,0), 3)
       else :
          cv.drawContours(img, contours_roi , -1, (255,0,0), 3)
          cv.drawContours(img, contours_pred, -1, (0,255,0), 3)
          for ct in contours_roi:
            each_roi= np.zeros(roi.shape, dtype="uint8")
            each_roi = cv.fillPoly(each_roi, pts=[ct], color=(255, 255, 255))
            # plt.imshow(each_roi, 'gray')
            # plt.show()
            # cv.drawContours(img, ct , -1, (255,0,0), 3)

            for ct_pred in contours_pred:
                  each_pred = np.zeros(pred.shape, dtype="uint8")
                  each_pred = cv.fillPoly(each_pred, pts=[ct_pred], color=(255, 255, 255))

                  # plt.imshow(each_pred, 'gray')
                  # plt.show()

                  bitwise_and_roi_pred_1 = cv.bitwise_and(each_pred, each_roi)

                  area_truth = np.count_nonzero(bitwise_and_roi_pred_1)
                  overlap = round(area_truth/np.count_nonzero(each_roi) *100,3)
                  # cv.drawContours(img, ct_pred, -1, (0,255,0), 3)

                  # cv.drawContours(img, ct_pred , -1, (255,0,0), 3)

                  if overlap>10:
                       overlap_list.append(overlap)
                       print("overlap", overlap)

                       x, y, w, h = cv.boundingRect(ct_pred)

                      #  overlap_img_anotation = cv.fillPoly(overlap_img_anotation, pts=[ct_pred], color=(0, 0))

                      #  overlap_img_anotation = ImageDraw.Draw(overlap_img_anotation)
                      #  overlap_img_anotation.text((x, y), overlap, fill=(0 , 0, 255))
                       font = cv.FONT_HERSHEY_SIMPLEX
                       org = (x-10, y-20)
                       fontScale = 3
                       color = (0, 0, 255)
                       thickness = 2
                       text = str(overlap)
                       img = cv.putText(img, text, org, font, fontScale, color, thickness, cv.LINE_AA)
  # cv.imshow(overlap_img_anotation, 0)
  plt.imshow(bitwise_and_roi_pred, 'gray')
  plt.show()
  # plt.imshow(img)
  # plt.show()
  return bitwise_and_roi_pred, overlap_list, img

In [ ]:
image_path = "/content/test_set_ddsm_remove_bkg/image/Mass-Test_P_00116_RIGHT_CC.png"
roi_path = "/content/test_set_ddsm_remove_bkg/mask/Mass-Test_P_00116_RIGHT_CC.png"
predict_path ="/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/overlap/Mass-Test_P_00116_RIGHT_CC.png"


predict = cv.imread(predict_path, 0).astype(np.uint8)
predict = np.array(predict)

image = cv.imread(image_path)


roi = cv.imread(roi_path, 0).astype(np.uint8)
roi = np.array(roi)

h_r, w_r = roi.shape



h_i, w_i = predict.shape

if (h_i>=h_r) or (w_i>=w_r):
      predict = predict[0:h_r, 0:w_r]
else:
      roi = roi[0:h_i, 0:w_i]

bitwise_and_roi_pred, overlap_list, img_anotation = cal_overlaps_per_large_img(roi, predict, image)
plt.imshow(bitwise_and_roi_pred)
plt.show()

plt.imshow(img_anotation)
plt.show()

In [ ]:
def detect_roi_by_patch_size(img, load_model_file, patch_size, step):

        h,w = img.shape

        roi_detect= np.zeros(img.shape, dtype="uint8")

        row = int(( h - patch_size)/step)
        col = int(( w - patch_size)/step)
        new_img_h = row * step + patch_size
        new_img_w = col * step + patch_size

        img_by_patch_size = img[0:new_img_h, 0: new_img_w]

        segmented_image_by_patch_size = prediction(load_model_file, img_by_patch_size, patch_size , step)
        h_1, w_1 = segmented_image_by_patch_size.shape
        roi_detect[0:h_1, 0:w_1] = segmented_image_by_patch_size


        return roi_detect

In [ ]:
csv_path = '/content/drive/MyDrive/Mammo dataset/Test DDSM/normal_image.csv'
pred_path = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Normal/original_predict'
overlap_path = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Normal/overlap'
draw_contour_path = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Normal/draw contour'
overlap_anotation_path = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Normal/overlap anotation'

import time

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path


overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []
small_image = []

with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    start_time = time.time()

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']
        name_file_saver = os.path.basename(img_file)

        large_image_1 = cv.imread(img_file, 0)
        image_anotation = cv.imread(img_file)
        large_image= np.float32(large_image_1)
        large_image = large_image/255.0

        roi = cv.imread(roi_file, 0).astype(np.uint8)

        # roi = cv.imread(roi_file, 0)
        # roi= np.uint8(roi)
        # print(roi.shape)
        h_r, w_r = roi.shape

        large_image, roi= check_Left_Orient (large_image, roi)

        img = np.array(large_image)

        h_i, w_i = img.shape

        if (h_i>=h_r) or (w_i>=w_r):
          img = img[0:h_r, 0:w_r]
          image_anotation = image_anotation[0:h_r, 0:w_r]
        else:
          roi = roi[0:h_i, 0:w_i]

        # print(img.shape)

        roi_predict_384 = detect_roi_by_patch_size(img, load_model_file, 348, 64)

        if 600 < w_i < 1024:
          roi_predict_1024 = np.zeros(img.shape, dtype="uint8")
          roi_predict_512 = detect_roi_by_patch_size(img, load_model_file, 512, 64)

        elif w_i <= 600:
          print("image < 512", name_file_saver)
          small_image.append(name_file_saver)
          continue
        else:
          roi_predict_1024 = detect_roi_by_patch_size(img, load_model_file, 1024, 128)
          roi_predict_512 = detect_roi_by_patch_size(img, load_model_file, 512, 64)

        # print(roi_predict_384.shape, roi_predict_512.shape, roi_predict_1024.shape)


        # combine = np.zeros(roi.shape, dtype=np.uint8)

        combine_1 = np.logical_or(roi_predict_384, roi_predict_512).astype(np.uint8)
        combine = np.logical_or(combine_1, roi_predict_1024).astype(np.uint8)


        y_pred_filter, n = filter_FP(combine, 2000)

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_CLOSE, kernel)
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)

        num_P_list.append(n)

        pred_file_path = os.path.join(pred_path,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path, y_pred_filter, cmap='gray')


        overlap_img, overlap_list, img_anotation_1 = cal_overlaps_per_large_img(roi, y_pred_filter, image_anotation)

        # overlap_img_anotation = 1*overlap_img_anotation
        img_overlap_anotation_path = os.path.join(overlap_anotation_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_overlap_anotation_path, img_anotation_1, cmap='gray')
        plt.title(name_file_saver)
        plt.imshow(img_anotation_1)
        plt.show()
        # print(overlap)
        area_truth_list.append(overlap_list)

        # overlap_img = 1*overlap_img
        file_overlap_img_path = os.path.join(overlap_path,
                                 ''.join([name_file_saver]))
        plt.imsave(file_overlap_img_path, overlap_img, cmap='gray')

        img_draw = cv.imread(img_file)
        # combine_predict = cv.imread(pred_file_path, 0)
        img_draw_contour = draw_contour_pred_and_truth(img_draw, y_pred_filter, roi)

        # plt.imshow(img_draw_contour)

        # plt.title(name_file_saver + "- overlap = "+ str(overlap))
        # plt.show()
        img_draw_contour_path = os.path.join(draw_contour_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_draw_contour_path, img_draw_contour, cmap='gray')

print("Time taken: %.2fs" % (time.time() - start_time))


#### Detect patches and calculate metrics

In [ ]:
def prediction(model, large_image, groundtruth_uint8, patch_size, step):
  patches = patchify(large_image, (patch_size, patch_size), step=step)  #Step=256 for 256 patches means no overlap
  gt_patches = patchify(groundtruth_uint8, (patch_size, patch_size), step=step)

  print(patches.shape)
  # print(gt_patches.shape)

  redicted_patches = []
  patches_input = []
  gt_patches_input = []
  for i in range(patches.shape[0]):
    for j in range(patches.shape[1]):
        # print(i,j)
        single_patch = patches[i,j,:,:]
        single_patch_input=np.expand_dims(single_patch, -1)
        # single_patch = np.array(single_patch)
        # print(single_patch.shape)
        #Predict and threshold for values above 0.5 probability

        single_patch_input = np.zeros((patch_size,patch_size,3), dtype="float32")
        single_patch_input[:,:,0] = single_patch
        single_patch_input[:,:,1] = single_patch
        single_patch_input[:,:,2] = single_patch

        single_patch_input = cv.resize(single_patch_input, (256,256), cv.INTER_AREA)
        single_patch_input = np.array(single_patch_input)
        patches_input.append(single_patch_input)

  for i in range(gt_patches.shape[0]):
      for j in range(gt_patches.shape[1]):
        # print(i,j)
        single_patch = gt_patches[i,j,:,:]
        single_patch_input=np.expand_dims(single_patch, -1)

        single_patch_input = np.zeros((patch_size,patch_size), dtype="uint8")
        single_patch_input[:,:] = single_patch
        single_patch_input = np.array(single_patch_input)
        gt_patches_input.append(single_patch_input)

  patches_input = np.array(patches_input)
  gt_patches_input = np.array(gt_patches_input)
  print("patches size", gt_patches_input.shape)
  print("GT patches size", patches_input.shape)

  patches_prediction = (model.predict(patches_input) >= 0.5).astype(np.uint8)

  # randomlist = random.sample(range(0, len(patches_prediction)),5)
  # show_y_predict(randomlist, patches_input, patches_prediction)
  # plt.show()

  patches_prediction_resize = []
  for i in patches_prediction:
    i = cv.resize(i, (patch_size,patch_size), cv.INTER_AREA)
    patches_prediction_resize.append(i)
  # print("patches size", patches_prediction_resize.shape)

  intersection = np.logical_and(gt_patches_input, patches_prediction_resize)
  union = np.logical_or(gt_patches_input, patches_prediction_resize)
  iou_score = np.sum(intersection) / np.sum(union)
  print("IoU socre is: ", iou_score)

  predicted_patches_reshaped = np.reshape(patches_prediction_resize, (patches.shape[0], patches.shape[1], patch_size,patch_size) )
  print(predicted_patches_reshaped.shape)

  reconstructed_image = unpatchify(predicted_patches_reshaped, large_image.shape)
  plt.imshow(np.squeeze(reconstructed_image), cmap='gray')
  return reconstructed_image

In [ ]:
from patchify import patchify,  unpatchify
image_path = "/content/test_set_ddsm_remove_bkg/image/Mass-Test_P_00116_RIGHT_CC.png"
roi_path = "/content/test_set_ddsm_remove_bkg/mask/Mass-Test_P_00116_RIGHT_CC.png"

patch_size = 512
step = 64

img = cv.imread(image_path, 0)
# image= np.float32(image)
# img = image/255.0


roi = cv.imread(roi_path, 0).astype(np.uint8)
roi = np.array(roi)


plt.imshow(img)
plt.show()
plt.imshow(roi)
plt.show()

# h_r, w_r = roi.shape
# h_i, w_i = image.shape

# if (h_i>=h_r) or (w_i>=w_r):
#       img = img[0:h_r, 0:w_r]
# else:
#       roi = roi[0:h_i, 0:w_i]

# h,w = img.shape
# row = int(( h - patch_size)/step)
# col = int(( w - patch_size)/step)
# new_img_h = row * step + patch_size
# new_img_w = col * step + patch_size

# img_by_patch_size = img[0:new_img_h, 0: new_img_w]
# roi_by_patch_size = roi[0:new_img_h, 0: new_img_w]

# img_predict = prediction(load_model_file, img_by_patch_size, roi_by_patch_size, patch_size, step)

# plt.imshow(img_predict)
# plt.show()


#### normal image

In [ ]:
!unrar x "/content/drive/MyDrive/Mammo dataset/Test DDSM/Normal.rar"

In [ ]:
drive_path = '/content/drive/MyDrive'
dataset_path = '/content'

files_dir = os.path.join(drive_path, 'Colab Notebooks', 'files_15_5_23', 'training_dataset_20_bkg')
model_file = os.path.join(files_dir,'unet_effb0-new-data-20_bkg-15_5_2023.h5')

test_x= sorted(glob(os.path.join(dataset_path, 'Normal', 'image', '*')))
test_y= sorted(glob(os.path.join(dataset_path, 'Normal', 'mask', '*')))

In [ ]:
import pandas as pd
df = pd.DataFrame({'img': test_x, 'roi': test_y})
df.to_csv ('/content/drive/MyDrive/Mammo dataset/Test DDSM/normal_image.csv', index = False, header=True)

In [ ]:
csv_path = '/content/drive/MyDrive/Mammo dataset/Test DDSM/normal_image.csv'
pred_path = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Normal/original_predict'
overlap_path = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Normal/overlap'
draw_contour_path = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Normal/draw contour'
overlap_anotation_path = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Normal/overlap anotation'

import time

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path


overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []
small_image = []

with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    start_time = time.time()

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']
        name_file_saver = os.path.basename(img_file)

        large_image_1 = cv.imread(img_file, 0)
        image_anotation = cv.imread(img_file)
        large_image= np.float32(large_image_1)
        large_image = large_image/255.0

        roi = cv.imread(roi_file, 0).astype(np.uint8)

        # roi = cv.imread(roi_file, 0)
        # roi= np.uint8(roi)
        # print(roi.shape)
        h_r, w_r = roi.shape

        large_image, roi= check_Left_Orient (large_image, roi)

        img = np.array(large_image)

        h_i, w_i = img.shape

        if (h_i>=h_r) or (w_i>=w_r):
          img = img[0:h_r, 0:w_r]
          image_anotation = image_anotation[0:h_r, 0:w_r]
        else:
          roi = roi[0:h_i, 0:w_i]

        # print(img.shape)

        roi_predict_384 = detect_roi_by_patch_size(img, load_model_file, 348, 64)

        if 600 < w_i < 1024:
          roi_predict_1024 = np.zeros(img.shape, dtype="uint8")
          roi_predict_512 = detect_roi_by_patch_size(img, load_model_file, 512, 64)

        elif w_i <= 600:
          print("image < 512", name_file_saver)
          small_image.append(name_file_saver)
          continue
        else:
          roi_predict_1024 = detect_roi_by_patch_size(img, load_model_file, 1024, 128)
          roi_predict_512 = detect_roi_by_patch_size(img, load_model_file, 512, 64)

        # print(roi_predict_384.shape, roi_predict_512.shape, roi_predict_1024.shape)


        # combine = np.zeros(roi.shape, dtype=np.uint8)

        combine_1 = np.logical_or(roi_predict_384, roi_predict_512).astype(np.uint8)
        combine = np.logical_or(combine_1, roi_predict_1024).astype(np.uint8)


        y_pred_filter, n = filter_FP(combine, 2000)

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_CLOSE, kernel)
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)

        num_P_list.append(n)

        pred_file_path = os.path.join(pred_path,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path, y_pred_filter, cmap='gray')

        overlap_img, overlap_list, img_anotation_1 = cal_overlaps_per_large_img(roi, y_pred_filter, image_anotation)

        # overlap_img_anotation = 1*overlap_img_anotation
        img_overlap_anotation_path = os.path.join(overlap_anotation_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_overlap_anotation_path, img_anotation_1, cmap='gray')
        plt.title(name_file_saver)
        plt.imshow(img_anotation_1)
        plt.show()
        # print(overlap)
        area_truth_list.append(overlap_list)

        # overlap_img = 1*overlap_img
        file_overlap_img_path = os.path.join(overlap_path,
                                 ''.join([name_file_saver]))
        plt.imsave(file_overlap_img_path, overlap_img, cmap='gray')

        img_draw = cv.imread(img_file)
        # combine_predict = cv.imread(pred_file_path, 0)
        img_draw_contour = draw_contour_pred_and_truth(img_draw, y_pred_filter, roi)

        # plt.imshow(img_draw_contour)

        # plt.title(name_file_saver + "- overlap = "+ str(overlap))
        # plt.show()
        img_draw_contour_path = os.path.join(draw_contour_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_draw_contour_path, img_draw_contour, cmap='gray')

print("Time taken: %.2fs" % (time.time() - start_time))


In [ ]:
print(small_image)

In [ ]:
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)
    imgs = []
    rois = []
    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']
        name_file_saver = os.path.basename(img_file)
        if name_file_saver!= "A_0002_1.LEFT_CC.png":
          imgs.append(img_file)
          rois.append(roi_file)

In [ ]:
print(len(imgs))

25G; 15G ram, v100

In [ ]:
# data = pd.read_csv(csv_path)
# img = data["img"]
# roi = data["roi"]
import pandas as pd
save_csv_path= '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Normal/predict_normal.csv'
df = pd.DataFrame({'img': imgs, 'roi': rois, 'overlap': area_truth_list, 'num_FP': num_P_list})
df.to_csv (save_csv_path, index = False, header=True)

#### normal image 144

In [ ]:
!unrar x "/content/drive/MyDrive/Mammo dataset/Test DDSM/new normal test ddsm.rar"

In [ ]:
drive_path = '/content/drive/MyDrive'
dataset_path = '/content'

files_dir = os.path.join(drive_path, 'Colab Notebooks', 'files_15_5_23', 'training_dataset_20_bkg')
model_file = os.path.join(files_dir,'unet_effb0-new-data-20_bkg-15_5_2023.h5')

test_x= sorted(glob(os.path.join(dataset_path, 'new normal test ddsm', 'image', '*')))
test_y= sorted(glob(os.path.join(dataset_path, 'new normal test ddsm', 'mask', '*')))

In [ ]:
load_model_file = tf.keras.models.load_model(model_file)

In [ ]:
import pandas as pd
df = pd.DataFrame({'img': test_x, 'roi': test_y})
df.to_csv ('/content/drive/MyDrive/Mammo dataset/Test DDSM/normal_144_image.csv', index = False, header=True)

In [ ]:
csv_path = '/content/drive/MyDrive/Mammo dataset/Test DDSM/normal_144_image.csv'
pred_path = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/144 normal/original_predict'

draw_contour_path = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/144 normal/draw contour'

import time

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path


overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []
small_image = []

with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    start_time = time.time()

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']
        name_file_saver = os.path.basename(img_file)

        large_image_1 = cv.imread(img_file, 0)
        image_anotation = cv.imread(img_file)
        large_image= np.float32(large_image_1)
        large_image = large_image/255.0

        roi = cv.imread(roi_file, 0).astype(np.uint8)

        # roi = cv.imread(roi_file, 0)
        # roi= np.uint8(roi)
        # print(roi.shape)
        h_r, w_r = roi.shape

        large_image, roi= check_Left_Orient (large_image, roi)

        img = np.array(large_image)

        h_i, w_i = img.shape

        if (h_i>=h_r) or (w_i>=w_r):
          img = img[0:h_r, 0:w_r]
          image_anotation = image_anotation[0:h_r, 0:w_r]
        else:
          roi = roi[0:h_i, 0:w_i]

        # print(img.shape)

        roi_predict_384 = detect_roi_by_patch_size(img, load_model_file, 348, 64)

        if 600 < w_i < 1024:
          roi_predict_1024 = np.zeros(img.shape, dtype="uint8")
          roi_predict_512 = detect_roi_by_patch_size(img, load_model_file, 512, 64)

        elif w_i <= 600:
          print("image < 512", name_file_saver)
          small_image.append(name_file_saver)
          continue
        else:
          roi_predict_1024 = detect_roi_by_patch_size(img, load_model_file, 1024, 128)
          roi_predict_512 = detect_roi_by_patch_size(img, load_model_file, 512, 64)

        # print(roi_predict_384.shape, roi_predict_512.shape, roi_predict_1024.shape)


        # combine = np.zeros(roi.shape, dtype=np.uint8)

        combine_1 = np.logical_or(roi_predict_384, roi_predict_512).astype(np.uint8)
        combine = np.logical_or(combine_1, roi_predict_1024).astype(np.uint8)


        y_pred_filter, n = filter_FP(combine, 2000)

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_CLOSE, kernel)
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)

        num_P_list.append(n)

        pred_file_path = os.path.join(pred_path,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path, y_pred_filter, cmap='gray')


        img_draw = cv.imread(img_file)
        img_draw_contour = draw_contour_pred_and_truth(img_draw, y_pred_filter, roi)

        img_draw_contour_path = os.path.join(draw_contour_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_draw_contour_path, img_draw_contour, cmap='gray')

print("Time taken: %.2fs" % (time.time() - start_time))


In [ ]:
pred_path = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model'

pred = sorted(glob(os.path.join(pred_path, '144 normal', 'original_predict', '*')))
draw_image = sorted(glob(os.path.join(pred_path, '144 normal', 'draw contour', '*')))

In [ ]:
# data = pd.read_csv(csv_path)
# img = data["img"]
# roi = data["roi"]
import pandas as pd
save_csv_path= '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/144 normal/144_predict_normal.csv'
df = pd.DataFrame({'pred': pred, 'draw contour': draw_image, 'num_Predict': num_P_list})
df.to_csv (save_csv_path, index = False, header=True)

###### 2nd model

In [ ]:
drive_path = '/content/drive/MyDrive'
dataset_path = '/content'

files_dir = os.path.join(drive_path, 'Colab Notebooks', 'files_15_5_23', 'training_dataset_20_bkg_unet_without_skip')
model_file = os.path.join(files_dir,'unet_effb0-new-data-20_bkg-without-skip.h5')

test_x= sorted(glob(os.path.join(dataset_path, 'new normal test ddsm', 'image', '*')))
test_y= sorted(glob(os.path.join(dataset_path, 'new normal test ddsm', 'mask', '*')))


In [ ]:
load_model_file_2nd = tf.keras.models.load_model(model_file)

In [ ]:
csv_path = '/content/drive/MyDrive/Mammo dataset/Test DDSM/normal_144_image.csv'
pred_path = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/144 normal/original_predict_2nd'

draw_contour_path = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/144 normal/draw contour 2 nd'

import time

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path


overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []
small_image = []

with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    start_time = time.time()

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']
        name_file_saver = os.path.basename(img_file)

        large_image_1 = cv.imread(img_file, 0)
        image_anotation = cv.imread(img_file)
        large_image= np.float32(large_image_1)
        large_image = large_image/255.0

        roi = cv.imread(roi_file, 0).astype(np.uint8)

        # roi = cv.imread(roi_file, 0)
        # roi= np.uint8(roi)
        # print(roi.shape)
        h_r, w_r = roi.shape

        large_image, roi= check_Left_Orient (large_image, roi)

        img = np.array(large_image)

        h_i, w_i = img.shape

        if (h_i>=h_r) or (w_i>=w_r):
          img = img[0:h_r, 0:w_r]
          image_anotation = image_anotation[0:h_r, 0:w_r]
        else:
          roi = roi[0:h_i, 0:w_i]

        # print(img.shape)

        roi_predict_384 = detect_roi_by_patch_size(img, load_model_file_2nd, 348, 64)

        if 600 < w_i < 1024:
          roi_predict_1024 = np.zeros(img.shape, dtype="uint8")
          roi_predict_512 = detect_roi_by_patch_size(img, load_model_file_2nd, 512, 64)

        elif w_i <= 600:
          print("image < 512", name_file_saver)
          small_image.append(name_file_saver)
          continue
        else:
          roi_predict_1024 = detect_roi_by_patch_size(img, load_model_file_2nd, 1024, 128)
          roi_predict_512 = detect_roi_by_patch_size(img, load_model_file_2nd, 512, 64)

        # print(roi_predict_384.shape, roi_predict_512.shape, roi_predict_1024.shape)


        # combine = np.zeros(roi.shape, dtype=np.uint8)

        combine_1 = np.logical_or(roi_predict_384, roi_predict_512).astype(np.uint8)
        combine = np.logical_or(combine_1, roi_predict_1024).astype(np.uint8)


        y_pred_filter, n = filter_FP(combine, 2000)

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_CLOSE, kernel)
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)

        num_P_list.append(n)

        pred_file_path = os.path.join(pred_path,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path, y_pred_filter, cmap='gray')


        img_draw = cv.imread(img_file)
        img_draw_contour = draw_contour_pred_and_truth(img_draw, y_pred_filter, roi)
        plt.imshow(img_draw_contour)

        plt.title(name_file_saver + "num pred = "+ str(n))
        plt.show()

        img_draw_contour_path = os.path.join(draw_contour_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_draw_contour_path, img_draw_contour, cmap='gray')

print("Time taken: %.2fs" % (time.time() - start_time))


In [ ]:
print(small_image)

In [ ]:
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)
    imgs = []
    rois = []
    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']
        name_file_saver = os.path.basename(img_file)
        if name_file_saver!= "A_0002_1.LEFT_CC.png":
          imgs.append(img_file)
          rois.append(roi_file)

In [ ]:
print(len(imgs))

25G; 15G ram, v100

In [ ]:
# data = pd.read_csv(csv_path)
# img = data["img"]
# roi = data["roi"]
import pandas as pd
save_csv_path= '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Normal/predict_normal.csv'
df = pd.DataFrame({'img': imgs, 'roi': rois, 'overlap': area_truth_list, 'num_FP': num_P_list})
df.to_csv (save_csv_path, index = False, header=True)

#### Abnormal image

In [ ]:
!unrar x "/content/drive/MyDrive/Mammo dataset/Test DDSM/Abnormal.rar"

In [ ]:
drive_path = '/content/drive/MyDrive'
dataset_path = '/content'

files_dir = os.path.join(drive_path, 'Colab Notebooks', 'files_15_5_23', 'training_dataset_20_bkg')
model_file = os.path.join(files_dir,'unet_effb0-new-data-20_bkg-15_5_2023.h5')

test_x= sorted(glob(os.path.join(dataset_path, 'Abnormal', 'image', '*')))
test_y= sorted(glob(os.path.join(dataset_path, 'Abnormal', 'mask', '*')))

In [ ]:
import pandas as pd
df = pd.DataFrame({'img': test_x, 'roi': test_y})
df.to_csv ('/content/drive/MyDrive/Mammo dataset/Test DDSM/abnormal_image.csv', index = False, header=True)

In [ ]:
def count_num_roi(roi):
  contours_roi, _ = cv.findContours(roi, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  num_roi = len(contours_roi)
  print("num ground truth", num_roi)
  return num_roi

In [ ]:
csv_path = '/content/drive/MyDrive/Mammo dataset/Test DDSM/abnormal_image.csv'

num_roi_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    start_time = time.time()

    for row in csv_dict_reader:
        roi_file = row['roi']
        roi = cv.imread(roi_file, 0)
        roi, n = filter_FP (roi, 300)
        roi = np.array(roi)
        num_roi = count_num_roi(roi)
        num_roi_list.append(num_roi)


In [ ]:
save_csv_path= '/content/drive/MyDrive/Mammo dataset/Test DDSM/118_num_roi_abnormal.csv'
df = pd.DataFrame({'num_roi': num_roi_list})
df.to_csv (save_csv_path, index = False, header=True)

In [ ]:
def cal_overlaps_per_large_img_1(roi, pred, large_image):

  contours_pred, _ = cv.findContours(pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  contours_roi, _ = cv.findContours(roi.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

  # for ct_roi in contours_roi:
  #       area = cv.contourArea(ct_roi)
  #       if (area < 500)
  #            roi = cv.fillPoly(roi, pts= [ct_pred], color=(0, 0))

  pred = cv.fillPoly(pred, pts= contours_pred, color=(255, 255))
  contours_pred, _ = cv.findContours(pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

  # roi = cv.fillPoly(roi, pts= contours_roi, color=(255, 255))
  # contours_roi, _ = cv.findContours(roi.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)


  img = large_image.copy()

  overlap_list = []

  roi_num_cnt = len(contours_roi)
  print("num ground truth", roi_num_cnt)
  print("num Predict:", len(contours_pred))

  bitwise_and_roi_pred = cv.bitwise_and(pred, roi)



  if  roi_num_cnt == 1:
       print("one ground truth")
       area_truth = np.count_nonzero(bitwise_and_roi_pred)
       overlap = round(area_truth/np.count_nonzero(roi) * 100,3)
       overlap_list.append(overlap)
       print("overlap", overlap)
       cv.drawContours(img, contours_roi , -1, (255,0,0), 3)
       cv.drawContours(img, contours_pred, -1, (0,255,0), 3)

       if overlap!=0:

           contours_overlaps, _ = cv.findContours(bitwise_and_roi_pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

           c = max(contours_overlaps, key = cv.contourArea)

           x, y, w, h = cv.boundingRect(c)

           font = cv.FONT_HERSHEY_SIMPLEX
           org = (x-10, y-10)
           fontScale = 3
           color = (0, 0, 255)
           thickness = 2
           text = str(overlap)
           img = cv.putText(img, text, org, font, fontScale, color, thickness, cv.LINE_AA)

  elif roi_num_cnt == 0:
       print("normal")
       cv.drawContours(img, contours_pred, -1, (0,255,0), 3)
  else:
       area_truth = np.count_nonzero(bitwise_and_roi_pred)
       overlap = round(area_truth/np.count_nonzero(roi) * 100,3)
       if overlap == 0:
          cv.drawContours(img, contours_roi , -1, (255,0,0), 3)
          cv.drawContours(img, contours_pred, -1, (0,255,0), 3)
       else :
          a = 0
          cv.drawContours(img, contours_roi , -1, (255,0,0), 3)
          cv.drawContours(img, contours_pred, -1, (0,255,0), 3)
          for ct in contours_roi:
              a = a + 1
              each_roi= np.zeros(roi.shape, dtype="uint8")
              each_roi = cv.fillPoly(each_roi, pts=[ct], color=(255, 255))

              # plt.imshow(each_roi, 'gray')
              # plt.title(a)
              # plt.show()
              # cv.drawContours(img, ct , -1, (255,0,0), 3)
              # plt.imshow(img, 'gray')
              # plt.title(a)
              # plt.show()


              for ct_pred in contours_pred:
                  each_pred = np.zeros(pred.shape, dtype="uint8")
                  each_pred = cv.fillPoly(each_pred, pts=[ct_pred], color=(255, 255))

                  # plt.imshow(each_pred, 'gray')
                  # plt.show()

                  bitwise_and_roi_pred_1 = cv.bitwise_and(each_pred, each_roi)

                  area_truth = np.count_nonzero(bitwise_and_roi_pred_1)
                  overlap = round(area_truth/np.count_nonzero(each_roi) *100,3)
                  # cv.drawContours(img, ct_pred, -1, (0,255,0), 3)

                  # cv.drawContours(img, ct_pred , -1, (255,0,0), 3)

                  if overlap > 0:
                       overlap_list.append(overlap)
                       print("overlap", overlap)

                       x, y, w, h = cv.boundingRect(ct_pred)

                      #  overlap_img_anotation = cv.fillPoly(overlap_img_anotation, pts=[ct_pred], color=(0, 0))

                      #  overlap_img_anotation = ImageDraw.Draw(overlap_img_anotation)
                      #  overlap_img_anotation.text((x, y), overlap, fill=(0 , 0, 255))
                       font = cv.FONT_HERSHEY_SIMPLEX
                       org = (x-10, y-20)
                       fontScale = 3
                       color = (0, 0, 255)
                       thickness = 2
                       text = str(overlap)
                       img = cv.putText(img, text, org, font, fontScale, color, thickness, cv.LINE_AA)

  # plt.imshow(pred)
  # plt.show()
  plt.imshow(bitwise_and_roi_pred, 'gray')
  plt.title('bitwise_and')
  plt.axis('off')
  plt.show()
  # plt.imshow(img)
  # plt.show()
  return bitwise_and_roi_pred, overlap_list, img

In [ ]:
def cal_overlaps_per_large_img_3(roi, pred, large_image):

  cal_overlaps_per_large_img_3
  contours_pred, _ = cv.findContours(pred, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  contours_roi, _ = cv.findContours(roi, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

  # pred = cv.fillPoly(pred, pts= contours_pred, color=(255, 255))
  # contours_pred, _ = cv.findContours(pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

  img = large_image.copy()

  overlap_list = []

  roi_num_cnt = len(contours_roi)
  print("num ground truth", roi_num_cnt)
  print("num Predict:", len(contours_pred))

  bitwise_and_roi_pred = cv.bitwise_and(pred, roi)

  cv.drawContours(img, contours_roi , -1, (255,0,0), 3)
  cv.drawContours(img, contours_pred, -1, (0,255,0), 3)

  for ct in contours_roi:
              each_roi= np.zeros(roi.shape, dtype="uint8")
              each_roi = cv.fillPoly(each_roi, pts=[ct], color=(255, 255))

              for ct_pred in contours_pred:
                  each_pred = np.zeros(pred.shape, dtype="uint8")
                  each_pred = cv.fillPoly(each_pred, pts=[ct_pred], color=(255, 255))

                  bitwise_and_roi_pred_1 = cv.bitwise_and(each_pred, each_roi)
                  area_truth = np.count_nonzero(bitwise_and_roi_pred_1)

                  if area_truth > 0:
                       overlap = round(area_truth/np.count_nonzero(each_roi) * 100,3)
                       overlap_list.append(overlap)
                       print("overlap", overlap)

                       x, y, w, h = cv.boundingRect(ct_pred)

                       font = cv.FONT_HERSHEY_SIMPLEX
                       org = (x-10, y-20)
                       fontScale = 3
                       color = (0, 0, 255)
                       thickness = 2
                       text = str(overlap)
                       img = cv.putText(img, text, org, font, fontScale, color, thickness, cv.LINE_AA)

  # plt.imshow(bitwise_and_roi_pred, 'gray')
  # plt.title('bitwise_and')
  # plt.axis('off')
  # plt.show()
  # plt.imshow(img)
  # plt.show()
  return bitwise_and_roi_pred, overlap_list, img

In [ ]:
def cal_overlaps_per_large_img_2(roi, pred, large_image):

  pred, n = filter_FP(pred, 2000)
  roi_filter, n = filter_FP(roi, 100)
  roi_filter_1 = np.zeros(roi.shape, dtype="uint8")

  roi_filter = roi.copy()
  kernel = np.ones((3,3),np.uint8)
  roi_filter = cv.erode(roi_filter, kernel)

  contours, _ = cv.findContours(roi_filter.copy(), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
  n = 0
  for ct in contours:
      contour_area = cv.contourArea(ct)
      if (contour_area>= 100):
          n += 1
          roi_filter_1 = cv.fillPoly(roi_filter_1, pts= [ct], color=(255, 255))

      # else:
      #   roi_filter = cv.fillPoly(roi_filter, pts=[ct], color=(0, 0))
  roi_filter_1 =  cv.dilate(roi_filter_1, kernel)

  plt.imshow(roi_filter_1)
  plt.show()

  contours_pred, _ = cv.findContours(pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  contours_roi, _ = cv.findContours(roi_filter_1, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

  # for ct_roi in contours_roi:
  #       area = cv.contourArea(ct_roi)
  #       if (area < 500):
  #            print(area)
  #            roi = cv.fillPoly(roi_filter, pts= [ct_roi], color=(0, 0))

  pred = cv.fillPoly(pred, pts= contours_pred, color=(255, 255))
  contours_pred, _ = cv.findContours(pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

  # roi_filter = cv.fillPoly(roi_filter, pts= contours_roi, color=(255, 255))
  # contours_roi, _ = cv.findContours(roi_filter.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)


  img = large_image.copy()

  overlap_list = []

  roi_num_cnt = len(contours_roi)
  print("num ground truth", roi_num_cnt)
  print("num Predict:", len(contours_pred))

  bitwise_and_roi_pred = cv.bitwise_and(pred, roi_filter_1)

  # plt.imshow(pred)
  # plt.show()
  # plt.imshow(bitwise_and_roi_pred, 'gray')
  # plt.show()
  # plt.imshow(img)
  # plt.show()
  return bitwise_and_roi_pred, overlap_list, img


In [ ]:
predict_path ="/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal/original_predict/Mass-Test_P_00116_RIGHT_CC.png"
image_path = "/content/Abnormal/image/Mass-Test_P_00116_RIGHT_CC.png"
roi_path = "/content/Abnormal/mask/Mass-Test_P_00116_RIGHT_CC.png"

predict = cv.imread(predict_path, 0).astype(np.uint8)
predict = np.array(predict)

image = cv.imread(image_path)


roi = cv.imread(roi_path, 0).astype(np.uint8)
roi = np.array(roi)

h_r, w_r = roi.shape



h_i, w_i = predict.shape

if (h_i>=h_r) or (w_i>=w_r):
      predict = predict[0:h_r, 0:w_r]
else:
      roi = roi[0:h_i, 0:w_i]

bitwise_and_roi_pred, overlap_list, img_anotation = cal_overlaps_per_large_img_1(roi, predict, image)



In [ ]:
def cal_dice(roi, pred):

  contours_pred, _ = cv.findContours(pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  contours_roi, _ = cv.findContours(roi.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)


  # pred = cv.fillPoly(pred, pts= contours_pred, color=(255, 255))
  # contours_pred, _ = cv.findContours(pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

  dice_list = []

  roi_num_cnt = len(contours_roi)
  print("num ground truth", roi_num_cnt)
  print("num Predict:", len(contours_pred))

  for ct in contours_roi:
              each_roi= np.zeros(roi.shape, dtype="uint8")
              each_roi = cv.fillPoly(each_roi, pts=[ct], color=(255, 255))

              for ct_pred in contours_pred:
                  each_pred = np.zeros(pred.shape, dtype="uint8")
                  each_pred = cv.fillPoly(each_pred, pts=[ct_pred], color=(255, 255))

                  dice = compute_dice_1(each_roi, each_pred)

                  if dice > 0:
                       dice_list.append(dice)

  return dice_list

In [ ]:
def cal_iou(roi, pred):

  contours_pred, _ = cv.findContours(pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  contours_roi, _ = cv.findContours(roi.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)


  # pred = cv.fillPoly(pred, pts= contours_pred, color=(255, 255))
  # contours_pred, _ = cv.findContours(pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

  iou_list = []

  roi_num_cnt = len(contours_roi)
  print("num ground truth", roi_num_cnt)
  print("num Predict:", len(contours_pred))

  for ct in contours_roi:
              each_roi= np.zeros(roi.shape, dtype="uint8")
              each_roi = cv.fillPoly(each_roi, pts=[ct], color=(255, 255))

              for ct_pred in contours_pred:
                  each_pred = np.zeros(pred.shape, dtype="uint8")
                  each_pred = cv.fillPoly(each_pred, pts=[ct_pred], color=(255, 255))

                  iou = compute_iou(each_roi, each_pred)

                  if iou > 0:
                       iou_list.append(iou)

  return iou_list

In [ ]:
def cal_dice_iou(roi, pred):

  contours_pred, _ = cv.findContours(pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  contours_roi, _ = cv.findContours(roi.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)


  # pred = cv.fillPoly(pred, pts= contours_pred, color=(255, 255))
  # contours_pred, _ = cv.findContours(pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

  dice_list = []
  iou_list =[]
  roi_num_cnt = len(contours_roi)
  print("num ground truth", roi_num_cnt)
  print("num Predict:", len(contours_pred))

  for ct in contours_roi:
              each_roi= np.zeros(roi.shape, dtype="uint8")
              each_roi = cv.fillPoly(each_roi, pts=[ct], color=(255, 255))

              for ct_pred in contours_pred:
                  each_pred = np.zeros(pred.shape, dtype="uint8")
                  each_pred = cv.fillPoly(each_pred, pts=[ct_pred], color=(255, 255))

                  dice = compute_dice_1(each_roi, each_pred)
                  iou = compute_iou(each_roi, each_pred)
                  if dice > 0:
                       dice_list.append(dice)
                  if iou > 0:
                       iou_list.append(iou)

  return dice_list, iou_list

In [ ]:
from csv import DictReader
csv_path = '/content/drive/MyDrive/Mammo dataset/Test DDSM/abnormal_image.csv'
predict_path = "/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal/original_predict/"

from csv import DictReader
from pathlib import Path

dice_list = []
iou_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)
    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        name_file_saver = os.path.basename(img_file)

        pred_path= os.path.join(predict_path,''.join([name_file_saver]))

        roi = cv.imread(roi_file, 0)
        roi, n_1 = filter_FP(roi, 300)


        pred = cv.imread(pred_path, 0)
        pred, n_1 = filter_FP(pred, 300)

        dice, iou = cal_dice_iou(roi, pred)

        print(dice, iou)
        dice_list.append(dice)
        iou_list.append(iou)


In [ ]:
import pandas as pd
df = pd.DataFrame({'standard_dice': dice_list, 'standard_iou': iou_list})
df.to_csv ('/content/drive/MyDrive/Mammo dataset/dice_iou_cbis_test_set_118_standard.csv', index = False, header=True)


In [ ]:
from csv import DictReader
csv_path = '/content/drive/MyDrive/Mammo dataset/Test DDSM/abnormal_image.csv'
predict_path = "/content/drive/MyDrive/Test DDSM 185 Images/without 2 skip connection/Abnormal/original_predict/"

from csv import DictReader
from pathlib import Path

dice_list_2nd = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)
    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        name_file_saver = os.path.basename(img_file)

        pred_path= os.path.join(predict_path,''.join([name_file_saver]))

        roi = cv.imread(roi_file, 0)
        roi, n_1 = filter_FP(roi, 300)


        pred = cv.imread(pred_path, 0)
        pred, n_1 = filter_FP(pred, 300)

        dice = cal_dice(roi, pred)

        print(dice)
        dice_list_2nd.append(dice)


In [ ]:
from csv import DictReader
csv_path = '/content/drive/MyDrive/Mammo dataset/Test DDSM/abnormal_image.csv'
predict_path = "/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal/original_predict/"

from csv import DictReader
from pathlib import Path

iou_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)
    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        name_file_saver = os.path.basename(img_file)

        pred_path= os.path.join(predict_path,''.join([name_file_saver]))

        roi = cv.imread(roi_file, 0)
        roi, n_1 = filter_FP(roi, 300)


        pred = cv.imread(pred_path, 0)
        pred, n_1 = filter_FP(pred, 300)

        iou = cal_iou(roi, pred)

        print(iou)
        iou_list.append(iou)


In [ ]:
from csv import DictReader
csv_path = '/content/drive/MyDrive/Mammo dataset/Test DDSM/abnormal_image.csv'
predict_path = "/content/drive/MyDrive/Test DDSM 185 Images/without 2 skip connection/Abnormal/original_predict/"

from csv import DictReader
from pathlib import Path

iou_list_2nd = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)
    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        name_file_saver = os.path.basename(img_file)

        pred_path= os.path.join(predict_path,''.join([name_file_saver]))

        roi = cv.imread(roi_file, 0)
        roi, n_1 = filter_FP(roi, 300)


        pred = cv.imread(pred_path, 0)
        pred, n_1 = filter_FP(pred, 300)

        iou = cal_iou(roi, pred)

        print(iou)
        iou_list_2nd.append(iou)


In [ ]:
import pandas as pd
df = pd.DataFrame({'standard_dice': dice_list, '2nd_dice': dice_list_2nd, 'standard_iou': iou_list,'2nd_iou': iou_list_2nd })
df.to_csv ('/content/drive/MyDrive/Mammo dataset/dice_iou_cbis_test_set_118.csv', index = False, header=True)


40G ram, GPU , A100

#####predict whole for each patch size

In [ ]:
load_model_file= tf.keras.models.load_model(model_file)

In [ ]:
patch_384 = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal_each_patch_size/patch 384'
patch_512 = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal_each_patch_size/patch 512'
patch_1024 = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal_each_patch_size/patch 1024'

csv_path = '/content/drive/MyDrive/Mammo dataset/Test DDSM/abnormal_image.csv'
pred_path = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal/original_predict'
overlap_path = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal/overlap'
draw_contour_path = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal/draw contour'
overlap_anotation_path = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal/overlap anotation'

import time

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path


overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []
small_image = []
num_FP_384 = []
num_FP_512 = []
num_FP_1024 = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    start_time = time.time()

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']
        name_file_saver = os.path.basename(img_file)

        large_image_1 = cv.imread(img_file, 0)
        image_anotation = cv.imread(img_file)
        large_image= np.float32(large_image_1)
        large_image = large_image/255.0

        roi = cv.imread(roi_file, 0).astype(np.uint8)

        # roi = cv.imread(roi_file, 0)
        # roi= np.uint8(roi)
        # print(roi.shape)
        h_r, w_r = roi.shape

        large_image, roi= check_Left_Orient (large_image, roi)

        img = np.array(large_image)

        h_i, w_i = img.shape

        if (h_i>=h_r) or (w_i>=w_r):
          img = img[0:h_r, 0:w_r]
          image_anotation = image_anotation[0:h_r, 0:w_r]
        else:
          roi = roi[0:h_i, 0:w_i]

        # print(img.shape)

        roi_predict_384 = detect_roi_by_patch_size(img, load_model_file, 348, 64)

        y_pred_filter_384, n_384 = filter_FP(roi_predict_384, 2000)
        num_FP_384.append(n_384)

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
        y_pred_filter_384 = cv.morphologyEx(y_pred_filter_384, cv.MORPH_CLOSE, kernel)
        y_pred_filter_384 = cv.morphologyEx(y_pred_filter_384, cv.MORPH_OPEN, kernel)

        file_patch_384_path = os.path.join(patch_384,
                                 ''.join([name_file_saver]))
        plt.imsave(file_patch_384_path, y_pred_filter_384, cmap='gray')



#         if 600 < w_i < 1024:
#           roi_predict_512 = detect_roi_by_patch_size(img, load_model_file, 512, 64)
#           roi_predict_1024 = np.zeros(img.shape, dtype="uint8")

#         elif w_i <= 600:
#           print("image < 512", name_file_saver)
#           small_image.append(name_file_saver)
#           continue
#         else:
#           roi_predict_1024 = detect_roi_by_patch_size(img, load_model_file, 1024, 128)
#           roi_predict_512 = detect_roi_by_patch_size(img, load_model_file, 512, 64)

#         y_pred_filter_512, n_512 = filter_FP(roi_predict_512, 2000)
#         num_FP_512.append(n_512)
#         y_pred_filter_512 = cv.morphologyEx(y_pred_filter_512, cv.MORPH_CLOSE, kernel)
#         y_pred_filter_512 = cv.morphologyEx(y_pred_filter_512, cv.MORPH_OPEN, kernel)

#         file_patch_512_path = os.path.join(patch_512,
#                                  ''.join([name_file_saver]))
#         plt.imsave(file_patch_512_path, y_pred_filter_512, cmap='gray')


#         y_pred_filter_1024, n_1024 = filter_FP(roi_predict_1024, 2000)
#         num_FP_1024.append(n_1024)
#         y_pred_filter_1024 = cv.morphologyEx(y_pred_filter_1024, cv.MORPH_CLOSE, kernel)
#         y_pred_filter_1024 = cv.morphologyEx(y_pred_filter_1024, cv.MORPH_OPEN, kernel)

#         file_patch_1024_path = os.path.join(patch_1024,
#                                  ''.join([name_file_saver]))
#         plt.imsave(file_patch_1024_path, y_pred_filter_1024, cmap='gray')

#         # print(roi_predict_384.shape, roi_predict_512.shape, roi_predict_1024.shape)


#         # combine = np.zeros(roi.shape, dtype=np.uint8)

#         combine_1 = np.logical_or(roi_predict_384, roi_predict_512).astype(np.uint8)
#         combine = np.logical_or(combine_1, roi_predict_1024).astype(np.uint8)


#         y_pred_filter, n = filter_FP(combine, 2000)
#         roi, n_1 = filter_FP(roi, 300)

#         kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
#         y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_CLOSE, kernel)
#         y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)

#         num_P_list.append(n)

#         pred_file_path = os.path.join(pred_path,
#                                  ''.join([name_file_saver]))
#         plt.imsave(pred_file_path, y_pred_filter, cmap='gray')

#         overlap_img, overlap_list, img_anotation_1 = cal_overlaps_per_large_img_1(roi, y_pred_filter, image_anotation)

#         # overlap_img_anotation = 1*overlap_img_anotation
#         img_overlap_anotation_path = os.path.join(overlap_anotation_path,
#                                  ''.join([name_file_saver]))
#         plt.imsave(img_overlap_anotation_path, img_anotation_1, cmap='gray')
#         plt.title(name_file_saver)
#         plt.imshow(img_anotation_1)
#         plt.show()
#         # print(overlap)
#         area_truth_list.append(overlap_list)

#         # overlap_img = 1*overlap_img
#         file_overlap_img_path = os.path.join(overlap_path,
#                                  ''.join([name_file_saver]))
#         plt.imsave(file_overlap_img_path, overlap_img, cmap='gray')

#         img_draw = cv.imread(img_file)
#         # combine_predict = cv.imread(pred_file_path, 0)
#         img_draw_contour = draw_contour_pred_and_truth(img_draw, y_pred_filter, roi)

#         # plt.imshow(img_draw_contour)

#         # plt.title(name_file_saver + "- overlap = "+ str(overlap))
#         # plt.show()
#         img_draw_contour_path = os.path.join(draw_contour_path,
#                                  ''.join([name_file_saver]))
#         plt.imsave(img_draw_contour_path, img_draw_contour, cmap='gray')

# print("Time taken: %.2fs" % (time.time() - start_time))


In [ ]:
csv_path = '/content/drive/MyDrive/Mammo dataset/Test DDSM/abnormal_image.csv'
pred_path = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal/original_predict'
overlap_path = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal/overlap'
draw_contour_path = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal/draw contour'
overlap_anotation_path = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal/overlap anotation'

import time

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path


overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []
small_image = []

with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    start_time = time.time()

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']
        name_file_saver = os.path.basename(img_file)

        large_image_1 = cv.imread(img_file, 0)
        image_anotation = cv.imread(img_file)
        large_image= np.float32(large_image_1)
        large_image = large_image/255.0

        roi = cv.imread(roi_file, 0).astype(np.uint8)

        # roi = cv.imread(roi_file, 0)
        # roi= np.uint8(roi)
        # print(roi.shape)
        h_r, w_r = roi.shape

        large_image, roi= check_Left_Orient (large_image, roi)

        img = np.array(large_image)

        h_i, w_i = img.shape

        if (h_i>=h_r) or (w_i>=w_r):
          img = img[0:h_r, 0:w_r]
          image_anotation = image_anotation[0:h_r, 0:w_r]
        else:
          roi = roi[0:h_i, 0:w_i]

        # print(img.shape)

        roi_predict_384 = detect_roi_by_patch_size(img, load_model_file, 348, 64)

        y_pred_filter_384, n = filter_FP(roi_predict_384, 2000)

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
        y_pred_filter_384 = cv.morphologyEx(y_pred_filter_384, cv.MORPH_CLOSE, kernel)
        y_pred_filter_384 = cv.morphologyEx(y_pred_filter_384, cv.MORPH_OPEN, kernel)



        if 600 < w_i < 1024:
          roi_predict_512 = detect_roi_by_patch_size(img, load_model_file, 512, 64)
          roi_predict_1024 = np.zeros(img.shape, dtype="uint8")

        elif w_i <= 600:
          print("image < 512", name_file_saver)
          small_image.append(name_file_saver)
          continue
        else:
          roi_predict_1024 = detect_roi_by_patch_size(img, load_model_file, 1024, 128)
          roi_predict_512 = detect_roi_by_patch_size(img, load_model_file, 512, 64)

        # print(roi_predict_384.shape, roi_predict_512.shape, roi_predict_1024.shape)


        # combine = np.zeros(roi.shape, dtype=np.uint8)

        combine_1 = np.logical_or(roi_predict_384, roi_predict_512).astype(np.uint8)
        combine = np.logical_or(combine_1, roi_predict_1024).astype(np.uint8)


        y_pred_filter, n = filter_FP(combine, 2000)
        roi, n_1 = filter_FP(roi, 300)

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_CLOSE, kernel)
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)

        num_P_list.append(n)

        pred_file_path = os.path.join(pred_path,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path, y_pred_filter, cmap='gray')

        overlap_img, overlap_list, img_anotation_1 = cal_overlaps_per_large_img_1(roi, y_pred_filter, image_anotation)

        # overlap_img_anotation = 1*overlap_img_anotation
        img_overlap_anotation_path = os.path.join(overlap_anotation_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_overlap_anotation_path, img_anotation_1, cmap='gray')
        plt.title(name_file_saver)
        plt.imshow(img_anotation_1)
        plt.show()
        # print(overlap)
        area_truth_list.append(overlap_list)

        # overlap_img = 1*overlap_img
        file_overlap_img_path = os.path.join(overlap_path,
                                 ''.join([name_file_saver]))
        plt.imsave(file_overlap_img_path, overlap_img, cmap='gray')

        img_draw = cv.imread(img_file)
        # combine_predict = cv.imread(pred_file_path, 0)
        img_draw_contour = draw_contour_pred_and_truth(img_draw, y_pred_filter, roi)

        # plt.imshow(img_draw_contour)

        # plt.title(name_file_saver + "- overlap = "+ str(overlap))
        # plt.show()
        img_draw_contour_path = os.path.join(draw_contour_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_draw_contour_path, img_draw_contour, cmap='gray')

print("Time taken: %.2fs" % (time.time() - start_time))


In [ ]:
csv_path = '/content/drive/MyDrive/Mammo dataset/Test DDSM/abnormal_image.csv'
pred_path = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal/original_predict'

import time

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path

num_P_list_1 = []

with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    for row in csv_dict_reader:
        img_file = row['img']
        name_file_saver = os.path.basename(img_file)

        predict_path = pred_path + "//" + name_file_saver

        predict = cv.imread(predict_path, 0)
        contours, _ = cv.findContours(predict.copy(), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        FP = len(contours)
        num_P_list_1.append(FP)


In [ ]:
csv_path = '/content/drive/MyDrive/Mammo dataset/Test DDSM/abnormal_image.csv'

import time

from csv import DictReader
from pathlib import Path

num_groundtruth_list = []

with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    for row in csv_dict_reader:
        roi_file = row['roi']

        roi = cv.imread(roi_file, 0)
        roi_1, n_1 = filter_FP(roi, 300)
        # contours, _ = cv.findContours(roi_1.copy(), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        # FP = len(contours)
        num_groundtruth_list.append(n_1)

In [ ]:
len(num_groundtruth_list)

In [ ]:
data = pd.read_csv(csv_path)
img = data["img"]
roi = data["roi"]
import pandas as pd
save_csv_path= '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal/num_GT_abnormal.csv'
df = pd.DataFrame({'img': img, 'roi': roi, 'num_GT': num_groundtruth_list})
df.to_csv (save_csv_path, index = False, header=True)

In [ ]:
print(len(num_P_list_1))

In [ ]:
data = pd.read_csv(csv_path)
img = data["img"]
roi = data["roi"]
import pandas as pd
save_csv_path= '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal/predict_abnormal.csv'
df = pd.DataFrame({'img': img, 'roi': roi, 'overlap': area_truth_list, 'num_FP': num_P_list_1})
df.to_csv (save_csv_path, index = False, header=True)

In [ ]:
import csv
from csv import DictReader
csv_path= '/content/predict_abnormal_v1.csv'

with open(csv_path, 'r') as csvfile:
    outwriter=csv.writer(csvfile, delimiter=',')
    csv_dict_reader = DictReader(csvfile)
    for row in csv_dict_reader:
        num_overlap = row['num overlap']
        num_row_insert = int(num_overlap)
        for n in range(num_row_insert):
                 outwriter.writerow([])

##### Combine whole image - each patch size-cbis-abnormal-standard

In [ ]:
drive_path_1 = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model'
drive_path = '/content/drive/MyDrive'
dataset_path = '/content'

# patch_384 = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal_each_patch_size/patch 384'
# patch_512 = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal_each_patch_size/patch 512'
# patch_1024 = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal_each_patch_size/patch 1024'

predict_patch_384= sorted(glob(os.path.join(drive_path_1, 'Abnormal_each_patch_size', 'patch 384', '*')))
predict_patch_512= sorted(glob(os.path.join(drive_path_1, 'Abnormal_each_patch_size', 'patch 512', '*')))
predict_patch_1024= sorted(glob(os.path.join(drive_path_1, 'Abnormal_each_patch_size', 'patch 1024', '*')))

img= sorted(glob(os.path.join(dataset_path, 'Abnormal', 'image', '*')))
mask= sorted(glob(os.path.join(dataset_path, 'Abnormal', 'mask', '*')))

In [ ]:
import pandas as pd
df = pd.DataFrame({'img': img, 'mask' : mask  ,'predict_patch_384': predict_patch_384 ,'predict_patch_512': predict_patch_512, 'predict_patch_1024': predict_patch_1024})
df.to_csv ('/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal_each_patch_size/combine_384_512_1024_predict_whole_cbis.csv', index = False, header=True)
print('Done')

In [ ]:
csv_path = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal_each_patch_size/combine_384_512_1024_predict_whole_cbis.csv'
pred_path = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal/original_predict'
overlap_path = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal/overlap'
draw_contour_path = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal/draw contour'
overlap_anotation_path = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal/overlap anotation'


from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path

overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['mask']

        image_anotation = cv.imread(img_file)

        mask_patch_384_file = row['predict_patch_384']
        mask_patch_512_file = row['predict_patch_512']
        mask_patch_1024_file = row['predict_patch_1024']

        name_file_saver = os.path.basename(img_file)

        large_image = cv.imread(img_file, 0)

        roi = cv.imread(roi_file, 0)
        roi, n_1 = filter_FP(roi, 300)

        mask_512 = cv.imread(mask_patch_512_file, 0)
        mask_1024 = cv.imread(mask_patch_1024_file, 0)
        mask_384 = cv.imread(mask_patch_384_file, 0)

        # mask_384 = np.array(mask_384)

        # h,w = mask_512.shape

        # mask_384 = mask_384[0:h, 0: w]

        combine = np.zeros(mask_1024.shape, dtype=np.uint8)

        combine_1 = np.logical_or(mask_384, mask_512).astype(np.uint8)
        combine = np.logical_or(combine_1, mask_1024).astype(np.uint8)
        # combine = np.array(combine)


        y_pred_filter, n = filter_FP(combine, 2000 )

        plt.imshow(y_pred_filter, 'gray')
        plt.title(name_file_saver + " - combine predict")
        plt.show()
        num_P_list.append(n)

        pred_file_path = os.path.join(pred_path,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path, y_pred_filter, cmap='gray')


        overlap_img, overlap_list, img_anotation_1 = cal_overlaps_per_large_img_3(roi, y_pred_filter, image_anotation)

        img_overlap_anotation_path = os.path.join(overlap_anotation_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_overlap_anotation_path, img_anotation_1, cmap='gray')
        plt.title(name_file_saver)
        plt.imshow(img_anotation_1)
        plt.show()

        area_truth_list.append(overlap_list)

        # overlap_img = 1*overlap_img
        file_overlap_img_path = os.path.join(overlap_path,
                                 ''.join([name_file_saver]))
        plt.imsave(file_overlap_img_path, overlap_img, cmap='gray')

        img_draw = cv.imread(img_file)
        # combine_predict = cv.imread(pred_file_path, 0)
        img_draw_contour = draw_contour_pred_and_truth(img_draw, y_pred_filter, roi)

        img_draw_contour_path = os.path.join(draw_contour_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_draw_contour_path, img_draw_contour, cmap='gray')


In [ ]:
from csv import DictReader
csv_path = '/content/drive/MyDrive/Mammo dataset/Test DDSM/abnormal_image.csv'
predict_path = "/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal/original_predict/"

from csv import DictReader
from pathlib import Path

dice_list = []
iou_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)
    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        name_file_saver = os.path.basename(img_file)

        pred_path= os.path.join(predict_path,''.join([name_file_saver]))

        roi = cv.imread(roi_file, 0)
        roi, n_1 = filter_FP(roi, 300)


        pred = cv.imread(pred_path, 0)
        pred, n_1 = filter_FP(pred, 2000)

        dice, iou = cal_dice_iou(roi, pred)

        print(dice, iou)
        dice_list.append(dice)
        iou_list.append(iou)


In [ ]:
import pandas as pd
df = pd.DataFrame({'standard_dice': dice_list, 'standard_iou': iou_list})
df.to_csv ('/content/drive/MyDrive/Mammo dataset/dice_iou_cbis_test_set_118_standard_13.9.csv', index = False, header=True)


In [ ]:
!unrar x "/content/drive/MyDrive/Mammo dataset/Abnormal_each_patch_size.rar"

#### tính lại overlap, iou, dice

In [ ]:
!unrar x "/content/drive/MyDrive/Predict 118 images.rar"

In [ ]:
drive_path_1 = '/content/Predict 118 images'
drive_path = '/content/drive/MyDrive'
dataset_path = '/content'

origin_predict_standard = sorted(glob(os.path.join(drive_path_1, 'standard', 'original_predict', '*')))
origin_predict_2nd = sorted(glob(os.path.join(drive_path_1, 'without 2 skip connection', 'original_predict', '*')))

In [ ]:
print(len(origin_predict_standard), len(origin_predict_2nd))

In [ ]:
import pandas as pd
df = pd.DataFrame({'origin_predict_standard': origin_predict_standard, 'origin_predict_2nd': origin_predict_2nd})
df.to_csv ('/content/drive/MyDrive/Mammo dataset/Test DDSM/origin_predict_abnormal_image.csv', index = False, header=True)

In [ ]:
csv_path = '/content/drive/MyDrive/Mammo dataset/Test DDSM/origin_predict_abnormal_image.csv'


from csv import DictReader
from pathlib import Path

num_Pre_standard_list = []
num_Pre_2nd_list = []

with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    for row in csv_dict_reader:
        origin_predict_standard_file = row['origin_predict_standard']
        origin_predict_2nd_file = row['origin_predict_2nd']

        origin_predict_standard = cv.imread(origin_predict_standard_file, 0)
        origin_predict_2nd = cv.imread(origin_predict_2nd_file, 0)
        # roi, n = filter_FP (roi, 300)
        contours_standard, _ = cv.findContours(origin_predict_standard, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        contours_2nd, _ = cv.findContours(origin_predict_2nd, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        num_Pre_standard = len(contours_standard)
        num_Pre_2nd = len(contours_2nd)

        num_Pre_2nd_list.append(num_Pre_2nd)
        num_Pre_standard_list.append(num_Pre_standard)




In [ ]:
import pandas as pd
df = pd.DataFrame({'num_Pre_standard': num_Pre_standard_list, 'num_Pre_2nd': num_Pre_2nd_list})
df.to_csv ('/content/drive/MyDrive/Mammo dataset/Test DDSM/num_predict_118__abnormal_image.csv', index = False, header=True)

#### test abnormal

In [ ]:
Mass-Training_P_00710_LEFT_CC.png

In [ ]:
        from patchify import patchify,  unpatchify
        image_path = "/content/Abnormal/image/Mass-Training_P_00710_LEFT_CC.png"
        roi_path = "/content/Abnormal/mask/Mass-Training_P_00710_LEFT_CC.png"

        patch_size = 512
        step = 64


        large_image_1 = cv.imread(image_path, 0)
        image_anotation = cv.imread(image_path)
        large_image= np.float32(large_image_1)
        large_image = large_image/255.0

        roi = cv.imread(roi_path,0)
        roi = np.uint8(roi)

        h_r, w_r = roi.shape

        large_image, roi= check_Left_Orient (large_image, roi)

        img = np.array(large_image)

        h_i, w_i = img.shape

        if (h_i>=h_r) or (w_i>=w_r):
          img = img[0:h_r, 0:w_r]
          image_anotation = image_anotation[0:h_r, 0:w_r]
        else:
          roi = roi[0:h_i, 0:w_i]

        # print(img.shape)

        roi_predict_384 = detect_roi_by_patch_size(img, load_model_file, 348, 64)
        plt.title('patch 384')
        plt.imshow(roi_predict_384, 'gray')
        plt.show()

        if 600 < w_i < 1024:
          roi_predict_512 = detect_roi_by_patch_size(img, load_model_file, 512, 64)
          roi_predict_1024 = np.zeros(img.shape, dtype="uint8")

        roi_predict_1024 = detect_roi_by_patch_size(img, load_model_file, 1024, 128)
        plt.title('patch 1024')
        plt.imshow(roi_predict_1024, 'gray')
        plt.show()

        roi_predict_512 = detect_roi_by_patch_size(img, load_model_file, 512, 64)
        plt.title('patch 512')
        plt.imshow(roi_predict_1024, 'gray')
        plt.show()



        # print(roi_predict_384.shape, roi_predict_512.shape, roi_predict_1024.shape)


        # combine = np.zeros(roi.shape, dtype=np.uint8)

        combine_1 = np.logical_or(roi_predict_384, roi_predict_512).astype(np.uint8)
        combine = np.logical_or(combine_1, roi_predict_1024).astype(np.uint8)

        plt.title('before filter')
        plt.imshow(combine, 'gray')
        plt.show()


        y_pred_filter, n = filter_FP(combine, 2000)
        roi, n_1 = filter_FP(roi, 300)

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_CLOSE, kernel)
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)

        plt.title('after filter')
        plt.imshow(y_pred_filter, 'gray')
        plt.show()

        # num_P_list.append(n)

        # pred_file_path = os.path.join(pred_path,
        #                          ''.join([name_file_saver]))
        # plt.imsave(pred_file_path, y_pred_filter, cmap='gray')

        overlap_img, overlap_list, img_anotation_1 = cal_overlaps_per_large_img_1(roi, y_pred_filter, image_anotation)

        # overlap_img_anotation = 1*overlap_img_anotation
        # img_overlap_anotation_path = os.path.join(overlap_anotation_path,
        #                          ''.join([name_file_saver]))
        # plt.imsave(img_overlap_anotation_path, img_anotation_1, cmap='gray')
        plt.title('anotation 1')
        plt.imshow(img_anotation_1)
        plt.show()
        # print(overlap)
        # area_truth_list.append(overlap_list)

        # overlap_img = 1*overlap_img
        # file_overlap_img_path = os.path.join(overlap_path,
        #                          ''.join([name_file_saver]))
        # plt.imsave(file_overlap_img_path, overlap_img, cmap='gray')



        img_draw = cv.imread(image_path)
        # combine_predict = cv.imread(pred_file_path, 0)
        img_draw_contour = draw_contour_pred_and_truth(img_draw, y_pred_filter, roi)

        plt.imshow(img_draw_contour)

        # plt.title(name_file_saver + "- overlap = "+ str(overlap))
        plt.show()

        roi = cv.imread(roi_path,0)
        plt.imshow(roi,'gray')
        plt.title("roi")
        plt.show()
        # img_draw_contour_path = os.path.join(draw_contour_path,
        #                          ''.join([name_file_saver]))
        # plt.imsave(img_draw_contour_path, img_draw_contour, cmap='gray')


In [ ]:
load_model_file = tf.keras.models.load_model(model_file)

In [ ]:
csv_path = '/content/drive/MyDrive/Mammo dataset/Test DDSM/abnormal_image.csv'
patch_384 = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal_each_patch_size/patch 384'
patch_512 = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal_each_patch_size/patch 512'
patch_1024 = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal_each_patch_size/patch 1024'

import time

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path


overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []
small_image = []

with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    start_time = time.time()

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']
        name_file_saver = os.path.basename(img_file)

        large_image_1 = cv.imread(img_file, 0)
        image_anotation = cv.imread(img_file)
        large_image= np.float32(large_image_1)
        large_image = large_image/255.0

        roi = cv.imread(roi_file, 0).astype(np.uint8)

        # roi = cv.imread(roi_file, 0)
        # roi= np.uint8(roi)
        # print(roi.shape)
        h_r, w_r = roi.shape

        large_image, roi= check_Left_Orient (large_image, roi)

        img = np.array(large_image)

        h_i, w_i = img.shape

        if (h_i>=h_r) or (w_i>=w_r):
          img = img[0:h_r, 0:w_r]
          image_anotation = image_anotation[0:h_r, 0:w_r]
        else:
          roi = roi[0:h_i, 0:w_i]

        # print(img.shape)

        roi_predict_384 = detect_roi_by_patch_size(img, load_model_file, 348, 64)


        if 600 < w_i < 1024:
          roi_predict_512 = detect_roi_by_patch_size(img, load_model_file, 512, 64)
          roi_predict_1024 = np.zeros(img.shape, dtype="uint8")

        elif w_i <= 600:
          print("image < 512", name_file_saver)
          small_image.append(name_file_saver)
          continue
        else:
          roi_predict_1024 = detect_roi_by_patch_size(img, load_model_file, 1024, 128)
          roi_predict_512 = detect_roi_by_patch_size(img, load_model_file, 512, 64)

        # print(roi_predict_384.shape, roi_predict_512.shape, roi_predict_1024.shape)


        # combine = np.zeros(roi.shape, dtype=np.uint8)

        combine_1 = np.logical_or(roi_predict_384, roi_predict_512).astype(np.uint8)
        combine = np.logical_or(combine_1, roi_predict_1024).astype(np.uint8)


        y_pred_filter, n = filter_FP(combine, 2000)
        roi, n_1 = filter_FP(roi, 300)

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_CLOSE, kernel)
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)

        num_P_list.append(n)

        pred_file_path = os.path.join(pred_path,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path, y_pred_filter, cmap='gray')

        overlap_img, overlap_list, img_anotation_1 = cal_overlaps_per_large_img_1(roi, y_pred_filter, image_anotation)

        # overlap_img_anotation = 1*overlap_img_anotation
        img_overlap_anotation_path = os.path.join(overlap_anotation_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_overlap_anotation_path, img_anotation_1, cmap='gray')
        plt.title(name_file_saver)
        plt.imshow(img_anotation_1)
        plt.show()
        # print(overlap)
        area_truth_list.append(overlap_list)

        # overlap_img = 1*overlap_img
        file_overlap_img_path = os.path.join(overlap_path,
                                 ''.join([name_file_saver]))
        plt.imsave(file_overlap_img_path, overlap_img, cmap='gray')

        img_draw = cv.imread(img_file)
        # combine_predict = cv.imread(pred_file_path, 0)
        img_draw_contour = draw_contour_pred_and_truth(img_draw, y_pred_filter, roi)

        # plt.imshow(img_draw_contour)

        # plt.title(name_file_saver + "- overlap = "+ str(overlap))
        # plt.show()
        img_draw_contour_path = os.path.join(draw_contour_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_draw_contour_path, img_draw_contour, cmap='gray')

print("Time taken: %.2fs" % (time.time() - start_time))


In [ ]:
!pip install patchify

In [ ]:
os.putenv('TF_GPU_ALLOCATOR', 'cuda_malloc_async')

In [ ]:
csv_path = '/content/drive/MyDrive/Mammo dataset/Test DDSM/abnormal_image_1.csv'
# pred_path = '/content/drive/MyDrive/Test CBIS Predict/original_predict'
# overlap_path = '/content/drive/MyDrive/Test CBIS Predict/overlap'
# draw_contour_path = '/content/drive/MyDrive/Test CBIS Predict/draw contour'
patch_384 = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal_each_patch_size/patch 384'

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path

overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        name_file_saver = os.path.basename(img_file)

        large_image_1 = cv.imread(img_file, 0)
        image_anotation = cv.imread(img_file)
        large_image= np.float32(large_image_1)
        large_image = large_image/255.0

        roi = cv.imread(roi_file,0)
        roi = np.uint8(roi)

        h_r, w_r = roi.shape

        large_image, roi= check_Left_Orient (large_image, roi)

        img = np.array(large_image)

        h_i, w_i = img.shape

        if (h_i>=h_r) or (w_i>=w_r):
          img = img[0:h_r, 0:w_r]
          image_anotation = image_anotation[0:h_r, 0:w_r]
        else:
          roi = roi[0:h_i, 0:w_i]

        # print(img.shape)

        roi_predict_384 = detect_roi_by_patch_size(img, load_model_file, 348, 64)
        # plt.title('patch 384')
        # plt.imshow(roi_predict_384, 'gray')
        # plt.show()

        y_pred_filter, n = filter_FP(roi_predict_384, 2000)
        roi, n_1 = filter_FP(roi, 300)

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_CLOSE, kernel)
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)

        file_patch_384_path = os.path.join(patch_384,
                                 ''.join([name_file_saver]))
        plt.imsave(file_patch_384_path, y_pred_filter, cmap='gray')

        num_P_list.append(n)

        overlap_img, overlap_list, img_anotation_1 = cal_overlaps_per_large_img_1(roi, y_pred_filter, image_anotation)

        area_truth_list.append(overlap_list)
        plt.title(name_file_saver)
        plt.imshow(img_anotation_1)
        plt.axis('off')
        plt.show()

        # img_draw = cv.imread(img_file)
        # img_draw_contour = draw_contour_pred_and_truth(img_draw,y_pred_filter,roi)

        # plt.imshow(img_draw_contour, 'gray')
        # plt.title(name_file_saver)
        # plt.axis('off')
        # plt.show()


In [ ]:
import gc
gc.collect()

In [ ]:
!nvidia-smi

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import torch

class ClearCache:
    def __enter__(self):
        torch.cuda.empty_cache()

    def __exit__(self, exc_type, exc_val, exc_tb):
        torch.cuda.empty_cache()

# Use the context manager
with ClearCache():
    # Define and train the PyTorch model
    ...

In [ ]:

from IPython.display import clear_output
clear_output()

In [ ]:
csv_path = '/content/drive/MyDrive/Mammo dataset/Test DDSM/abnormal_image_1.csv'
# pred_path = '/content/drive/MyDrive/Test CBIS Predict/original_predict'
# overlap_path = '/content/drive/MyDrive/Test CBIS Predict/overlap'
# draw_contour_path = '/content/drive/MyDrive/Test CBIS Predict/draw contour'
patch_512 = '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal_each_patch_size/patch 512'

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path

overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        name_file_saver = os.path.basename(img_file)

        large_image_1 = cv.imread(img_file, 0)
        image_anotation = cv.imread(img_file)
        large_image= np.float32(large_image_1)
        large_image = large_image/255.0

        roi = cv.imread(roi_file,0)
        roi = np.uint8(roi)

        h_r, w_r = roi.shape

        large_image, roi= check_Left_Orient (large_image, roi)

        img = np.array(large_image)

        h_i, w_i = img.shape

        if (h_i>=h_r) or (w_i>=w_r):
          img = img[0:h_r, 0:w_r]
          image_anotation = image_anotation[0:h_r, 0:w_r]
        else:
          roi = roi[0:h_i, 0:w_i]

        # print(img.shape)

        roi_predict_512 = detect_roi_by_patch_size(img, load_model_file, 512, 64)
        # plt.title('patch 384')
        # plt.imshow(roi_predict_384, 'gray')
        # plt.show()

        y_pred_filter, n = filter_FP(roi_predict_512, 2000)
        roi, n_1 = filter_FP(roi, 300)

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_CLOSE, kernel)
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)


        file_patch_512_path = os.path.join(patch_512,
                                 ''.join([name_file_saver]))
        plt.imsave(file_patch_512_path, y_pred_filter, cmap='gray')

        num_P_list.append(n)

        overlap_img, overlap_list, img_anotation_1 = cal_overlaps_per_large_img_1(roi, y_pred_filter, image_anotation)

        area_truth_list.append(overlap_list)

        plt.imshow(img_anotation_1)
        plt.title(name_file_saver)
        plt.axis('off')
        plt.show()

        # img_draw = cv.imread(img_file)
        # img_draw_contour = draw_contour_pred_and_truth(img_draw,y_pred_filter,roi)

        # plt.imshow(img_draw_contour, 'gray')
        # plt.title(name_file_saver)
        # plt.axis('off')
        # plt.show()


In [ ]:
csv_path = '/content/drive/MyDrive/Mammo dataset/Test DDSM/abnormal_image.csv'
# pred_path = '/content/drive/MyDrive/Test CBIS Predict/original_predict'
# overlap_path = '/content/drive/MyDrive/Test CBIS Predict/overlap'
# draw_contour_path = '/content/drive/MyDrive/Test CBIS Predict/draw contour'
patch_1024= '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal_each_patch_size/patch 1024'

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path

overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        name_file_saver = os.path.basename(img_file)

        large_image_1 = cv.imread(img_file, 0)
        image_anotation = cv.imread(img_file)
        large_image= np.float32(large_image_1)
        large_image = large_image/255.0

        roi = cv.imread(roi_file,0)
        roi = np.uint8(roi)

        h_r, w_r = roi.shape

        large_image, roi= check_Left_Orient (large_image, roi)

        img = np.array(large_image)

        h_i, w_i = img.shape

        if (h_i>=h_r) or (w_i>=w_r):
          img = img[0:h_r, 0:w_r]
          image_anotation = image_anotation[0:h_r, 0:w_r]
        else:
          roi = roi[0:h_i, 0:w_i]

        # print(img.shape)

        roi_predict_1024 = detect_roi_by_patch_size(img, load_model_file, 1024, 128)
        # plt.title('patch 384')
        # plt.imshow(roi_predict_384, 'gray')
        # plt.show()

        y_pred_filter, n = filter_FP(roi_predict_1024, 2000)
        roi, n_1 = filter_FP(roi, 300)

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_CLOSE, kernel)
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)


        file_patch_1024_path = os.path.join(patch_1024,
                                 ''.join([name_file_saver]))
        plt.imsave(file_patch_1024_path, y_pred_filter, cmap='gray')

        num_P_list.append(n)

        overlap_img, overlap_list, img_anotation_1 = cal_overlaps_per_large_img_1(roi, y_pred_filter, image_anotation)

        area_truth_list.append(overlap_list)

        plt.imshow(img_anotation_1)
        plt.title(name_file_saver)
        plt.axis('off')
        plt.show()

        # img_draw = cv.imread(img_file)
        # img_draw_contour = draw_contour_pred_and_truth(img_draw,y_pred_filter,roi)

        # plt.imshow(img_draw_contour, 'gray')
        # plt.title(name_file_saver)
        # plt.axis('off')
        # plt.show()


In [ ]:
csv_path = '/content/drive/MyDrive/Mammo dataset/Test DDSM/abnormal_image.csv'
# pred_path = '/content/drive/MyDrive/Test CBIS Predict/original_predict'
# overlap_path = '/content/drive/MyDrive/Test CBIS Predict/overlap'
# draw_contour_path = '/content/drive/MyDrive/Test CBIS Predict/draw contour'

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path

overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        name_file_saver = os.path.basename(img_file)

        large_image_1 = cv.imread(img_file, 0)
        image_anotation = cv.imread(img_file)
        large_image= np.float32(large_image_1)
        large_image = large_image/255.0

        roi = cv.imread(roi_file,0)
        roi = np.uint8(roi)

        h_r, w_r = roi.shape

        large_image, roi= check_Left_Orient (large_image, roi)

        img = np.array(large_image)

        h_i, w_i = img.shape

        if (h_i>=h_r) or (w_i>=w_r):
          img = img[0:h_r, 0:w_r]
          image_anotation = image_anotation[0:h_r, 0:w_r]
        else:
          roi = roi[0:h_i, 0:w_i]

        # print(img.shape)

        roi_predict_384 = detect_roi_by_patch_size(img, load_model_file, 348, 64)
        # plt.title('patch 384')
        # plt.imshow(roi_predict_384, 'gray')
        # plt.show()

        y_pred_filter, n = filter_FP(roi_predict_384, 2000)
        roi, n_1 = filter_FP(roi, 300)

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_CLOSE, kernel)
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)

        num_P_list.append(n)

        overlap_img, overlap_list, img_anotation_1 = cal_overlaps_per_large_img_1(roi, y_pred_filter, image_anotation)

        area_truth_list.append(overlap_list)

        plt.imshow(img_anotation_1)
        plt.axis('off')
        plt.show()

        img_draw = cv.imread(img_file)
        img_draw_contour = draw_contour_pred_and_truth(img_draw,y_pred_filter,roi)

        plt.imshow(img_draw_contour, 'gray')
        plt.title(name_file_saver)
        plt.axis('off')
        plt.show()


In [ ]:
data = pd.read_csv(csv_path)
img = data["img"]
roi = data["roi"]
import pandas as pd
save_csv_path= '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal_each_patch_size/patch_384_118_cbis.csv'
df = pd.DataFrame({'img': img, 'roi': roi, 'overlap': area_truth_list, 'num_FP': num_P_list})
df.to_csv (save_csv_path, index = False, header=True)

In [ ]:
csv_path = '/content/drive/MyDrive/Mammo dataset/Test DDSM/abnormal_image.csv'
# pred_path = '/content/drive/MyDrive/Test CBIS Predict/original_predict'
# overlap_path = '/content/drive/MyDrive/Test CBIS Predict/overlap'
# draw_contour_path = '/content/drive/MyDrive/Test CBIS Predict/draw contour'

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path

overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        name_file_saver = os.path.basename(img_file)

        large_image_1 = cv.imread(img_file, 0)
        image_anotation = cv.imread(img_file)
        large_image= np.float32(large_image_1)
        large_image = large_image/255.0

        roi = cv.imread(roi_file,0)
        roi = np.uint8(roi)

        h_r, w_r = roi.shape

        large_image, roi= check_Left_Orient (large_image, roi)

        img = np.array(large_image)

        h_i, w_i = img.shape

        if (h_i>=h_r) or (w_i>=w_r):
          img = img[0:h_r, 0:w_r]
          image_anotation = image_anotation[0:h_r, 0:w_r]
        else:
          roi = roi[0:h_i, 0:w_i]

        # print(img.shape)

        roi_predict_512 = detect_roi_by_patch_size(img, load_model_file, 512, 64)
        # plt.title('patch 384')
        # plt.imshow(roi_predict_384, 'gray')
        # plt.show()

        y_pred_filter, n = filter_FP(roi_predict_512, 2000)
        roi, n_1 = filter_FP(roi, 300)

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_CLOSE, kernel)
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)

        num_P_list.append(n)

        overlap_img, overlap_list, img_anotation_1 = cal_overlaps_per_large_img_1(roi, y_pred_filter, image_anotation)

        area_truth_list.append(overlap_list)

        plt.imshow(img_anotation_1)
        plt.axis('off')
        plt.show()

        img_draw = cv.imread(img_file)
        img_draw_contour = draw_contour_pred_and_truth(img_draw,y_pred_filter,roi)

        plt.imshow(img_draw_contour, 'gray')
        plt.title(name_file_saver)
        plt.axis('off')
        plt.show()


In [ ]:
data = pd.read_csv(csv_path)
img = data["img"]
roi = data["roi"]
import pandas as pd
save_csv_path= '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal_each_patch_size/patch_512_118_cbis.csv'
df = pd.DataFrame({'img': img, 'roi': roi, 'overlap': area_truth_list, 'num_FP': num_P_list})
df.to_csv (save_csv_path, index = False, header=True)

In [ ]:
csv_path = '/content/drive/MyDrive/Mammo dataset/Test DDSM/abnormal_image.csv'
# pred_path = '/content/drive/MyDrive/Test CBIS Predict/original_predict'
# overlap_path = '/content/drive/MyDrive/Test CBIS Predict/overlap'
# draw_contour_path = '/content/drive/MyDrive/Test CBIS Predict/draw contour'

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path

overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        name_file_saver = os.path.basename(img_file)

        large_image_1 = cv.imread(img_file, 0)
        image_anotation = cv.imread(img_file)
        large_image= np.float32(large_image_1)
        large_image = large_image/255.0

        roi = cv.imread(roi_file,0)
        roi = np.uint8(roi)

        h_r, w_r = roi.shape

        large_image, roi= check_Left_Orient (large_image, roi)

        img = np.array(large_image)

        h_i, w_i = img.shape

        if (h_i>=h_r) or (w_i>=w_r):
          img = img[0:h_r, 0:w_r]
          image_anotation = image_anotation[0:h_r, 0:w_r]
        else:
          roi = roi[0:h_i, 0:w_i]

        # print(img.shape)

        roi_predict_1024 = detect_roi_by_patch_size(img, load_model_file, 1024, 128)
        # plt.title('patch 384')
        # plt.imshow(roi_predict_384, 'gray')
        # plt.show()

        y_pred_filter, n = filter_FP(roi_predict_1024, 2000)
        roi, n_1 = filter_FP(roi, 300)

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_CLOSE, kernel)
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)

        num_P_list.append(n)

        overlap_img, overlap_list, img_anotation_1 = cal_overlaps_per_large_img_1(roi, y_pred_filter, image_anotation)

        area_truth_list.append(overlap_list)

        plt.imshow(img_anotation_1)
        plt.axis('off')
        plt.show()

        img_draw = cv.imread(img_file)
        img_draw_contour = draw_contour_pred_and_truth(img_draw,y_pred_filter,roi)

        plt.imshow(img_draw_contour, 'gray')
        plt.title(name_file_saver)
        plt.axis('off')
        plt.show()


In [ ]:
data = pd.read_csv(csv_path)
img = data["img"]
roi = data["roi"]
import pandas as pd
save_csv_path= '/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal_each_patch_size/patch_1024_118_cbis.csv'
df = pd.DataFrame({'img': img, 'roi': roi, 'overlap': area_truth_list, 'num_FP': num_P_list})
df.to_csv (save_csv_path, index = False, header=True)

####Vẽ trên cả 2 mô hình

In [ ]:
def large_image_roi_predict_pred_2nd_overlap(image, roi, pred, pred_2nd):

    _, binarized = cv.threshold(roi, 0, 255, cv.THRESH_BINARY)
    contours_roi, hierarchy = cv.findContours(binarized, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    _, binarized = cv.threshold(pred, 0, 255, cv.THRESH_BINARY)
    contours_pred, hierarchy = cv.findContours(binarized,cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    _, binarized_2nd = cv.threshold(pred_2nd, 0, 255, cv.THRESH_BINARY)
    contours_pred_2nd, hierarchy = cv.findContours(binarized_2nd,cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    img_copy = image.copy()
    cv.drawContours(img_copy, contours_roi,-1,(255,0,0),2)
    cv.drawContours(img_copy, contours_pred,-1, (0,255,0), 2)
    cv.drawContours(img_copy, contours_pred_2nd,-1, (0,0,255), 2)
    plt.imshow(img_copy)
    plt.axis('off')
    plt.title('Image')
    plt.show()
    pred_file_path = os.path.join(save_path,
                                 ''.join([name_file_saver]))
    plt.imsave(pred_file_path, img_copy, cmap='gray')

In [ ]:
image_path = "/content/Abnormal/image/Mass-Training_P_01039_RIGHT_CC.png"
roi_path = "/content/Abnormal/mask/Mass-Training_P_01039_RIGHT_CC.png"
predict_path = "/content/Predict 118 images/standard/original_predict/Mass-Training_P_01039_RIGHT_CC.png"
predict_2nd_path ="/content/Predict 118 images/without 2 skip connection/original_predict/Mass-Training_P_01039_RIGHT_CC.png"

save_path = "/content/drive/MyDrive/Mammo dataset"
name_file_saver = os.path.basename(image_path)
predict = cv.imread(predict_path, 0).astype(np.uint8)
predict = np.array(predict)

predict_2nd = cv.imread(predict_2nd_path, 0).astype(np.uint8)
predict_2nd = np.array(predict_2nd)

image = cv.imread(image_path)

roi = cv.imread(roi_path, 0).astype(np.uint8)
roi = np.array(roi)

large_image_roi_predict_pred_2nd_overlap(image, roi, predict, predict_2nd)


In [ ]:
image_path = "/content/Abnormal/image/Mass-Training_P_01596_RIGHT_MLO.png"
roi_path = "/content/Abnormal/mask/Mass-Training_P_01596_RIGHT_MLO.png"
predict_path = "/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal/original_predict/Mass-Training_P_01596_RIGHT_MLO.png"
predict_2nd_path ="/content/drive/MyDrive/Test DDSM 185 Images/without 2 skip connection/Abnormal/original_predict/Mass-Training_P_01596_RIGHT_MLO.png"

save_path = "/content/drive/MyDrive/Test DDSM 185 Images"
name_file_saver = os.path.basename(image_path)
predict = cv.imread(predict_path, 0).astype(np.uint8)
predict = np.array(predict)

predict_2nd = cv.imread(predict_2nd_path, 0).astype(np.uint8)
predict_2nd = np.array(predict_2nd)

image = cv.imread(image_path)

roi = cv.imread(roi_path, 0).astype(np.uint8)
roi = np.array(roi)

large_image_roi_predict_pred_2nd_overlap(image, roi, predict, predict_2nd)




In [ ]:
image_path = "/content/Abnormal/image/P_01833_RIGHT_MLO.png"
roi_path = "/content/Abnormal/mask/P_01833_RIGHT_MLO.png"
predict_path = "/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal/original_predict/P_01833_RIGHT_MLO.png"
predict_2nd_path ="/content/drive/MyDrive/Test DDSM 185 Images/without 2 skip connection/Abnormal/original_predict/P_01833_RIGHT_MLO.png"

save_path = "/content/drive/MyDrive/Test DDSM 185 Images"
name_file_saver = os.path.basename(image_path)
predict = cv.imread(predict_path, 0).astype(np.uint8)
predict = np.array(predict)

predict_2nd = cv.imread(predict_2nd_path, 0).astype(np.uint8)
predict_2nd = np.array(predict_2nd)

image = cv.imread(image_path)

roi = cv.imread(roi_path, 0).astype(np.uint8)
roi = np.array(roi)

large_image_roi_predict_pred_2nd_overlap(image, roi, predict, predict_2nd)

In [ ]:


image_path = "/content/Abnormal/image/Mass-Training_P_00573_RIGHT_MLO.png"
roi_path = "/content/Abnormal/mask/Mass-Training_P_00573_RIGHT_MLO.png"
predict_path = "/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal/original_predict/Mass-Training_P_00573_RIGHT_MLO.png"
predict_2nd_path ="/content/drive/MyDrive/Test DDSM 185 Images/without 2 skip connection/Abnormal/original_predict/Mass-Training_P_00573_RIGHT_MLO.png"

save_path = "/content/drive/MyDrive/Test DDSM 185 Images"
name_file_saver = os.path.basename(image_path)
predict = cv.imread(predict_path, 0).astype(np.uint8)
predict = np.array(predict)

predict_2nd = cv.imread(predict_2nd_path, 0).astype(np.uint8)
predict_2nd = np.array(predict_2nd)

image = cv.imread(image_path)

roi = cv.imread(roi_path, 0).astype(np.uint8)
roi = np.array(roi)

large_image_roi_predict_pred_2nd_overlap(image, roi, predict, predict_2nd)

In [ ]:

image_path = "/content/Abnormal/image/Mass-Training_P_01152_RIGHT_CC.png"
roi_path = "/content/Abnormal/mask/Mass-Training_P_01152_RIGHT_CC.png"
predict_path = "/content/drive/MyDrive/Test DDSM 185 Images/Standard Model/Abnormal/original_predict/Mass-Training_P_01152_RIGHT_CC.png"
predict_2nd_path ="/content/drive/MyDrive/Test DDSM 185 Images/without 2 skip connection/Abnormal/original_predict/Mass-Training_P_01152_RIGHT_CC.png"

save_path = "/content/drive/MyDrive/Test DDSM 185 Images"
name_file_saver = os.path.basename(image_path)
predict = cv.imread(predict_path, 0).astype(np.uint8)
predict = np.array(predict)

predict_2nd = cv.imread(predict_2nd_path, 0).astype(np.uint8)
predict_2nd = np.array(predict_2nd)

image = cv.imread(image_path)

roi = cv.imread(roi_path, 0).astype(np.uint8)
roi = np.array(roi)

large_image_roi_predict_pred_2nd_overlap(image, roi, predict, predict_2nd)

### CBIS Valid Whole Images - 285

####Standard model

######read data

In [ ]:
!pip install patchify

In [ ]:
!unrar x "/content/drive/MyDrive/CBIS Valid Whole Images 285/CBIS-Validation-Test-Whole-image.rar"

In [ ]:
drive_path = '/content/drive/MyDrive'
dataset_path = '/content'

files_dir = os.path.join(drive_path, 'Colab Notebooks', 'files_15_5_23', 'training_dataset_20_bkg')
model_file = os.path.join(files_dir,'unet_effb0-new-data-20_bkg-15_5_2023.h5')

test_x= sorted(glob(os.path.join(dataset_path, 'Image', '*')))
test_y= sorted(glob(os.path.join(dataset_path, 'Mask', '*')))

In [ ]:
import pandas as pd
df = pd.DataFrame({'img': test_x, 'roi': test_y})
df.to_csv ('/content/drive/MyDrive/CBIS Valid Whole Images 285/CBIS Valid Whole images.csv', index = False, header=True)

In [ ]:
load_model_file = tf.keras.models.load_model(model_file)

In [ ]:
def show_y_predict(randomlist, y, y_pred):
  fg, ax = plt.subplots(2, 5, figsize=(20, 8))
  for i, index in enumerate(randomlist):
    ax[0, i].imshow(np.squeeze(y[index]), 'gray')
    ax[0, i].set_xlabel('y ' + str(index))
    # print(y[index].shape)

  for i, index in enumerate(randomlist):
    ax[1, i].imshow(np.squeeze(y_pred[index]), 'gray')
    ax[1, i].set_xlabel('y_pred ' + str(index))

def prediction(model, large_image, patch_size, step):
  patches = patchify(large_image, (patch_size, patch_size), step=step)  #Step=256 for 256 patches means no overlap

  print(patches.shape)

  redicted_patches = []
  patches_input = []
  for i in range(patches.shape[0]):
    for j in range(patches.shape[1]):
        # print(i,j)
        single_patch = patches[i,j,:,:]
        single_patch_input=np.expand_dims(single_patch, -1)
        # single_patch = np.array(single_patch)
        # print(single_patch.shape)
        #Predict and threshold for values above 0.5 probability

        single_patch_input = np.zeros((patch_size,patch_size,3), dtype="float32")
        single_patch_input[:,:,0] = single_patch
        single_patch_input[:,:,1] = single_patch
        single_patch_input[:,:,2] = single_patch

        single_patch_input = cv.resize(single_patch_input, (256,256), cv.INTER_AREA)
        single_patch_input = np.array(single_patch_input)
        patches_input.append(single_patch_input)

  patches_input = np.array(patches_input)

  patches_prediction = (model.predict(patches_input) >= 0.5).astype(np.uint8)

  # randomlist = random.sample(range(0, len(patches_prediction)),5)
  # show_y_predict(randomlist, patches_input, patches_prediction)
  # plt.show()

  patches_prediction_resize = []
  for i in patches_prediction:
    i = cv.resize(i, (patch_size,patch_size), cv.INTER_AREA)
    patches_prediction_resize.append(i)

  predicted_patches_reshaped = np.reshape(patches_prediction_resize, (patches.shape[0], patches.shape[1], patch_size,patch_size) )
  # print(predicted_patches_reshaped.shape)

  reconstructed_image = unpatchify(predicted_patches_reshaped, large_image.shape)
  plt.imshow(np.squeeze(reconstructed_image), cmap='gray')
  return reconstructed_image

def filter_FP(y_pred, thresh):
  y_pred_filter = y_pred.copy()
  kernel = np.ones((3,3),np.uint8)
  y_pred_filter = cv.erode(y_pred_filter, kernel)

  contours, _ = cv.findContours(y_pred_filter.copy(), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
  n = 0
  for ct in contours:
      contour_area = cv.contourArea(ct)
      if (contour_area>= thresh):
          n += 1
      else:
        y_pred_filter = cv.fillPoly(y_pred_filter, pts=[ct], color=(0, 0))
  y_pred_filter =  cv.dilate(y_pred_filter, kernel)
  return y_pred_filter, n


def draw_contour_pred_and_truth(large_image, pred, roi):
  img_copy = large_image.copy()

  contours_pred, _ = cv.findContours(pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  contours_roi, _ = cv.findContours(roi.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  cv.drawContours(img_copy, contours_roi , -1, (255,0,0), 3)
  cv.drawContours(img_copy, contours_pred, -1, (0,255,0), 3)

  return img_copy

def check_Left_Orient(img, roi):
    left_nonzero = cv.countNonZero(img[:, 0:int(img.shape[1] / 2)])
    right_nonzero = cv.countNonZero(img[:, int(img.shape[1] / 2):])

    if (left_nonzero <= right_nonzero):
            img = cv.flip(img, 1)
            roi = cv.flip(roi, 1)

    return img, roi

def cal_overlap_per_large_img(roi, overlap_img):
  bitwise_and_roi_pred = cv.bitwise_and(overlap_img, roi)
  area_truth = np.count_nonzero(bitwise_and_roi_pred)
  overlap = round(area_truth / np.count_nonzero(roi), 4)*100
  return bitwise_and_roi_pred, area_truth, overlap



In [ ]:
def detect_roi_by_patch_size(img, load_model_file, patch_size, step):

        h,w = img.shape

        roi_detect= np.zeros(img.shape, dtype="uint8")

        row = int(( h - patch_size)/step)
        col = int(( w - patch_size)/step)
        new_img_h = row * step + patch_size
        new_img_w = col * step + patch_size

        img_by_patch_size = img[0:new_img_h, 0: new_img_w]

        segmented_image_by_patch_size = prediction(load_model_file, img_by_patch_size, patch_size , step)
        h_1, w_1 = segmented_image_by_patch_size.shape
        roi_detect[0:h_1, 0:w_1] = segmented_image_by_patch_size


        return roi_detect

In [ ]:
def cal_overlaps_per_large_img(roi, pred, large_image):

  contours_pred, _ = cv.findContours(pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  contours_roi, _ = cv.findContours(roi.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

  img = large_image.copy()

  overlap_list = []

  roi_num_cnt = len(contours_roi)

  if  roi_num_cnt == 1:
       bitwise_and_roi_pred = cv.bitwise_and(pred, roi)
       area_truth = np.count_nonzero(bitwise_and_roi_pred)
       overlap = round(area_truth/np.count_nonzero(roi) * 100,2)
       overlap_list.append(overlap)
       print("overlap", overlap)
       cv.drawContours(img, contours_roi , -1, (255,0,0), 3)
       cv.drawContours(img, contours_pred, -1, (0,255,0), 3)

       if overlap!=0:

           contours_overlaps, _ = cv.findContours(bitwise_and_roi_pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

           c = max(contours_overlaps, key = cv.contourArea)

           x, y, w, h = cv.boundingRect(c)

           font = cv.FONT_HERSHEY_SIMPLEX
           org = (x-10, y-10)
           fontScale = 3
           color = (0, 0, 255)
           thickness = 2
           text = str(overlap)
           img = cv.putText(img, text, org, font, fontScale, color, thickness, cv.LINE_AA)

  else:
       bitwise_and_roi_pred = cv.bitwise_and(pred, roi)
       for ct in contours_roi:
            each_roi= np.zeros(roi.shape, dtype="uint8")
            each_roi = cv.fillPoly(each_roi, pts=[ct], color=(255, 255, 255))
            # plt.imshow(each_roi, 'gray')
            # plt.show()
            cv.drawContours(img, ct , -1, (255,0,0), 3)

            for ct_pred in contours_pred:
                  each_pred = np.zeros(pred.shape, dtype="uint8")
                  each_pred = cv.fillPoly(each_pred, pts=[ct_pred], color=(255, 255, 255))

                  # plt.imshow(each_pred, 'gray')
                  # plt.show()

                  bitwise_and_roi_pred_1 = cv.bitwise_and(each_pred, each_roi)

                  area_truth = np.count_nonzero(bitwise_and_roi_pred_1)
                  overlap = round(area_truth/np.count_nonzero(each_roi) *100,3)
                  cv.drawContours(img, ct_pred, -1, (0,255,0), 3)

                  # cv.drawContours(img, ct_pred , -1, (255,0,0), 3)

                  if overlap>0:
                       overlap_list.append(overlap)
                       print("overlap", overlap)

                       x, y, w, h = cv.boundingRect(ct_pred)

                      #  overlap_img_anotation = cv.fillPoly(overlap_img_anotation, pts=[ct_pred], color=(0, 0))

                      #  overlap_img_anotation = ImageDraw.Draw(overlap_img_anotation)
                      #  overlap_img_anotation.text((x, y), overlap, fill=(0 , 0, 255))
                       font = cv.FONT_HERSHEY_SIMPLEX
                       org = (x-10, y-20)
                       fontScale = 3
                       color = (0, 0, 255)
                       thickness = 2
                       text = str(overlap)
                       img = cv.putText(img, text, org, font, fontScale, color, thickness, cv.LINE_AA)
  # cv.imshow(overlap_img_anotation, 0)
  plt.imshow(bitwise_and_roi_pred, 'gray')
  plt.show()
  # plt.imshow(img)
  # plt.show()
  return bitwise_and_roi_pred, overlap_list, img

In [ ]:
def cal_overlaps_per_large_img_3(roi, pred, large_image):
  roi, n_1 = filter_FP(roi, 300)
  pred, n = filter_FP(pred, 2000)

  contours_pred, _ = cv.findContours(pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  contours_roi, _ = cv.findContours(roi.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

  # pred = cv.fillPoly(pred, pts= contours_pred, color=(255, 255))
  # contours_pred, _ = cv.findContours(pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

  img = large_image.copy()

  overlap_list = []

  roi_num_cnt = len(contours_roi)
  print("num ground truth", roi_num_cnt)
  print("num Predict:", len(contours_pred))

  bitwise_and_roi_pred = cv.bitwise_and(pred, roi)

  cv.drawContours(img, contours_roi , -1, (255,0,0), 3)
  cv.drawContours(img, contours_pred, -1, (0,255,0), 3)

  for ct in contours_roi:
              each_roi= np.zeros(roi.shape, dtype="uint8")
              each_roi = cv.fillPoly(each_roi, pts=[ct], color=(255, 255))

              for ct_pred in contours_pred:
                  each_pred = np.zeros(pred.shape, dtype="uint8")
                  each_pred = cv.fillPoly(each_pred, pts=[ct_pred], color=(255, 255))

                  bitwise_and_roi_pred_1 = cv.bitwise_and(each_pred, each_roi)
                  area_truth = np.count_nonzero(bitwise_and_roi_pred_1)

                  if area_truth > 0:
                       overlap = round(area_truth/np.count_nonzero(each_roi) * 100,3)
                       overlap_list.append(overlap)
                       print("overlap", overlap)

                       x, y, w, h = cv.boundingRect(ct_pred)

                       font = cv.FONT_HERSHEY_SIMPLEX
                       org = (x-10, y-20)
                       fontScale = 3
                       color = (0, 0, 255)
                       thickness = 2
                       text = str(overlap)
                       img = cv.putText(img, text, org, font, fontScale, color, thickness, cv.LINE_AA)

  plt.imshow(bitwise_and_roi_pred, 'gray')
  plt.title('bitwise_and')
  plt.axis('off')
  plt.show()
  return bitwise_and_roi_pred, overlap_list, img, roi_num_cnt

##### predict full image

In [ ]:
import tensorflow as tf

# Tạo RunOptions
options = tf.RunOptions(report_tensor_allocations_upon_oom=True)

# Sử dụng RunOptions khi chạy một session
with tf.Session() as sess:
    sess.run(your_operation, options=options)

In [ ]:
import tensorflow as tf
run_opts = tf.compat.v1.RunOptions(report_tensor_allocations_upon_oom = True)

In [ ]:
csv_path = '/content/drive/MyDrive/CBIS Valid Whole Images 285/CBIS Valid Whole images_1.csv'
pred_path = '/content/drive/MyDrive/CBIS Valid Whole Images 285/Standard model/original_predict'
overlap_path = '/content/drive/MyDrive/CBIS Valid Whole Images 285/Standard model/overlap'
draw_contour_path = '/content/drive/MyDrive/CBIS Valid Whole Images 285/Standard model/draw contour'
overlap_anotation_path = '/content/drive/MyDrive/CBIS Valid Whole Images 285/Standard model/overlap anotation'

patch_384 = '/content/drive/MyDrive/CBIS Valid Whole Images 285/patch 384'
patch_512 = '/content/drive/MyDrive/CBIS Valid Whole Images 285/patch 512'
patch_1024 = '/content/drive/MyDrive/CBIS Valid Whole Images 285/patch 1024'

import time
import torch


from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path

overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
ground_truth = []
num_P_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    start_time = time.time()

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        # path = os.path.dirname(img_file)
        name_file_saver = os.path.basename(img_file)

        large_image_1 = cv.imread(img_file, 0)
        image_anotation = cv.imread(img_file)
        large_image= np.float32(large_image_1)
        large_image = large_image/255.0

        roi = cv.imread(roi_file, 0).astype(np.uint8)

        # roi = cv.imread(roi_file, 0)
        # roi= np.uint8(roi)
        # print(roi.shape)
        h_r, w_r = roi.shape

        large_image, roi= check_Left_Orient (large_image, roi)

        img = np.array(large_image)

        h_i, w_i = img.shape

        # if (h_i>=h_r) or (w_i>=w_r):
        #   img = img[0:h_r, 0:w_r]
        #   image_anotation = image_anotation[0:h_r, 0:w_r]
        # else:
        #   roi = roi[0:h_i, 0:w_i]

        # print(img.shape)

        roi_predict_384 = detect_roi_by_patch_size(img, load_model_file, 348, 64)
        y_pred_filter_384, n_384 = filter_FP(roi_predict_384, 2000)

        pred_file_path_384 = os.path.join(patch_384,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path_384, y_pred_filter_384, cmap='gray')


        roi_predict_512 = detect_roi_by_patch_size(img, load_model_file, 512, 64)

        y_pred_filter_512, n_512 = filter_FP(roi_predict_512, 2000)

        pred_file_path_512 = os.path.join(patch_512,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path_512, y_pred_filter_512, cmap='gray')

        if w_i < 1024:
          roi_predict_1024 = np.zeros(img.shape, dtype="uint8")
        else:
          roi_predict_1024 = detect_roi_by_patch_size(img, load_model_file, 1024, 128)

        y_pred_filter_1024, n_1024 = filter_FP(roi_predict_1024, 2000)

        pred_file_path_1024 = os.path.join(patch_1024,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path_1024, y_pred_filter_1024, cmap='gray')


        # print(roi_predict_384.shape, roi_predict_512.shape, roi_predict_1024.shape)

        combine = np.zeros(roi.shape, dtype=np.uint8)

        combine_1 = np.logical_or(y_pred_filter_384, y_pred_filter_512).astype(np.uint8)
        combine = np.logical_or(combine_1, y_pred_filter_1024).astype(np.uint8)


        y_pred_filter, n = filter_FP(combine, 2000)
        roi, n_1 = filter_FP(roi, 300)

        num_P_list.append(n)

        pred_file_path = os.path.join(pred_path,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path, y_pred_filter, cmap='gray')


        overlap_img, overlap_list, img_anotation_1, num_GT = cal_overlaps_per_large_img_3(roi, y_pred_filter, image_anotation)

        ground_truth.append(num_GT)

        # overlap_img_anotation = 1*overlap_img_anotation
        img_overlap_anotation_path = os.path.join(overlap_anotation_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_overlap_anotation_path, img_anotation_1, cmap='gray')
        plt.title(name_file_saver)
        plt.imshow(img_anotation_1)
        plt.show()
        # print(overlap)
        area_truth_list.append(overlap_list)

        # overlap_img = 1*overlap_img
        file_overlap_img_path = os.path.join(overlap_path,
                                 ''.join([name_file_saver]))
        plt.imsave(file_overlap_img_path, overlap_img, cmap='gray')

        img_draw = cv.imread(img_file)
        # combine_predict = cv.imread(pred_file_path, 0)
        img_draw_contour = draw_contour_pred_and_truth(img_draw, y_pred_filter, roi)

        # plt.imshow(img_draw_contour)

        # plt.title(name_file_saver + "- overlap = "+ str(overlap))
        # plt.show()
        img_draw_contour_path = os.path.join(draw_contour_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_draw_contour_path, img_draw_contour, cmap='gray')
        torch.cuda.empty_cache()



In [ ]:
csv_path = '/content/drive/MyDrive/CBIS Valid Whole Images 285/CBIS Valid Whole images_1.csv'
pred_path = '/content/drive/MyDrive/CBIS Valid Whole Images 285/Standard model/original_predict'
overlap_path = '/content/drive/MyDrive/CBIS Valid Whole Images 285/Standard model/overlap'
draw_contour_path = '/content/drive/MyDrive/CBIS Valid Whole Images 285/Standard model/draw contour'
overlap_anotation_path = '/content/drive/MyDrive/CBIS Valid Whole Images 285/Standard model/overlap anotation'

patch_384 = '/content/drive/MyDrive/CBIS Valid Whole Images 285/patch 384'
patch_512 = '/content/drive/MyDrive/CBIS Valid Whole Images 285/patch 512'
patch_1024 = '/content/drive/MyDrive/CBIS Valid Whole Images 285/patch 1024'

import time
import torch


from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path

overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
ground_truth = []
num_P_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    start_time = time.time()

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        # path = os.path.dirname(img_file)
        name_file_saver = os.path.basename(img_file)

        large_image_1 = cv.imread(img_file, 0)
        image_anotation = cv.imread(img_file)
        large_image= np.float32(large_image_1)
        large_image = large_image/255.0

        roi = cv.imread(roi_file, 0).astype(np.uint8)

        # roi = cv.imread(roi_file, 0)
        # roi= np.uint8(roi)
        # print(roi.shape)
        h_r, w_r = roi.shape

        large_image, roi= check_Left_Orient (large_image, roi)

        img = np.array(large_image)

        h_i, w_i = img.shape

        # if (h_i>=h_r) or (w_i>=w_r):
        #   img = img[0:h_r, 0:w_r]
        #   image_anotation = image_anotation[0:h_r, 0:w_r]
        # else:
        #   roi = roi[0:h_i, 0:w_i]

        # print(img.shape)

        roi_predict_384 = detect_roi_by_patch_size(img, load_model_file, 348, 64)
        y_pred_filter_384, n_384 = filter_FP(roi_predict_384, 2000)

        pred_file_path_384 = os.path.join(patch_384,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path_384, y_pred_filter_384, cmap='gray')


        roi_predict_512 = detect_roi_by_patch_size(img, load_model_file, 512, 64)

        y_pred_filter_512, n_512 = filter_FP(roi_predict_512, 2000)

        pred_file_path_512 = os.path.join(patch_512,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path_512, y_pred_filter_512, cmap='gray')

        if w_i < 1024:
          roi_predict_1024 = np.zeros(img.shape, dtype="uint8")
        else:
          roi_predict_1024 = detect_roi_by_patch_size(img, load_model_file, 1024, 128)

        y_pred_filter_1024, n_1024 = filter_FP(roi_predict_1024, 2000)

        pred_file_path_1024 = os.path.join(patch_1024,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path_1024, y_pred_filter_1024, cmap='gray')


        # print(roi_predict_384.shape, roi_predict_512.shape, roi_predict_1024.shape)

        combine = np.zeros(roi.shape, dtype=np.uint8)

        combine_1 = np.logical_or(y_pred_filter_384, y_pred_filter_512).astype(np.uint8)
        combine = np.logical_or(combine_1, y_pred_filter_1024).astype(np.uint8)


        y_pred_filter, n = filter_FP(combine, 2000)
        roi, n_1 = filter_FP(roi, 300)

        num_P_list.append(n)

        pred_file_path = os.path.join(pred_path,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path, y_pred_filter, cmap='gray')


        overlap_img, overlap_list, img_anotation_1, num_GT = cal_overlaps_per_large_img_3(roi, y_pred_filter, image_anotation)

        ground_truth.append(num_GT)

        # overlap_img_anotation = 1*overlap_img_anotation
        img_overlap_anotation_path = os.path.join(overlap_anotation_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_overlap_anotation_path, img_anotation_1, cmap='gray')
        plt.title(name_file_saver)
        plt.imshow(img_anotation_1)
        plt.show()
        # print(overlap)
        area_truth_list.append(overlap_list)

        # overlap_img = 1*overlap_img
        file_overlap_img_path = os.path.join(overlap_path,
                                 ''.join([name_file_saver]))
        plt.imsave(file_overlap_img_path, overlap_img, cmap='gray')

        img_draw = cv.imread(img_file)
        # combine_predict = cv.imread(pred_file_path, 0)
        img_draw_contour = draw_contour_pred_and_truth(img_draw, y_pred_filter, roi)

        # plt.imshow(img_draw_contour)

        # plt.title(name_file_saver + "- overlap = "+ str(overlap))
        # plt.show()
        img_draw_contour_path = os.path.join(draw_contour_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_draw_contour_path, img_draw_contour, cmap='gray')
        torch.cuda.empty_cache()


In [ ]:
csv_path = '/content/drive/MyDrive/CBIS Valid Whole Images 285/CBIS Valid Whole images_1.csv'
pred_path = '/content/drive/MyDrive/CBIS Valid Whole Images 285/Standard model/original_predict'
overlap_path = '/content/drive/MyDrive/CBIS Valid Whole Images 285/Standard model/overlap'
draw_contour_path = '/content/drive/MyDrive/CBIS Valid Whole Images 285/Standard model/draw contour'
overlap_anotation_path = '/content/drive/MyDrive/CBIS Valid Whole Images 285/Standard model/overlap anotation'

patch_384 = '/content/drive/MyDrive/CBIS Valid Whole Images 285/patch 384'
patch_512 = '/content/drive/MyDrive/CBIS Valid Whole Images 285/patch 512'
patch_1024 = '/content/drive/MyDrive/CBIS Valid Whole Images 285/patch 1024'

import time
import torch


from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path

overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
ground_truth = []
num_P_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    start_time = time.time()

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        # path = os.path.dirname(img_file)
        name_file_saver = os.path.basename(img_file)

        large_image_1 = cv.imread(img_file, 0)
        image_anotation = cv.imread(img_file)
        large_image= np.float32(large_image_1)
        large_image = large_image/255.0

        roi = cv.imread(roi_file, 0).astype(np.uint8)

        # roi = cv.imread(roi_file, 0)
        # roi= np.uint8(roi)
        # print(roi.shape)
        h_r, w_r = roi.shape

        large_image, roi= check_Left_Orient (large_image, roi)

        img = np.array(large_image)

        h_i, w_i = img.shape

        # if (h_i>=h_r) or (w_i>=w_r):
        #   img = img[0:h_r, 0:w_r]
        #   image_anotation = image_anotation[0:h_r, 0:w_r]
        # else:
        #   roi = roi[0:h_i, 0:w_i]

        # print(img.shape)

        roi_predict_384 = detect_roi_by_patch_size(img, load_model_file, 348, 64)
        y_pred_filter_384, n_384 = filter_FP(roi_predict_384, 2000)

        pred_file_path_384 = os.path.join(patch_384,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path_384, y_pred_filter_384, cmap='gray')


        roi_predict_512 = detect_roi_by_patch_size(img, load_model_file, 512, 64)

        y_pred_filter_512, n_512 = filter_FP(roi_predict_512, 2000)

        pred_file_path_512 = os.path.join(patch_512,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path_512, y_pred_filter_512, cmap='gray')

        if w_i < 1024:
          roi_predict_1024 = np.zeros(img.shape, dtype="uint8")
        else:
          roi_predict_1024 = detect_roi_by_patch_size(img, load_model_file, 1024, 128)

        y_pred_filter_1024, n_1024 = filter_FP(roi_predict_1024, 2000)

        pred_file_path_1024 = os.path.join(patch_1024,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path_1024, y_pred_filter_1024, cmap='gray')


        # print(roi_predict_384.shape, roi_predict_512.shape, roi_predict_1024.shape)

        combine = np.zeros(roi.shape, dtype=np.uint8)

        combine_1 = np.logical_or(y_pred_filter_384, y_pred_filter_512).astype(np.uint8)
        combine = np.logical_or(combine_1, y_pred_filter_1024).astype(np.uint8)


        y_pred_filter, n = filter_FP(combine, 2000)
        roi, n_1 = filter_FP(roi, 300)

        num_P_list.append(n)

        pred_file_path = os.path.join(pred_path,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path, y_pred_filter, cmap='gray')


        overlap_img, overlap_list, img_anotation_1, num_GT = cal_overlaps_per_large_img_3(roi, y_pred_filter, image_anotation)

        ground_truth.append(num_GT)

        # overlap_img_anotation = 1*overlap_img_anotation
        img_overlap_anotation_path = os.path.join(overlap_anotation_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_overlap_anotation_path, img_anotation_1, cmap='gray')
        plt.title(name_file_saver)
        plt.imshow(img_anotation_1)
        plt.show()
        # print(overlap)
        area_truth_list.append(overlap_list)

        # overlap_img = 1*overlap_img
        file_overlap_img_path = os.path.join(overlap_path,
                                 ''.join([name_file_saver]))
        plt.imsave(file_overlap_img_path, overlap_img, cmap='gray')

        img_draw = cv.imread(img_file)
        # combine_predict = cv.imread(pred_file_path, 0)
        img_draw_contour = draw_contour_pred_and_truth(img_draw, y_pred_filter, roi)

        # plt.imshow(img_draw_contour)

        # plt.title(name_file_saver + "- overlap = "+ str(overlap))
        # plt.show()
        img_draw_contour_path = os.path.join(draw_contour_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_draw_contour_path, img_draw_contour, cmap='gray')
        torch.cuda.empty_cache()


In [ ]:
csv_path = '/content/drive/MyDrive/CBIS Valid Whole Images 285/CBIS Valid Whole images_1.csv'
pred_path = '/content/drive/MyDrive/CBIS Valid Whole Images 285/Standard model/original_predict'
overlap_path = '/content/drive/MyDrive/CBIS Valid Whole Images 285/Standard model/overlap'
draw_contour_path = '/content/drive/MyDrive/CBIS Valid Whole Images 285/Standard model/draw contour'
overlap_anotation_path = '/content/drive/MyDrive/CBIS Valid Whole Images 285/Standard model/overlap anotation'

patch_384 = '/content/drive/MyDrive/CBIS Valid Whole Images 285/patch 384'
patch_512 = '/content/drive/MyDrive/CBIS Valid Whole Images 285/patch 512'
patch_1024 = '/content/drive/MyDrive/CBIS Valid Whole Images 285/patch 1024'

import time
import torch


from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path

overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
ground_truth = []
num_P_list = []
with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)

    start_time = time.time()

    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        # path = os.path.dirname(img_file)
        name_file_saver = os.path.basename(img_file)

        large_image_1 = cv.imread(img_file, 0)
        image_anotation = cv.imread(img_file)
        large_image= np.float32(large_image_1)
        large_image = large_image/255.0

        roi = cv.imread(roi_file, 0).astype(np.uint8)

        # roi = cv.imread(roi_file, 0)
        # roi= np.uint8(roi)
        # print(roi.shape)
        h_r, w_r = roi.shape

        large_image, roi= check_Left_Orient (large_image, roi)

        img = np.array(large_image)

        h_i, w_i = img.shape

        # if (h_i>=h_r) or (w_i>=w_r):
        #   img = img[0:h_r, 0:w_r]
        #   image_anotation = image_anotation[0:h_r, 0:w_r]
        # else:
        #   roi = roi[0:h_i, 0:w_i]

        # print(img.shape)

        roi_predict_384 = detect_roi_by_patch_size(img, load_model_file, 348, 64)
        y_pred_filter_384, n_384 = filter_FP(roi_predict_384, 2000)

        pred_file_path_384 = os.path.join(patch_384,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path_384, y_pred_filter_384, cmap='gray')


        roi_predict_512 = detect_roi_by_patch_size(img, load_model_file, 512, 64)

        y_pred_filter_512, n_512 = filter_FP(roi_predict_512, 2000)

        pred_file_path_512 = os.path.join(patch_512,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path_512, y_pred_filter_512, cmap='gray')

        if w_i < 1024:
          roi_predict_1024 = np.zeros(img.shape, dtype="uint8")
        else:
          roi_predict_1024 = detect_roi_by_patch_size(img, load_model_file, 1024, 128)

        y_pred_filter_1024, n_1024 = filter_FP(roi_predict_1024, 2000)

        pred_file_path_1024 = os.path.join(patch_1024,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path_1024, y_pred_filter_1024, cmap='gray')


        # print(roi_predict_384.shape, roi_predict_512.shape, roi_predict_1024.shape)

        combine = np.zeros(roi.shape, dtype=np.uint8)

        combine_1 = np.logical_or(y_pred_filter_384, y_pred_filter_512).astype(np.uint8)
        combine = np.logical_or(combine_1, y_pred_filter_1024).astype(np.uint8)


        y_pred_filter, n = filter_FP(combine, 2000)
        roi, n_1 = filter_FP(roi, 300)

        num_P_list.append(n)

        pred_file_path = os.path.join(pred_path,
                                 ''.join([name_file_saver]))
        plt.imsave(pred_file_path, y_pred_filter, cmap='gray')


        overlap_img, overlap_list, img_anotation_1, num_GT = cal_overlaps_per_large_img_3(roi, y_pred_filter, image_anotation)

        ground_truth.append(num_GT)

        # overlap_img_anotation = 1*overlap_img_anotation
        img_overlap_anotation_path = os.path.join(overlap_anotation_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_overlap_anotation_path, img_anotation_1, cmap='gray')
        plt.title(name_file_saver)
        plt.imshow(img_anotation_1)
        plt.show()
        # print(overlap)
        area_truth_list.append(overlap_list)

        # overlap_img = 1*overlap_img
        file_overlap_img_path = os.path.join(overlap_path,
                                 ''.join([name_file_saver]))
        plt.imsave(file_overlap_img_path, overlap_img, cmap='gray')

        img_draw = cv.imread(img_file)
        # combine_predict = cv.imread(pred_file_path, 0)
        img_draw_contour = draw_contour_pred_and_truth(img_draw, y_pred_filter, roi)

        # plt.imshow(img_draw_contour)

        # plt.title(name_file_saver + "- overlap = "+ str(overlap))
        # plt.show()
        img_draw_contour_path = os.path.join(draw_contour_path,
                                 ''.join([name_file_saver]))
        plt.imsave(img_draw_contour_path, img_draw_contour, cmap='gray')
        torch.cuda.empty_cache()


###Test VinDr

###Test Ultrasound BUSI

In [ ]:
!pip install patchify

In [ ]:
!unrar x "/content/drive/MyDrive/Test-Ultrasound-Busi/data_benign.rar"

In [ ]:
drive_path = '/content/drive/MyDrive'
dataset_path = '/content'

files_dir = os.path.join(drive_path, 'Colab Notebooks', 'files_15_5_23', 'training_dataset_20_bkg')
model_file = os.path.join(files_dir,'unet_effb0-new-data-20_bkg-15_5_2023.h5')

test_x= sorted(glob(os.path.join(dataset_path, 'data_benign', 'img', '*')))
test_y= sorted(glob(os.path.join(dataset_path, 'data_benign', 'mask', '*')))

print(len(test_x), len(test_y))

In [ ]:
import pandas as pd
df = pd.DataFrame({'img': test_x, 'roi': test_y})
df.to_csv ('/content/drive/MyDrive/Test-Ultrasound-Busi/data_benign.csv', index = False, header=True)

In [ ]:
def show_y_predict(randomlist, y, y_pred):
  fg, ax = plt.subplots(2, 5, figsize=(20, 8))
  for i, index in enumerate(randomlist):
    ax[0, i].imshow(np.squeeze(y[index]), 'gray')
    ax[0, i].set_xlabel('y ' + str(index))
    # print(y[index].shape)

  for i, index in enumerate(randomlist):
    ax[1, i].imshow(np.squeeze(y_pred[index]), 'gray')
    ax[1, i].set_xlabel('y_pred ' + str(index))


def filter_FP(y_pred, thresh):
  y_pred_filter = y_pred.copy()
  kernel = np.ones((3,3),np.uint8)
  y_pred_filter = cv.erode(y_pred_filter, kernel)

  contours, _ = cv.findContours(y_pred_filter.copy(), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
  n = 0
  for ct in contours:
      contour_area = cv.contourArea(ct)
      if (contour_area>= thresh):
          n += 1
      else:
        y_pred_filter = cv.fillPoly(y_pred_filter, pts=[ct], color=(0, 0))
  y_pred_filter =  cv.dilate(y_pred_filter, kernel)
  return y_pred_filter, n


def draw_contour_pred_and_truth(large_image, pred, roi):
  img_copy = large_image.copy()

  contours_pred, _ = cv.findContours(pred.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  contours_roi, _ = cv.findContours(roi.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  cv.drawContours(img_copy, contours_roi , -1, (255,0,0), 3)
  cv.drawContours(img_copy, contours_pred, -1, (0,255,0), 3)

  return img_copy

def cal_overlap_per_large_img(roi, overlap_img):
  bitwise_and_roi_pred = cv.bitwise_and(overlap_img, roi)
  area_truth = np.count_nonzero(bitwise_and_roi_pred)
  overlap = round(area_truth / np.count_nonzero(roi), 4)*100
  return bitwise_and_roi_pred, area_truth, overlap


In [ ]:
model = tf.keras.models.load_model(model_file)

In [ ]:
csv_path = '/content/drive/MyDrive/Test-Ultrasound-Busi/data_benign.csv'

data = pd.read_csv(csv_path)
img_file = data['img']
roi_file = data['roi']

In [ ]:
def read_image(path):
  path = path.decode()
  x = cv.imread(path).astype(np.float32)
  x = x/255.0
  return x
def read_mask(path):
  path = path.decode()
  x = cv.imread(path, 0).astype(np.bool_)
  # x = x/255.0
  x = np.expand_dims(x, axis = -1)
  return x

In [ ]:
def load_data(path):
  test_x = sorted(glob(os.path.join(path, 'data_benign', 'img', '*')))
  test_y = sorted(glob(os.path.join(path, 'data_benign', 'mask', '*')))
  return test_x, test_y

In [ ]:
dataset_path = '/content'
test_x, test_y = load_data(dataset_path)
print(len(test_x), len(test_y))

In [ ]:
IMG_HEIGHT=256
IMG_WIDTH=256
IMG_CHANNELS=3

X_test = np.ndarray(shape = (len(test_x), IMG_HEIGHT, IMG_WIDTH, 3), dtype=np.float32)

for n, file_path in enumerate(test_x):
        img = cv.imread(file_path).astype(np.float32)
        # img = img/255.0
        img = cv.resize(img, (256,256), cv.INTER_AREA)
        img = np.array(img)
        # img = cv.bitwise_not(img)
        img = 255 - img
        X_test[n] = img


In [ ]:
img = cv.imread(test_x[2], 0).astype(np.float32)
img = img/255.0
img = cv.resize(img, (256,256), cv.INTER_AREA)
# img = (cv.bitwise_not(img)).astype(np.float32)
img = (255-img)
plt.imshow(img, 'gray')
plt.show()


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(X_test)
# dataset = dataset.map(tf_parse, num_parallel_calls= tf.data.AUTOTUNE)
dataset = dataset.batch(8)

In [ ]:
mask_predict = (model.predict(dataset) >= 0.5).astype(np.uint8)

In [ ]:
def show_image_roi_predict(randomlist, x, predict, name):
  fg, ax = plt.subplots(2, 5, figsize=(20, 8))
  fg.suptitle(name)

  for i, index in enumerate(randomlist):
    ax[0, i].imshow((x[index]).astype('uint8'), 'gray')
    ax[0, i].set_xlabel('Image ' + str(index))

  for i, index in enumerate(randomlist):
    ax[1, i].imshow((np.squeeze(predict[index]).astype('uint8')), 'gray')
    ax[1, i].set_xlabel('Predict ' + str(index))

In [ ]:
randomlist = random.sample(range(0, 8),5)
print(randomlist)
show_image_roi_predict(randomlist, X_test, mask_predict, 'Test set')

In [ ]:
csv_path = '/content/drive/MyDrive/Test-Ultrasound-Busi/data_benign.csv'
pred_path = '/content/drive/MyDrive/Test-Ultrasound-Busi/data_benign_predict/original_predict'
overlap_path = '/content/drive/MyDrive/Test-Ultrasound-Busi/data_benign_predict/overlap'
draw_contour_path = '/content/drive/MyDrive/Test-Ultrasound-Busi/data_benign_predict/draw contour'

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path

from patchify import patchify,  unpatchify
from csv import DictReader
from pathlib import Path

overlap_list = []
area_truth_list = []
num_FP = 0
name_pred = []
num_P_list = []


with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)
    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        name_file = os.path.basename(img_file)
        print(name_file)

        large_image = cv.imread(img_file, 0).astype(np.float32)
        large_image = large_image/255.0
        large_image = cv.bitwise_not(large_image)

        roi = cv.imread(roi_file, 0).astype(np.uint8)

        img = np.array(large_image)

        h,w= img.shape

        img_1 = cv.resize(img, (256,256), cv.INTER_AREA)

        # img_1 = np.array(img_1)
        # plt.imshow(img_1, 'gray')
        # plt.show()

        single_patch_input = np.zeros((256,256,3), dtype="float32")
        single_patch_input[:,:,0] = img_1
        single_patch_input[:,:,1] = img_1
        single_patch_input[:,:,2] = img_1

        # single_patch_input = cv.resize(single_patch_input, (256,256), cv.INTER_AREA)
        single_patch_input = np.array(single_patch_input)

        mask_predict = (model.predict(single_patch_input) >= 0.5).astype(np.uint8)

        mask_predict = cv.resize(img, (h,w), cv.INTER_AREA)

        plt.imshow(np.squeeze(mask_predict), cmap='gray')

        pred_img = mask_predict

        y_pred_filter, n = filter_FP(pred_img, 1000 )

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_CLOSE, kernel)
        y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)

        num_FP = num_FP + n
        num_P_list.append(n)
        print("number FP  = ", n )

        pred_file_path = os.path.join(pred_path,
                                 ''.join([name_file_saver]))
        # plt.imsave(pred_file_path, y_pred_filter, cmap='gray')

        overlap_img, area_truth, overlap = cal_overlap_per_large_img(roi, y_pred_filter)
        print(overlap)
        area_truth_list.append(area_truth)
        overlap_list.append(overlap)
        overlap_img = 1*overlap_img
        file_overlap_img_path = os.path.join(overlap_path,
                                 ''.join([name_file_saver]))
        # plt.imsave(file_overlap_img_path, overlap_img, cmap='gray')

        img_draw = cv.imread(img_file)
        img_draw_contour = draw_contour_pred_and_truth(img_draw,y_pred_filter,roi)

        plt.imshow(img_draw_contour)
        plt.title(name_file_saver + "- overlap = "+ str(overlap) + ", Roi = " + str(n))
        plt.show()
        img_draw_contour_path = os.path.join(draw_contour_path,
                                 ''.join([name_file_saver]))
        # plt.imsave(img_draw_contour_path, img_draw_contour, cmap='gray')


### dice loss

In [ ]:
callbacks = [
    ModelCheckpoint(model_file, verbose = 1, save_best_only= True),
    ReduceLROnPlateau(monitor='val_dice_coef', factor = 0.1, patience = 4, mode='min'),
    CSVLogger(log_file),
    EarlyStopping(monitor = 'val_dice_coef', patience = 20, restore_best_weights = False)
]

In [ ]:
from keras.losses import binary_crossentropy
from tensorflow.keras.metrics import Precision, Recall, MeanIoU

model.fit(
    train_dataset,
    validation_data= valid_dataset,
    epochs= epochs,
    callbacks= callbacks
)

In [ ]:
import numpy as np
from numpy.testing import assert_allclose
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.callbacks import ModelCheckpoint

# load the model
new_model = load_model(model_file
                       )

log_file_1 = os.path.join(files_dir,'log_new-data-bkg_5-5_vs2_.csv')

callbacks = [
    ModelCheckpoint(model_file, verbose = 1, save_best_only= True),
    ReduceLROnPlateau(monitor='val_dice_coef', factor = 0.1, patience = 4, mode='min'),
    CSVLogger(log_file_1),
    EarlyStopping(monitor = 'val_dice_coef', patience = 20, restore_best_weights = False)
]



In [ ]:
# fit the model
new_model.fit(
    train_dataset,
    validation_data= valid_dataset,
    epochs= epochs,
    callbacks= callbacks
)

### Test INBreast (bb<850) patch (1416 mass patch

In [ ]:
!unrar x "//content/drive/MyDrive/test INBreast patch/INBreast roi 850.rar"

In [ ]:
drive_path = '/content/drive/MyDrive'
dataset_path = '/content'
dataset_inbreast_path = '/content/INBreast roi 850'
files_dir = os.path.join(drive_path, 'Colab Notebooks', 'files_15_5_23', 'training_dataset_20_bkg')
model_file = os.path.join(files_dir,'unet_effb0-new-data-20_bkg-15_5_2023.h5')

test_x= sorted(glob(os.path.join(dataset_inbreast_path, 'Extract Patches', 'Patch', '*')))
test_y= sorted(glob(os.path.join(dataset_inbreast_path, 'Extract Patches', 'Mask', '*')))



In [ ]:
import pandas as pd
df = pd.DataFrame({'img': test_x, 'roi': test_y})
df.to_csv ('/content/drive/MyDrive/test INBreast patch/patch_INBreast.csv', index = False, header=True)

In [ ]:
def create_h5_file1(csv_file, h5_path, filename_x, filename_y):
    col_list = [filename_x, filename_y]
    df = pd.read_csv(csv_file, usecols=col_list)

    x_train = df[filename_x]
    y_train = df[filename_y]

    # x_train.sort()
    # y_train.sort()

    IMG_HEIGHT=256
    IMG_WIDTH=256
    IMG_CHANNELS=3


    # X_train = np.zeros((len(x_train), IMG_HEIGHT, IMG_WIDTH, 3), dtype= np.uint8)
    X_train = np.ndarray(shape = (len(x_train), IMG_HEIGHT, IMG_WIDTH, 3), dtype=np.float32)
    Y_train = np.ndarray(shape = (len(y_train), IMG_HEIGHT, IMG_WIDTH, 1), dtype= np.bool_)

    x_train.sort_values()
    y_train.sort_values()

    for n, file_path in enumerate(x_train):
        img = cv.imread(file_path)
        img = np.array(img)
        img = img / 255.
        # print(file_path)
        X_train[n] = img
    #
    for n, file_path in enumerate(y_train):
        roi = cv.imread(file_path, 0)
        roi = np.array(roi)
        # roi = roi / 255.
        # print(file_path)
        roi = np.expand_dims(roi, -1)
        Y_train[n] = roi

    print("number of X examples = " + str(X_train.shape[0]))
    print("X shape: " + str(X_train.shape))
    print("Y shape: " + str(Y_train.shape))
    # print('pixel values in the mask are: ', np.unique(Y_train))

    # return X_train, Y_train

    with h5py.File(h5_path, 'w') as hf:
        dset_x_train = hf.create_dataset('x', data=X_train, shape=(X_train.shape), compression='gzip',
                                         chunks=True)
        dset_y_train = hf.create_dataset('y', data=Y_train, shape=(Y_train.shape), compression='gzip',
                                         chunks=True)
    hf.close()
    print('Done')

In [ ]:
create_h5_file1('/content/drive/MyDrive/test INBreast patch/patch_INBreast.csv', '/content/drive/MyDrive/test INBreast patch/patch_mask_INBreast.h5','img', 'roi')

In [ ]:
def read_h5_file(h5_path):
    test_dataset = h5py.File(h5_path, "r")
    test_set_x_orig = np.array(test_dataset["x"][:])  # your train set features
    test_set_y_orig = np.array(test_dataset["y"][:])

    print("number of training examples = " + str(test_set_x_orig.shape[0]))
    print("X_INBreast shape: " + str(test_set_x_orig.shape))
    print("Y_INBreast shape: " + str(test_set_y_orig.shape))

    return test_set_x_orig, test_set_y_orig

In [ ]:
inbreast_set_x, inbreast_set_y = read_h5_file('/content/drive/MyDrive/test INBreast patch/patch_mask_INBreast.h5')

In [ ]:
load_model_file = tf.keras.models.load_model(model_file)

In [ ]:
preds_inbreast = load_model_file.predict(inbreast_set_x, verbose=1)
inbreast_y_pred = (preds_inbreast > 0.5).astype(np.float32)

In [ ]:
from csv import DictReader
csv_path = '/content/drive/MyDrive/test INBreast patch/patch_INBreast.csv'
predict_path = "/content/drive/MyDrive/test INBreast patch/Predict_standard_model"
data = pd.read_csv(csv_path)
test_set_x_orig = data['img']

# print(test_set_x_orig.shape)
n = 0
for i in test_set_x_orig:
      name_file = os.path.basename(i)
      # print(name_file)
      save_path = os.path.join(predict_path,''.join([name_file]))
      # print(save_path)
      predict = np.squeeze(inbreast_y_pred[n])
      plt.imsave(save_path, predict, cmap='gray')
      # plt.imshow(predict)
      # plt.show
      # cv.imwrite(save_path, predict)
      n = n + 1
print(n)
print("Done")

In [ ]:
def show_image_roi_predict1_img(index, x, y, predict):

    kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
    y_pred_filter = cv.morphologyEx(np.squeeze(predict[index]), cv.MORPH_CLOSE, kernel)
    y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)

    overlap = cal_overlap(x[index], y[index], y_pred_filter)
    print (overlap)

    roi = np.squeeze(y[index])
    plt.subplot(131)
    plt.axis('off')
    plt.imshow(roi, 'gray')
    plt.title('ROI ' + str(index))
    image_8bit = np.uint8(roi * 255)
    _, binarized = cv.threshold(image_8bit, 0, 255, cv.THRESH_BINARY)
    contours_roi, hierarchy = cv.findContours(binarized, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)


    roi_pred = np.squeeze(y_pred_filter)
    plt.subplot(132)
    plt.axis('off')
    plt.imshow(roi_pred)
    plt.title('ROI pred ' + str(index))
    image_8bit = np.uint8(roi_pred * 255)
    _, binarized = cv.threshold(image_8bit, 0, 255, cv.THRESH_BINARY)
    contours_pred, hierarchy = cv.findContours(binarized,cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    img = x[index]
    img_copy = img.copy()
    cv.drawContours(img_copy, contours_roi,-1,(255,0,0),2)
    cv.drawContours(img_copy, contours_pred,-1, (0,255,0), 2)
    plt.subplot(133)
    plt.imshow(img_copy)
    plt.title('Image' + str(index))
    plt.axis('off')
    plt.show()


In [ ]:
def cal_overlap(x,y,y_pred):
  bitwise_and_roi_patch = np.logical_and(np.squeeze(y), np.squeeze(y_pred))
  overlap = round(np.sum(bitwise_and_roi_patch) / np.sum(np.squeeze(y)), 2)
  return overlap

In [ ]:
show_image_roi_predict1_img(42, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(100, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(500, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(200, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(5, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(60, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
def show_image_roi_predict(randomlist, x, y, predict, name):
  fg, ax = plt.subplots(3, 5, figsize=(20, 8))
  fg.suptitle(name)

  for i, index in enumerate(randomlist):
    ax[0, i].imshow(x[index], 'gray')
    ax[0, i].set_xlabel('Image ' + str(index))

  for i, index in enumerate(randomlist):
    ax[1, i].imshow(np.squeeze(y[index]), 'gray')
    ax[1, i].set_xlabel('ROI ' + str(index))

  for i, index in enumerate(randomlist):
    ax[2, i].imshow(np.squeeze(predict[index]), 'gray')
    ax[2, i].set_xlabel('Predict ' + str(index))

In [ ]:
randomlist = random.sample(range(0, 100),5)
print(randomlist)
show_image_roi_predict(randomlist, inbreast_set_x, inbreast_set_y, inbreast_y_pred, 'inbreast')

#### metrics - chưa hậu xử lý predict

In [ ]:
from csv import DictReader
from csv import DictReader
from pathlib import Path

csv_path = '/content/drive/MyDrive/test INBreast patch/patch_INBreast.csv'
predict_path = "/content/drive/MyDrive/test INBreast patch/Predict_standard_model"
data = pd.read_csv(csv_path)

dice_list = []
iou_list = []
overlap_list = []

for i in range(len(inbreast_set_x)):
        roi = np.squeeze(inbreast_set_x[i])
        print(np.shape(inbreast_set_x[i]))
        pred = np.squeeze(inbreast_y_pred[i])
        print(np.shape(inbreast_y_pred[i]))

        # roi_bw = cv.threshold(roi, 1 , 255, cv.THRESH_BINARY)
        # roi = np.array(roi_bw)

        # kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
        # pred = cv.morphologyEx(pred, cv.MORPH_CLOSE, kernel)
        # pred = cv.morphologyEx(pred, cv.MORPH_OPEN, kernel)
        # pred_bw = cv.threshold(pred, 1 , 255, cv.THRESH_BINARY)
        # pred1 = np.array(pred_bw)

        dice = compute_dice_1(roi, pred)
        dice_list.append(dice)

        # bitwise_and_roi_patch = np.logical_and(np.squeeze(roi), np.squeeze(pred))
        # overlap = round(np.sum(bitwise_and_roi_patch) / np.sum(np.squeeze(roi)), 3)
        overlap = compute_overlap(roi, pred)
        overlap_list.append(overlap)

        iou = compute_iou(roi, pred)
        iou_list.append(iou)


In [ ]:
csv_path = '/content/drive/MyDrive/test INBreast patch/patch_INBreast.csv'
predict_path = "/content/drive/MyDrive/test INBreast patch/Predict_standard_model"
data = pd.read_csv(csv_path)
inbreast_set_y_orig = data['roi']

# print(test_set_x_orig.shape)
dice_list = []
iou_list = []
overlap_list = []

for i in inbreast_set_y_orig:
      name_file = os.path.basename(i)
      roi = cv.imread(i)
      # roi = np.array(roi)
      pred_path = os.path.join(predict_path,''.join([name_file]))
      pred = cv.imread(pred_path)
      # pred = np.array(pred)

      dice = compute_dice_1(roi, pred)
      dice_list.append(dice)

        # bitwise_and_roi_patch = np.logical_and(np.squeeze(roi), np.squeeze(pred))
        # overlap = round(np.sum(bitwise_and_roi_patch) / np.sum(np.squeeze(roi)), 3)
      overlap = compute_overlap(roi, pred)
      overlap_list.append(overlap)

      iou = compute_iou(roi, pred)
      iou_list.append(iou)


In [ ]:
import pandas as pd
df = pd.DataFrame({'overlap': overlap_list, 'iou': iou_list, 'dice': dice_list })
df.to_csv ('/content/drive/MyDrive/test INBreast patch/overlap_iou_dice_inbreast_bb_850.csv', index = False, header=True)

#### metrics - hậu xử lý morphology

In [ ]:
csv_path = '/content/drive/MyDrive/test INBreast patch/patch_INBreast.csv'
predict_path = "/content/drive/MyDrive/test INBreast patch/Predict_standard_model"
data = pd.read_csv(csv_path)
inbreast_set_y_orig = data['roi']

# print(test_set_x_orig.shape)
dice_list = []
iou_list = []
overlap_list = []

for i in inbreast_set_y_orig:
      name_file = os.path.basename(i)
      roi = cv.imread(i)

      pred_path = os.path.join(predict_path,''.join([name_file]))
      pred = cv.imread(pred_path)

      kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
      pred = cv.morphologyEx(pred, cv.MORPH_CLOSE, kernel)
      pred = cv.morphologyEx(pred, cv.MORPH_OPEN, kernel)
      # pred = cv.threshold(pred, 1 , 255, cv.THRESH_BINARY)
      # pred = np.array(pred)
      # pred = cv.imdecode(pred, cv.IMREAD_GRAYSCALE)
      # pred = pred.reshape(-1, pred.shape[-1])




      dice = compute_dice_1(roi, pred)
      dice_list.append(dice)

      overlap = compute_overlap(roi, pred)
      overlap_list.append(overlap)

      iou = compute_iou(roi, pred)
      iou_list.append(iou)


In [ ]:
import pandas as pd
df = pd.DataFrame({'overlap': overlap_list, 'iou': iou_list, 'dice': dice_list })
df.to_csv ('/content/drive/MyDrive/test INBreast patch/overlap_iou_dice_inbreast_bb_850_morphology.csv', index = False, header=True)

#### model without 2nd skip connection

In [ ]:
def delete_bkg(patch_img_folder_path):
    # name_patch_roi = []
    for r, d, f in os.walk(patch_img_folder_path):
        for file in f:
            if '_bkg.png' in file:
                name_file = os.path.basename(os.path.join(r, file))
                path = patch_img_folder_path + "//" + name_file
                os.remove(path)
    #             name_patch_roi.append(name_file)
    # name_patch_img = []
    # for r, d, f in os.walk(patch_img_folder_path):
    #     for file in f:
    #         if '.png' in file:
    #             name_file = os.path.basename(os.path.join(r, file))
    #             name_patch_img.append(name_file)

    # for i in name_patch_roi:
    #     if i not in name_patch_img:
    #         path = patch_roi_folder_path + "\\" + i
    #         os.remove(path)


In [ ]:
delete_bkg("/content/drive/MyDrive/test INBreast patch/Predict_mass patch_2nd")


In [ ]:
delete_bkg("/content/Extract mass patch/Patch")


In [ ]:
dataset_path = '/content'
test_x= sorted(glob(os.path.join(dataset_path, 'Extract mass patch', 'Patch', '*')))
test_y= sorted(glob(os.path.join(dataset_path, 'Extract mass patch', 'Mask', '*')))

import pandas as pd
df = pd.DataFrame({'img': test_x, 'roi': test_y})
df.to_csv ('/content/drive/MyDrive/test INBreast patch/mass_patch_INBreast.csv', index = False, header=True)

In [ ]:
create_h5_file1('/content/drive/MyDrive/test INBreast patch/mass_patch_INBreast.csv', '/content/drive/MyDrive/test INBreast patch/mass_patch_mask_INBreast.h5','img', 'roi')

In [ ]:
inbreast_set_x, inbreast_set_y = read_h5_file('/content/drive/MyDrive/test INBreast patch/mass_patch_mask_INBreast.h5')

In [ ]:
def compute_dice_mass(label_img, pred_img, p_threshold=0.5):
    p = pred_img.astype(np.float)
    l = label_img.astype(np.float)
    if p.max() > 127:
        p /= 255.
    if l.max() > 127:
        l /= 255.

    p = np.clip(p, 0, 1.0)
    l = np.clip(l, 0, 1.0)
    p[p > 0.5] = 1.0
    p[p < 0.5] = 0.0
    l[l > 0.5] = 1.0
    l[l < 0.5] = 0.0
    product = ((l.flatten())*(p.flatten())).sum()
    dice_num = 2 * product
    pred_sum = p.sum()
    label_sum = l.sum()
    dice_den = pred_sum + label_sum
    dice_val = 100*dice_num / dice_den
    return round(dice_val,2)
def compute_iou_mass(label_img, pred_img, p_threshold=0.5):
    p = pred_img.astype(np.float)
    l = label_img.astype(np.float)
    if p.max() >= 127:
        p /= 255.
    if l.max() >= 127:
        l /= 255.

    p = np.clip(p, 0, 1.0)
    l = np.clip(l, 0, 1.0)
    p[p >= 0.5] = 1.0
    p[p < 0.5] = 0.0
    l[l >= 0.5] = 1.0
    l[l < 0.5] = 0.0

    product = ((l.flatten())* (p.flatten())).sum()
    iou_num = product
    pred_sum = p.sum()
    label_sum = l.sum()
    iou_den = pred_sum + label_sum  - product
    iou = 100*iou_num / iou_den

    return round(iou,2)

def compute_overlap_mass(label_img, pred_img):
    p = pred_img.astype(np.float)
    l = label_img.astype(np.float)
    if p.max() >= 127:
        p /= 255.
    if l.max() >= 127:
        l /= 255.

    p = np.clip(p, 0, 1.0)
    l = np.clip(l, 0, 1.0)
    p[p >= 0.5] = 1.0
    p[p < 0.5] = 0.0
    l[l >= 0.5] = 1.0
    l[l < 0.5] = 0.0

    bitwise_and_roi_patch = np.logical_and(l, p).sum()
    overlap = 100*(bitwise_and_roi_patch + 1)/(np.sum(l))
    return round(overlap,2)


In [ ]:
from csv import DictReader
csv_path = '/content/drive/MyDrive/test INBreast patch/mass_patch_INBreast.csv'

predict_2nd_path = "/content/drive/MyDrive/test INBreast patch/Predict_mass patch_2nd"
predict_path = "/content/drive/MyDrive/test INBreast patch/test 1416 mass patch standard model/Predict_standard_model"
from pathlib import Path

dice_list = []
iou_list = []
overlap_list = []

dice_2nd_list = []
iou_2nd_list = []
overlap_2nd_list = []

with open(csv_path, 'r') as csvfile:
    csv_dict_reader = DictReader(csvfile)
    for row in csv_dict_reader:
        img_file = row['img']
        roi_file = row['roi']

        name_file_saver = os.path.basename(roi_file)

        pred_file= os.path.join(predict_path,''.join([name_file_saver]))
        pred_2nd_file= os.path.join(predict_2nd_path,''.join([name_file_saver]))

        roi = cv.imread(roi_file,0)

        kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))

        y_pred = cv.imread(pred_file, 0)
        print(pred_file)
        y_pred = cv.morphologyEx(y_pred, cv.MORPH_CLOSE, kernel)
        y_pred = cv.morphologyEx(y_pred, cv.MORPH_OPEN, kernel)
        # y_pred, n_2 = filter_FP(pred, 300)

        y_pred_2nd = cv.imread(pred_2nd_file, 0)
        y_pred_2nd = cv.morphologyEx(y_pred_2nd, cv.MORPH_CLOSE, kernel)
        y_pred_2nd = cv.morphologyEx(y_pred_2nd, cv.MORPH_OPEN, kernel)
        # y_pred_2nd, n_3 = filter_FP(pred, 300)

        dice = compute_dice_mass(roi, y_pred)
        dice_list.append(dice)

        overlap = compute_overlap_mass(roi, y_pred)
        overlap_list.append(overlap)

        iou = compute_iou_mass(roi, y_pred)
        iou_list.append(iou)

        dice_2nd = compute_dice_mass(roi, y_pred_2nd)
        dice_2nd_list.append(dice_2nd)

        overlap_2nd = compute_overlap_mass(roi, y_pred_2nd)
        overlap_2nd_list.append(overlap_2nd)

        iou_2nd = compute_iou_mass(roi, y_pred_2nd)
        iou_2nd_list.append(iou_2nd)


In [ ]:
import pandas as pd
df = pd.DataFrame({'overlap': overlap_list, 'iou': iou_list, 'dice': dice_list,
                   'overlap_2nd': overlap_2nd_list, 'iou_2nd': iou_2nd_list, 'dice_2nd': dice_2nd_list })
df.to_csv ('/content/drive/MyDrive/test INBreast patch/overlap_iou_dice_mass_patch_standard_2nd_inbreast_bb_850_morphology.csv', index = False, header=True)

# INBreast (mass and non-mass patch - 1416 mass patch, bb<= 850)

In [ ]:
!unrar x "//content/drive/MyDrive/test INBreast patch/mass and non mass patch/Extract mass and non-mass patch.rar"

In [ ]:
drive_path = '/content/drive/MyDrive'
dataset_path = '/content'
dataset_inbreast_path = '/content/INBreast roi 850'
files_dir = os.path.join(drive_path, 'Colab Notebooks', 'files_15_5_23', 'training_dataset_20_bkg')
model_file = os.path.join(files_dir,'unet_effb0-new-data-20_bkg-15_5_2023.h5')

test_x= sorted(glob(os.path.join(dataset_path, 'Extract mass and non-mass patch', 'Patch', '*')))
test_y= sorted(glob(os.path.join(dataset_path, 'Extract mass and non-mass patch', 'Mask', '*')))

import pandas as pd
df = pd.DataFrame({'img': test_x, 'roi': test_y})
df.to_csv ('/content/drive/MyDrive/test INBreast patch/mass and non mass patch/patch_INBreast.csv', index = False, header=True)

In [ ]:
create_h5_file1('/content/drive/MyDrive/test INBreast patch/mass and non mass patch/patch_INBreast.csv',
                '/content/drive/MyDrive/test INBreast patch/mass and non mass patch/patch_mask_INBreast.h5','img', 'roi')

In [ ]:
from IPython.display import clear_output
clear_output()

In [ ]:
inbreast_set_x, inbreast_set_y = read_h5_file('/content/drive/MyDrive/test INBreast patch/mass and non mass patch/patch_mask_INBreast.h5')

In [ ]:
def compute_dice(label_img, pred_img, p_threshold=0.5):
    p = pred_img.astype(np.float)
    l = label_img.astype(np.float)
    if p.max() > 127:
        p /= 255.
    if l.max() > 127:
        l /= 255.

    p = np.clip(p, 0, 1.0)
    l = np.clip(l, 0, 1.0)
    p[p > 0.5] = 1.0
    p[p < 0.5] = 0.0
    l[l > 0.5] = 1.0
    l[l < 0.5] = 0.0
    product = ((l.flatten())*(p.flatten())).sum()
    dice_num = 2 * product + 1
    pred_sum = p.sum()
    label_sum = l.sum()
    dice_den = pred_sum + label_sum + 1
    dice_val = 100*dice_num / dice_den
    return round(dice_val,2)
def compute_iou(label_img, pred_img, p_threshold=0.5):
    p = pred_img.astype(np.float)
    l = label_img.astype(np.float)
    if p.max() >= 127:
        p /= 255.
    if l.max() >= 127:
        l /= 255.

    p = np.clip(p, 0, 1.0)
    l = np.clip(l, 0, 1.0)
    p[p >= 0.5] = 1.0
    p[p < 0.5] = 0.0
    l[l >= 0.5] = 1.0
    l[l < 0.5] = 0.0

    product = ((l.flatten())* (p.flatten())).sum()
    iou_num = product + 1
    pred_sum = p.sum()
    label_sum = l.sum()
    iou_den = pred_sum + label_sum + 1 - product
    iou = 100*iou_num / iou_den

    return round(iou,2)

def compute_overlap_1(label_img, pred_img):
    p = pred_img.astype(np.float)
    l = label_img.astype(np.float)
    if p.max() >= 127:
        p /= 255.
    if l.max() >= 127:
        l /= 255.

    p = np.clip(p, 0, 1.0)
    l = np.clip(l, 0, 1.0)
    p[p >= 0.5] = 1.0
    p[p < 0.5] = 0.0
    l[l >= 0.5] = 1.0
    l[l < 0.5] = 0.0

    # product = ((l.flatten())* (p.flatten())).sum()
    # label_sum = l.sum()
    # overlap = 100*product /label_sum

    # bitwise_and_roi_patch = np.logical_and(label_img, pred_img)

    if (np.sum(l) == 0):
      if (np.sum(p) != 0):
          overlap = 0.0
      else:
          bitwise_and_roi_patch = np.logical_and(l, p).sum()
          overlap = 100*(bitwise_and_roi_patch + 1)/(np.sum(l)+1)
    else:
          bitwise_and_roi_patch = np.logical_and(l, p).sum()
          overlap = 100*(bitwise_and_roi_patch + 1)/(np.sum(l)+1)
    return round(overlap,2)

#### Standard model

In [ ]:
load_model_file = tf.keras.models.load_model(model_file)

In [ ]:
preds_inbreast = load_model_file.predict(inbreast_set_x, verbose=1)
inbreast_y_pred = (preds_inbreast > 0.5).astype(np.float32)

In [ ]:
from csv import DictReader
csv_path = '/content/drive/MyDrive/test INBreast patch/mass and non mass patch/patch_INBreast.csv'
predict_path = "/content/drive/MyDrive/test INBreast patch/mass and non mass patch/Standard Model/Predict"
data = pd.read_csv(csv_path)
test_set_x_orig = data['img']

# print(test_set_x_orig.shape)
n = 0
for i in test_set_x_orig:
      name_file = os.path.basename(i)
      # print(name_file)
      save_path = os.path.join(predict_path,''.join([name_file]))
      # print(save_path)
      predict = np.squeeze(inbreast_y_pred[n])
      plt.imsave(save_path, predict, cmap='gray')
      # plt.imshow(predict)
      # plt.show
      # cv.imwrite(save_path, predict)
      n = n + 1
print(n)
print("Done")

##### show predict

In [ ]:
show_image_roi_predict1_img(66, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(86, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(65, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(42, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(1, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(15, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(20, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(40, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(60, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(80, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(100, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(120, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(140, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(150, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(200, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(280, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(300, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(350, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(351, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(349, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(370, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(380, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(400, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(500, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(550, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(600, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(700, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(800, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(900, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(1000, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(1100, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(1120, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(1300, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(1400, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(1500, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(1600, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(1700, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(1800, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(1900, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(2000, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(2100, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(2200, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(2797, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(2807, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(6, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(5, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(2583, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
show_image_roi_predict1_img(7, inbreast_set_x, inbreast_set_y, inbreast_y_pred)

In [ ]:
randomlist = random.sample(range(0, 100),5)
print(randomlist)
show_image_roi_predict(randomlist, inbreast_set_x, inbreast_set_y, inbreast_y_pred, 'inbreast')

In [ ]:
randomlist = random.sample(range(100, 200),5)
print(randomlist)
show_image_roi_predict(randomlist, inbreast_set_x, inbreast_set_y, inbreast_y_pred, 'inbreast')

In [ ]:
randomlist = random.sample(range(0, 2800),5)
print(randomlist)
show_image_roi_predict(randomlist, inbreast_set_x, inbreast_set_y, inbreast_y_pred, 'inbreast')

In [ ]:
randomlist = random.sample(range(0, 2800),5)
print(randomlist)
show_image_roi_predict(randomlist, inbreast_set_x, inbreast_set_y, inbreast_y_pred, 'inbreast')

In [ ]:
y_label, y_pred_label = create_label_map(inbreast_set_y,inbreast_y_pred )

In [ ]:
import seaborn as sn
from sklearn.metrics import confusion_matrix

# y_pred_2 = test_set_y_pred.ravel()
# test_set_y_2 = test_set_y.ravel()
cm=confusion_matrix(y_label, y_pred_label)
print(cm)
df_cm = pd.DataFrame(cm, index = (0, 1), columns = (0, 1))
plt.figure(figsize = (10,7))
plt.title('confusion matrix - Test data')
sn.set(font_scale=1.4)
sn.heatmap(df_cm, annot=True, fmt='g')

In [ ]:
total1=sum(sum(cm))
#####from confusion matrix calculate accuracy
accuracy1=(cm[0,0]+cm[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cm[0,0]/(cm[0,0]+cm[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cm[1,1]/(cm[1,0]+cm[1,1])
print('Specificity : ', specificity1)

In [ ]:
overlaps = []
labels_y = []
labels_y_pred = []
for i in range(len(inbreast_set_x)):
  x = inbreast_set_x[i]
  y = inbreast_set_y[i]
  y_pred = inbreast_y_pred[i] ,
  overlap, label_y, label_y_pred = cal_overlap_label(x,y,y_pred)
  overlaps.append(overlap)
  labels_y.append(label_y)
  labels_y_pred.append(label_y_pred)

In [ ]:
import seaborn as sn
from sklearn.metrics import confusion_matrix

# y_pred_2 = test_set_y_pred.ravel()
# test_set_y_2 = test_set_y.ravel()
cm=confusion_matrix(labels_y, labels_y_pred)
print(cm)
df_cm = pd.DataFrame(cm, index = (0, 1), columns = (0, 1))
plt.figure(figsize = (10,7))
plt.title('confusion matrix - Test data')
sn.set(font_scale=1.4)
sn.heatmap(df_cm, annot=True, fmt='g')

In [ ]:
total1=sum(sum(cm))
#####from confusion matrix calculate accuracy
accuracy1=(cm[0,0]+cm[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cm[0,0]/(cm[0,0]+cm[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cm[1,1]/(cm[1,0]+cm[1,1])
print('Specificity : ', specificity1)

##### metrics - hậu xử lý morphology

In [ ]:
csv_path = '/content/drive/MyDrive/test INBreast patch/mass and non mass patch/patch_INBreast.csv'
predict_path = "/content/drive/MyDrive/test INBreast patch/mass and non mass patch/Standard Model/Predict"
data = pd.read_csv(csv_path)
inbreast_set_y_orig = data['roi']

# print(test_set_x_orig.shape)
dice_list = []
iou_list = []
overlap_list = []

for i in inbreast_set_y_orig:
      name_file = os.path.basename(i)
      roi = cv.imread(i,0)

      pred_path = os.path.join(predict_path,''.join([name_file]))
      pred = cv.imread(pred_path,0)

      kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
      pred = cv.morphologyEx(pred, cv.MORPH_CLOSE, kernel)
      pred = cv.morphologyEx(pred, cv.MORPH_OPEN, kernel)

      # dice, iou, overlap, label_y, label_y_pred, roi_num_cnt, num_y_pred = cal_dice_iou_overlap(roi, pred)

      dice = compute_dice(roi, pred)
      dice_list.append(dice)

      overlap = compute_overlap_1(roi, pred)
      overlap_list.append(overlap)

      iou = compute_iou(roi, pred)
      iou_list.append(iou)





In [ ]:
import pandas as pd
df = pd.DataFrame({'overlap': overlap_list, 'iou': iou_list, 'dice': dice_list })
df.to_csv ('/content/drive/MyDrive/test INBreast patch/mass and non mass patch/Standard Model/overlap_iou_dice_inbreast_bb_850_morphology.csv', index = False, header=True)

In [ ]:
# tính lại do overlap sai
import pandas as pd
df = pd.DataFrame({'overlap': overlap_list, 'iou': iou_list, 'dice': dice_list })
df.to_csv ('/content/drive/MyDrive/test INBreast patch/mass and non mass patch/Standard Model/overlap_iou_dice_inbreast_bb_850_morphology_5.csv', index = False, header=True)

#### Model without 2nd skip connection

In [ ]:
files_dir_2nd = os.path.join(drive_path, 'Colab Notebooks', 'files_15_5_23', 'training_dataset_20_bkg_unet_without_skip')
model_file_2nd = os.path.join(files_dir_2nd,'unet_effb0-new-data-20_bkg-without-skip.h5')
load_model_file_2nd = tf.keras.models.load_model(model_file_2nd)

In [ ]:
preds_inbreast_2nd = load_model_file_2nd.predict(inbreast_set_x, verbose=1)
inbreast_y_pred_2nd = (preds_inbreast_2nd > 0.5).astype(np.float32)

In [ ]:
from csv import DictReader
csv_path = '/content/drive/MyDrive/test INBreast patch/mass and non mass patch/patch_INBreast.csv'
predict_path = "/content/drive/MyDrive/test INBreast patch/mass and non mass patch/Model without 2nd skip connection/Predict"
data = pd.read_csv(csv_path)
test_set_x_orig = data['img']

# print(test_set_x_orig.shape)
n = 0
for i in test_set_x_orig:
      name_file = os.path.basename(i)
      # print(name_file)
      save_path = os.path.join(predict_path,''.join([name_file]))
      # print(save_path)
      predict = np.squeeze(inbreast_y_pred_2nd[n])
      plt.imsave(save_path, predict, cmap='gray')
      # plt.imshow(predict)
      # plt.show
      # cv.imwrite(save_path, predict)
      n = n + 1
print(n)
print("Done")

##### show predict

In [ ]:
show_image_roi_predict1_img(66, inbreast_set_x, inbreast_set_y, inbreast_y_pred_2nd)

In [ ]:
show_image_roi_predict1_img(45, inbreast_set_x, inbreast_set_y, inbreast_y_pred_2nd)

In [ ]:
show_image_roi_predict1_img(100, inbreast_set_x, inbreast_set_y, inbreast_y_pred_2nd)

In [ ]:
show_image_roi_predict1_img(200, inbreast_set_x, inbreast_set_y, inbreast_y_pred_2nd)

In [ ]:
show_image_roi_predict1_img(86, inbreast_set_x, inbreast_set_y, inbreast_y_pred_2nd)

In [ ]:
show_image_roi_predict1_img(65, inbreast_set_x, inbreast_set_y, inbreast_y_pred_2nd)

In [ ]:
show_image_roi_predict1_img(1, inbreast_set_x, inbreast_set_y, inbreast_y_pred_2nd)

In [ ]:
show_image_roi_predict1_img(15, inbreast_set_x, inbreast_set_y, inbreast_y_pred_2nd)

In [ ]:
show_image_roi_predict1_img(20, inbreast_set_x, inbreast_set_y, inbreast_y_pred_2nd)

In [ ]:
show_image_roi_predict1_img(80, inbreast_set_x, inbreast_set_y, inbreast_y_pred_2nd)

In [ ]:
show_image_roi_predict1_img(120, inbreast_set_x, inbreast_set_y, inbreast_y_pred_2nd)

In [ ]:
show_image_roi_predict1_img(300, inbreast_set_x, inbreast_set_y, inbreast_y_pred_2nd)

In [ ]:
show_image_roi_predict1_img(350, inbreast_set_x, inbreast_set_y, inbreast_y_pred_2nd)

In [ ]:
show_image_roi_predict1_img(500, inbreast_set_x, inbreast_set_y, inbreast_y_pred_2nd)

In [ ]:
show_image_roi_predict1_img(800, inbreast_set_x, inbreast_set_y, inbreast_y_pred_2nd)

In [ ]:
show_image_roi_predict1_img(900, inbreast_set_x, inbreast_set_y, inbreast_y_pred_2nd)

In [ ]:
show_image_roi_predict1_img(1400, inbreast_set_x, inbreast_set_y, inbreast_y_pred_2nd)

##### metrics

In [ ]:
csv_path = '/content/drive/MyDrive/test INBreast patch/mass and non mass patch/patch_INBreast.csv'
predict_path = "/content/drive/MyDrive/test INBreast patch/mass and non mass patch/Model without 2nd skip connection/Predict"
data = pd.read_csv(csv_path)
inbreast_set_y_orig = data['roi']

# print(test_set_x_orig.shape)
dice_list_2nd = []
iou_list_2nd = []
overlap_list_2nd = []

for i in inbreast_set_y_orig:
      name_file = os.path.basename(i)
      roi = cv.imread(i,0)

      pred_path = os.path.join(predict_path,''.join([name_file]))
      pred = cv.imread(pred_path,0)

      kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
      pred = cv.morphologyEx(pred, cv.MORPH_CLOSE, kernel)
      pred = cv.morphologyEx(pred, cv.MORPH_OPEN, kernel)

      dice = compute_dice(roi, pred)
      dice_list_2nd.append(dice)

      overlap = compute_overlap_1(roi, pred)
      overlap_list_2nd.append(overlap)

      iou = compute_iou(roi, pred)
      iou_list_2nd.append(iou)





In [ ]:
# tính lại do overlap sai
import pandas as pd
df = pd.DataFrame({'overlap': overlap_list_2nd, 'iou': iou_list_2nd, 'dice': dice_list_2nd })
df.to_csv ('/content/drive/MyDrive/test INBreast patch/mass and non mass patch/Model without 2nd skip connection//overlap_iou_dice_inbreast_bb_850_morphology_2nd.csv', index = False, header=True)

In [ ]:
overlaps_2nd = []
labels_y_2nd = []
labels_y_pred_2nd = []
for i in range(len(inbreast_set_x)):
  x = inbreast_set_x[i]
  y = inbreast_set_y[i]
  y_pred = inbreast_y_pred_2nd[i] ,
  overlap, label_y, label_y_pred = cal_overlap_label(x,y,y_pred)
  overlaps_2nd.append(overlap)
  labels_y_2nd.append(label_y)
  labels_y_pred_2nd.append(label_y_pred)

In [ ]:
import seaborn as sn
from sklearn.metrics import confusion_matrix

# y_pred_2 = test_set_y_pred.ravel()
# test_set_y_2 = test_set_y.ravel()
cm=confusion_matrix(labels_y_2nd, labels_y_pred_2nd)
print(cm)
df_cm = pd.DataFrame(cm, index = (0, 1), columns = (0, 1))
plt.figure(figsize = (10,7))
plt.title('confusion matrix - Test data')
sn.set(font_scale=1.4)
sn.heatmap(df_cm, annot=True, fmt='g')

In [ ]:
total1=sum(sum(cm))
#####from confusion matrix calculate accuracy
accuracy1=(cm[0,0]+cm[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cm[0,0]/(cm[0,0]+cm[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cm[1,1]/(cm[1,0]+cm[1,1])
print('Specificity : ', specificity1)

#### so sánh 2 model

In [ ]:
def image_roi_predict_pred_2nd_overlap(index, x, y, predict, pred_2nd):
    # overlap = cal_overlap(x[index], y[index], predict[index])
    # print (overlap)

    roi = np.squeeze(y[index])
    # print(roi.dtype)
    image_8bit = np.uint8(roi)
    _, binarized = cv.threshold(image_8bit, 0, 255, cv.THRESH_BINARY)
    contours_roi, hierarchy = cv.findContours(binarized, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    kernel = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
    y_pred_filter = cv.morphologyEx(np.squeeze(predict[index]), cv.MORPH_CLOSE, kernel)
    y_pred_filter = cv.morphologyEx(y_pred_filter, cv.MORPH_OPEN, kernel)
    # roi_pred = np.squeeze(predict[index])
    # print(roi_pred.dtype)
    y_pred_filter = np.uint8(y_pred_filter)
    _, binarized = cv.threshold(y_pred_filter, 0, 255, cv.THRESH_BINARY)
    contours_pred, hierarchy = cv.findContours(binarized,cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    y_pred_filter_2nd = cv.morphologyEx(np.squeeze(pred_2nd[index]), cv.MORPH_CLOSE, kernel)
    y_pred_filter_2nd = cv.morphologyEx(y_pred_filter_2nd, cv.MORPH_OPEN, kernel)
    # roi_pred_2nd = np.squeeze(pred_2nd[index])
    # print(roi_pred_2nd.dtype)
    y_pred_filter_2nd = np.uint8(y_pred_filter_2nd)
    _, binarized_2nd = cv.threshold(y_pred_filter_2nd, 0, 255, cv.THRESH_BINARY)
    contours_pred_2nd, hierarchy = cv.findContours(binarized_2nd,cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    img = x[index]
    img_copy = img.copy()
    cv.drawContours(img_copy, contours_roi,-1,(255,0,0),2)
    cv.drawContours(img_copy, contours_pred,-1, (0,255,0), 2)
    cv.drawContours(img_copy, contours_pred_2nd,-1, (0,0,255), 2)
    plt.imshow(img_copy)
    plt.axis('off')
    plt.title('Image' + str(index))
    plt.show()

In [ ]:
image_roi_predict_pred_2nd_overlap(1, inbreast_set_x, inbreast_set_y, inbreast_y_pred, inbreast_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(10, inbreast_set_x, inbreast_set_y, inbreast_y_pred, inbreast_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(5, inbreast_set_x, inbreast_set_y, inbreast_y_pred, inbreast_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(6, inbreast_set_x, inbreast_set_y, inbreast_y_pred, inbreast_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(120, inbreast_set_x, inbreast_set_y, inbreast_y_pred, inbreast_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(86, inbreast_set_x, inbreast_set_y, inbreast_y_pred, inbreast_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(45, inbreast_set_x, inbreast_set_y, inbreast_y_pred, inbreast_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(66, inbreast_set_x, inbreast_set_y, inbreast_y_pred, inbreast_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(15, inbreast_set_x, inbreast_set_y, inbreast_y_pred, inbreast_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(20, inbreast_set_x, inbreast_set_y, inbreast_y_pred, inbreast_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(120, inbreast_set_x, inbreast_set_y, inbreast_y_pred, inbreast_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(3, inbreast_set_x, inbreast_set_y, inbreast_y_pred, inbreast_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(6, inbreast_set_x, inbreast_set_y, inbreast_y_pred, inbreast_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(39, inbreast_set_x, inbreast_set_y, inbreast_y_pred, inbreast_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(47, inbreast_set_x, inbreast_set_y, inbreast_y_pred, inbreast_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(212, inbreast_set_x, inbreast_set_y, inbreast_y_pred, inbreast_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(274, inbreast_set_x, inbreast_set_y, inbreast_y_pred, inbreast_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(275, inbreast_set_x, inbreast_set_y, inbreast_y_pred, inbreast_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(355, inbreast_set_x, inbreast_set_y, inbreast_y_pred, inbreast_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(433, inbreast_set_x, inbreast_set_y, inbreast_y_pred, inbreast_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(569, inbreast_set_x, inbreast_set_y, inbreast_y_pred, inbreast_y_pred_2nd)

In [ ]:
image_roi_predict_pred_2nd_overlap(579, inbreast_set_x, inbreast_set_y, inbreast_y_pred, inbreast_y_pred_2nd)